In [13]:
import requests
from collections import defaultdict
MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

def parse_polymarket_event_slug(slug: str) -> dict | None:
    """
    epl-wes-sun-2026-01-24
    epl-wes-sun-2026-01-24-more-markets
    """

    slug = slug.lower()

    market_family = "totals" if slug.endswith("-more-markets") else "game"
    core = slug.replace("-more-markets", "")
    parts = core.split("-")

    if len(parts) != 6:
        return None

    league, home, away, year, month, day = parts

    return {
        "league": league,
        "date": f"{year}-{month}-{day}",
        "home": home,
        "away": away,
        "market_family": market_family
    }

def parse_kalshi_event_id(event_id: str) -> dict | None:
    """
    KXLALIGAGAME-26JAN26GIRGET
    KXLALIGATOTALS-26JAN25BAROVI
    """

    if "-" not in event_id:
        return None

    prefix, payload = event_id.upper().split("-", 1)

    # remove KX
    prefix = prefix[2:]

    if prefix.endswith("GAME"):
        market_family = "game"
        league = prefix[:-4]
    elif prefix.endswith("TOTALS"):
        market_family = "totals"
        league = prefix[:-6]
    else:
        return None

    league = league.lower()

    if len(payload) < 13:
        return None

    home = payload[-6:-3].lower()
    away = payload[-3:].lower()

    date_block = payload[:-6]  # YYMMMDD
    year = "20" + date_block[:2]
    month = MONTH_MAP.get(date_block[2:5])
    day = date_block[5:7]


    if not month:
        return None

    return {
        "league": league,
        "date": f"{year}-{month}-{day}",
        "home": home,
        "away": away,
        "market_family": market_family
    }

def events_match(poly_evt: dict, kalshi_evt: dict) -> bool:
    return (
        poly_evt["date"] == kalshi_evt["date"]
        and poly_evt["market_family"] == kalshi_evt["market_family"]
        and {
            poly_evt["home"], poly_evt["away"]
        } == {
            kalshi_evt["home"], kalshi_evt["away"]
        }
    )


import requests
from collections import defaultdict

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

# -------- FETCH ALL SPORTS --------
response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    sport
    for sport in filters_data.get("sport_ordering", [])
    if sport != "All sports"
]

print(f"Found {len(all_sports)} sports: {all_sports}")

# This will replace the earlier kalshi_events_raw
kalshi_events_raw = []

# -------- LOOP OVER EACH SPORT --------
for sport in all_sports:
    print(f"\n========== SPORT: {sport} ==========")

    # -------- FETCH SERIES TICKERS FOR THIS SPORT --------
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]

    print(f"Found {len(sport_tickers)} {sport} series tickers")

    # -------- FETCH EVENTS FOR EACH SERIES TICKER --------
    for ticker in sport_tickers:
        cursor = None
        limit = 200

        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "with_nested_markets": True,
                "limit": limit,
                "with_milestones": True
            }
            if cursor:
                params["cursor"] = cursor

            response = requests.get(f"{KALSHI_API}/events", params=params)
            data = response.json()

            events = data.get("events", [])
            if not events:
                break

            kalshi_events_raw.extend(events)

            cursor = data.get("cursor")
            if not cursor:
                break

kalshi_events = []

for e in kalshi_events_raw:
    parsed = parse_kalshi_event_id(e.get("event_ticker", ""))
    if parsed:
        parsed["raw"] = e
        kalshi_events.append(parsed)

GAMMA_API = "https://gamma-api.polymarket.com"

response_3 = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response_3.json()

sport_series_all = {}

for item in sports_data_all:
    sport = item.get("sport")
    series = item.get("series")
    sport_id = item.get("id")

    if not sport:
        continue

    try:
        series = int(series)
        sport_id = int(sport_id)
    except (TypeError, ValueError):
        continue

    sport_series_all[sport] = {
        "sport": sport,
        "series": series,
        "id": sport_id
    }

serieses_all = [v["series"] for v in sport_series_all.values()]

events_by_series = defaultdict(list)

for series in serieses_all:
    page_id = 0
    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "tag_id": 100639,
                "order": "startTime",
                "ascending": True,
                "limit": 50,
                "offset": page_id * 50
            }
        )

        events = response.json()
        if not events:
            break

        events_by_series[series].extend(events)
        page_id += 1

poly_events = []

for events in events_by_series.values():
    for e in events:
        slug = e.get("slug")
        if not slug:
            continue

        parsed = parse_polymarket_event_slug(slug)
        if parsed:
            parsed["raw"] = e
            poly_events.append(parsed)

matches = []

for pe in poly_events:
    for ke in kalshi_events:
        if events_match(pe, ke):
            matches.append((pe, ke))

print(f"\nMATCHED EVENTS: {len(matches)}\n")

for pe, ke in matches:
    print("POLY :", pe)
    print("KALSH:", ke)
    print("-" * 60)


Found 15 sports: ['Football', 'Basketball', 'Hockey', 'Soccer', 'Tennis', 'Golf', 'MMA', 'Baseball', 'Boxing', 'Chess', 'Cricket', 'Esports', 'Motorsport', 'Olympics', 'Table Tennis']

========== SPORT: Football ==========
Found 235 Football series tickers

========== SPORT: Basketball ==========
Found 148 Basketball series tickers

========== SPORT: Hockey ==========
Found 38 Hockey series tickers

========== SPORT: Soccer ==========
Found 188 Soccer series tickers

========== SPORT: Tennis ==========
Found 72 Tennis series tickers

========== SPORT: Golf ==========
Found 30 Golf series tickers

========== SPORT: MMA ==========
Found 5 MMA series tickers

========== SPORT: Baseball ==========
Found 75 Baseball series tickers

========== SPORT: Boxing ==========
Found 7 Boxing series tickers

========== SPORT: Chess ==========
Found 10 Chess series tickers

========== SPORT: Cricket ==========
Found 12 Cricket series tickers

========== SPORT: Esports ==========
Found 87 Esports series

### Debugging

In [1]:
POLY_LEAGUE_TO_KALSHI_SPORT = {

    # ===================== SOCCER =====================
    "epl": "Soccer",
    "lal": "Soccer",
    "fl1": "Soccer",
    "bun": "Soccer",
    "sea": "Soccer",
    "ere": "Soccer",
    "por": "Soccer",
    "tur": "Soccer",
    "mls": "Soccer",
    "bra": "Soccer",
    "arg": "Soccer",
    "mex": "Soccer",
    "jap": "Soccer",
    "ja2": "Soccer",
    "kor": "Soccer",
    "spl": "Soccer",
    "chi": "Soccer",
    "aus": "Soccer",
    "ind": "Soccer",
    "nor": "Soccer",
    "den": "Soccer",
    "rus": "Soccer",
    "egy1": "Soccer",
    "cze1": "Soccer",

    # Cups & federations
    "ucl": "Soccer",
    "uel": "Soccer",
    "col": "Soccer",
    "cdr": "Soccer",
    "cde": "Soccer",
    "dfb": "Soccer",
    "efa": "Soccer",
    "efl": "Soccer",
    "ssc": "Soccer",
    "lcs": "Soccer",
    "lib": "Soccer",
    "sud": "Soccer",
    "caf": "Soccer",
    "afc": "Soccer",
    "uef": "Soccer",
    "cof": "Soccer",
    "con": "Soccer",
    "ofc": "Soccer",
    "fif": "Soccer",
    "acn": "Soccer",
    "mar1": "Soccer",

    # ===================== BASKETBALL =====================
    "nba": "Basketball",
    "wnba": "Basketball",
    "ncaab": "Basketball",
    "cbb": "Basketball",
    "cwbb": "Basketball",
    "bkcl": "Basketball",
    "bkligend": "Basketball",
    "bknbl": "Basketball",
    "bkseriea": "Basketball",
    "bkcba": "Basketball",
    "bkfr1": "Basketball",
    "bkarg": "Basketball",
    "bkkbl": "Basketball",

    # ===================== AMERICAN FOOTBALL =====================
    "nfl": "Football",
    "cfb": "Football",

    # ===================== BASEBALL =====================
    "mlb": "Baseball",
    "kbo": "Baseball",

    # ===================== HOCKEY =====================
    "nhl": "Hockey",
    "shl": "Hockey",
    "cehl": "Hockey",
    "dehl": "Hockey",
    "snhl": "Hockey",
    "khl": "Hockey",
    "ahl": "Hockey",

    # ===================== TENNIS =====================
    "atp": "Tennis",
    "wta": "Tennis",

    # ===================== CRICKET =====================
    "ipl": "Cricket",
    "odi": "Cricket",
    "t20": "Cricket",
    "test": "Cricket",
    "abb": "Cricket",
    "csa": "Cricket",
    "she": "Cricket",
    "sasa": "Cricket",
    "lpl": "Cricket",
    "psp": "Cricket",
    "crban": "Cricket",
    "crint": "Cricket",
    "crind": "Cricket",
    "crpak": "Cricket",
    "creng": "Cricket",
    "cruae": "Cricket",
    "crnew": "Cricket",
    "craus": "Cricket",
    "crsou": "Cricket",
    "cru19wc": "Cricket",
    "crwpl20": "Cricket",
    "crwncl": "Cricket",
    "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket",
    "crbtnmlyhkg20": "Cricket",

    # ===================== RUGBY =====================
    "ruprem": "Rugby",
    "rutopft": "Rugby",
    "rusixnat": "Rugby",
    "rueuchamp": "Rugby",
    "ruurc": "Rugby",
    "rusrp": "Rugby",
    "ruchamp": "Rugby",

    # ===================== MMA =====================
    "mma": "MMA",
    "zuffa": "MMA",

    # ===================== ESPORTS =====================
    "lol": "Esports",
    "dota2": "Esports",
    "cs2": "Esports",
    "val": "Esports",
    "sc2": "Esports",
    "sc": "Esports",
    "pubg": "Esports",
    "r6siege": "Esports",
    "rl": "Esports",
    "codmw": "Esports",
    "mlbb": "Esports",
    "ow": "Esports",
    "hok": "Esports",
    "wildrift": "Esports",
    "fifa": "Esports",
}


In [2]:
import requests
from collections import defaultdict

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# =========================
# PARSERS (UNCHANGED)
# =========================

def parse_polymarket_event_slug(slug: str):
    slug = slug.lower()

    market_family = "totals" if slug.endswith("-more-markets") else "game"
    core = slug.replace("-more-markets", "")
    parts = core.split("-")

    if len(parts) != 6:
        return None

    league, home, away, year, month, day = parts

    return {
        "league": league,
        "date": f"{year}-{month}-{day}",
        "home": home,
        "away": away,
        "market_family": market_family,
        "slug": slug,
    }

def parse_kalshi_event_id(event_id: str):
    if "-" not in event_id:
        return None

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return None

    prefix = prefix[2:]  # remove KX

    # Identify league and market family more flexibly
    market_family = None

    KNOWN_MARKET_SUFFIXES = [
        "GAME", "TOTALS", "SPREAD", "ALT", "LINES"
    ]

    for suffix in KNOWN_MARKET_SUFFIXES:
        if prefix.endswith(suffix):
            market_family = suffix.lower()
            league = prefix[:-len(suffix)]
            break
    else:
        # Not a matchup-style market (likely futures/props)
        return None

    # Payload must contain date + teams
    if len(payload) < 13:
        return None

    date_block = payload[:-6]  # YYMMMDD
    year = "20" + date_block[:2]
    month = MONTH_MAP.get(date_block[2:5])
    day = date_block[5:7]

    if not month:
        return None

    home = payload[-6:-3].lower()
    away = payload[-3:].lower()

    return {
        "league": league.lower(),
        "date": f"{year}-{month}-{day}",
        "home": home,
        "away": away,
        "market_family": market_family,
        "event_id": event_id,
    }


def events_match(poly_evt, kalshi_evt):
    return (
        poly_evt["date"] == kalshi_evt["date"]
        and poly_evt["market_family"] == kalshi_evt["market_family"]
        and {poly_evt["home"], poly_evt["away"]}
        == {kalshi_evt["home"], kalshi_evt["away"]}
    )

# =========================
# FETCH KALSHI EVENTS (BY SPORT)
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]

    for ticker in tickers:
        cursor = None
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event_id(e.get("event_ticker", ""))
                if parsed:
                    kalshi_events_by_sport[sport].append(parsed)

            cursor = r.get("cursor")
            if not cursor:
                break

# =========================
# FETCH & CLASSIFY POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []

for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()

    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    if not kalshi_sport:
        continue  # unmapped league

    valid_series.append((series, kalshi_sport))

print(f"Valid Polymarket series mapped: {len(valid_series)}")

# -------- FETCH EVENTS --------
for series, kalshi_sport in valid_series:
    page = 0
    limit = 50

    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "tag_id": 100639,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            slug = e.get("slug")
            if not slug:
                continue

            parsed = parse_polymarket_event_slug(slug)
            if parsed:
                parsed["raw"] = e
                poly_events_by_kalshi_sport[kalshi_sport].append(parsed)

        page += 1


# =========================
# DEBUG MATCHING BY SPORT
# =========================

print("\n================ MATCH DEBUG (BY SPORT) =================\n")

total_matches = 0

for sport in all_sports:
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])

    matches = []

    for ke in kalshi_evts:
        for pe in poly_evts:
            if events_match(pe, ke):
                matches.append((pe, ke))

    print(f"SPORT: {sport}")
    print(f"  Kalshi events     : {len(kalshi_evts)}")
    print(f"  Polymarket events : {len(poly_evts)}")
    print(f"  Matched events    : {len(matches)}")

    for pe, ke in matches:
        print(
            f"    ✓ {pe['date']} | "
            f"{pe['home']} vs {pe['away']} | "
            f"{pe['market_family']} | "
            f"POLY={pe['slug']} | "
            f"KALSHI={ke['event_id']}"
        )

    print("-" * 80)
    total_matches += len(matches)

print(f"\nTOTAL MATCHES ACROSS ALL SPORTS: {total_matches}")

Valid Polymarket series mapped: 119

================ MATCH DEBUG (BY SPORT) =================

SPORT: Football
  Kalshi events     : 0
  Polymarket events : 2
  Matched events    : 0
--------------------------------------------------------------------------------
SPORT: Basketball
  Kalshi events     : 222
  Polymarket events : 855
  Matched events    : 23
    ✓ 2026-01-25 | xin vs lia | game | POLY=bkcba-xin-lia-2026-01-25 | KALSHI=KXCBAGAME-26JAN25LIAXIN
    ✓ 2026-01-25 | nin vs jia | game | POLY=bkcba-nin-jia-2026-01-25 | KALSHI=KXCBAGAME-26JAN25JIANIN
    ✓ 2026-01-24 | tia vs jil | game | POLY=bkcba-tia-jil-2026-01-24 | KALSHI=KXCBAGAME-26JAN24JILTIA
    ✓ 2026-01-26 | obr vs rac | game | POLY=bkarg-obr-rac-2026-01-26 | KALSHI=KXARGLNBGAME-26JAN26RACOBR
    ✓ 2026-01-26 | dae vs son | game | POLY=bkkbl-dae-son-2026-01-26 | KALSHI=KXKBLGAME-26JAN26SONDAE
    ✓ 2026-01-24 | cha vs cho | game | POLY=bkfr1-cha-cho-2026-01-24 | KALSHI=KXLNBELITEGAME-26JAN24CHOCHA
    ✓ 2026-01-25 | b

In [3]:
import requests
from collections import defaultdict
import re

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# Map Polymarket league codes to Kalshi sport names
POLY_LEAGUE_TO_KALSHI_SPORT = {
    "nfl": "Football",
    "nba": "Basketball",
    "nhl": "Hockey",
    "mlb": "Baseball",
    "epl": "Soccer",
    "mma": "MMA",
    "ncaaf": "Football",
    "ncaab": "Basketball",
}

# =========================
# POLYMARKET PARSER
# =========================

def parse_polymarket_event_slug(slug: str):
    """Parse Polymarket slug and extract all available information"""
    slug_lower = slug.lower()
    
    # Detect market type from slug patterns
    market_type = "unknown"
    if slug_lower.endswith("-more-markets"):
        market_type = "totals"
    elif any(kw in slug_lower for kw in ["winner", "champion", "playoff"]):
        market_type = "futures"
    elif any(kw in slug_lower for kw in ["mvp", "award", "dpoy", "roty"]):
        market_type = "awards"
    else:
        market_type = "game"
    
    parts = slug_lower.replace("-more-markets", "").split("-")
    
    # Try to extract date (YYYY-MM-DD pattern in parts)
    date = None
    date_idx = None
    for i in range(len(parts) - 2):
        if (parts[i].isdigit() and len(parts[i]) == 4 and  # Year
            parts[i+1].isdigit() and len(parts[i+1]) <= 2 and  # Month
            parts[i+2].isdigit() and len(parts[i+2]) <= 2):  # Day
            date = f"{parts[i]}-{parts[i+1].zfill(2)}-{parts[i+2].zfill(2)}"
            date_idx = i
            break
    
    league = parts[0] if parts else None
    
    # Extract teams if this is a game-type market
    home = None
    away = None
    if date_idx and date_idx >= 3:
        team_parts = parts[1:date_idx]
        if len(team_parts) >= 2:
            mid = len(team_parts) // 2
            home = "-".join(team_parts[:mid])
            away = "-".join(team_parts[mid:])
    
    return {
        "league": league,
        "date": date,
        "home": home,
        "away": away,
        "market_type": market_type,
        "slug": slug,
        "slug_normalized": slug_lower.replace("-more-markets", ""),
    }

# =========================
# KALSHI PARSER
# =========================

def parse_kalshi_event_id(event_id: str, event_data: dict = None):
    """Parse Kalshi event ticker and extract all information including title"""
    if "-" not in event_id:
        return None

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return None

    prefix = prefix[2:]  # remove KX
    
    # Extract league and scope using known patterns
    league = None
    scope = None
    
    scope_patterns = [
        ("GAME", "games"),
        ("TOTALS", "totals"),
        ("SPREAD", "spread"),
        ("ALT", "alt"),
        ("LINES", "lines"),
        ("FUT", "futures"),
        ("FUTURES", "futures"),
        ("WINS", "wins"),
        ("WINTOTALS", "wintotals"),
        ("AWARDS", "awards"),
        ("MVP", "awards"),
        ("DPOY", "awards"),
        ("ROTY", "awards"),
        ("DRAFT", "draft"),
        ("LEADER", "leader"),
        ("ADVANCE", "advance"),
        ("EVENT", "events"),
    ]
    
    for pattern, scope_name in scope_patterns:
        if prefix.endswith(pattern):
            scope = scope_name
            league = prefix[:-len(pattern)]
            break
    
    if not scope:
        scope = "unknown"
        league = prefix
    
    # Parse date and teams for game-like events
    date = None
    home = None
    away = None
    
    # Try to extract date (YYMMMDD format)
    if len(payload) >= 7:
        date_block = payload[:7]
        year_part = date_block[:2]
        month_part = date_block[2:5]
        day_part = date_block[5:7]
        
        if month_part in MONTH_MAP and day_part.isdigit():
            year = "20" + year_part
            month = MONTH_MAP[month_part]
            day = day_part
            date = f"{year}-{month}-{day}"
            
            # Extract teams (remaining payload after date)
            team_block = payload[7:]
            if len(team_block) >= 6:
                # Common pattern: last 6 chars are two 3-letter team codes
                home = team_block[-6:-3].lower()
                away = team_block[-3:].lower()
    
    # Extract title if available from event_data
    title = None
    if event_data:
        title = event_data.get("title", "")
    
    return {
        "league": league.lower() if league else None,
        "date": date,
        "home": home,
        "away": away,
        "scope": scope,
        "event_id": event_id,
        "title": title,
        "raw_payload": payload,
    }

# =========================
# MATCHING STRATEGIES
# =========================

def normalize_team(team: str) -> str:
    """Normalize team name for comparison"""
    if not team:
        return ""
    return team.lower().replace("-", "").replace(" ", "").replace(".", "")

def team_sets_match(poly_evt, kalshi_evt) -> bool:
    """Check if team sets match between events"""
    poly_teams = {
        normalize_team(poly_evt.get("home", "")),
        normalize_team(poly_evt.get("away", ""))
    }
    kalshi_teams = {
        normalize_team(kalshi_evt.get("home", "")),
        normalize_team(kalshi_evt.get("away", ""))
    }
    
    poly_teams.discard("")
    kalshi_teams.discard("")
    
    if not (poly_teams and kalshi_teams):
        return False
    
    return poly_teams == kalshi_teams

def match_games(poly_evt, kalshi_evt) -> bool:
    """Match game/totals type events"""
    # Must have matching dates
    if not (poly_evt.get("date") and kalshi_evt.get("date")):
        return False
    
    if poly_evt["date"] != kalshi_evt["date"]:
        return False
    
    # Must have matching teams
    return team_sets_match(poly_evt, kalshi_evt)

def match_by_title_similarity(poly_evt, kalshi_evt) -> bool:
    """Fallback: try to match based on title/slug text similarity"""
    poly_text = poly_evt.get("slug_normalized", "")
    kalshi_title = kalshi_evt.get("title", "").lower()
    
    if not (poly_text and kalshi_title):
        return False
    
    # Extract key terms from both
    poly_terms = set(poly_text.split("-"))
    kalshi_terms = set(re.findall(r'\w+', kalshi_title))
    
    # Check for significant overlap
    common_terms = poly_terms & kalshi_terms
    
    # Need at least 3 common meaningful terms
    common_terms = {t for t in common_terms if len(t) > 2}
    
    return len(common_terms) >= 3

def events_match(poly_evt, kalshi_evt) -> tuple[bool, str]:
    """
    Determine if Polymarket and Kalshi events match.
    Returns (matched: bool, reason: str)
    """
    
    # Strategy 1: Game/totals with date and teams
    if kalshi_evt.get("scope") in ["games", "totals", "spread", "alt", "lines", "events"]:
        if match_games(poly_evt, kalshi_evt):
            return True, "game_match"
    
    # Strategy 2: Title/slug similarity (for futures, awards, etc.)
    if match_by_title_similarity(poly_evt, kalshi_evt):
        return True, "title_similarity"
    
    return False, "no_match"

# =========================
# FETCH KALSHI EVENTS
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

print("="*80)
print("FETCHING KALSHI EVENTS")
print("="*80 + "\n")

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    print(f"Fetching {sport}...")
    
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]
    
    for ticker in tickers:
        cursor = None
        
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event_id(
                    e.get("event_ticker", ""),
                    event_data=e
                )
                if parsed:
                    kalshi_events_by_sport[sport].append(parsed)

            cursor = r.get("cursor")
            if not cursor:
                break

# =========================
# FETCH POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

print("\n" + "="*80)
print("FETCHING POLYMARKET EVENTS")
print("="*80 + "\n")

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []

for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()
    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    
    if not kalshi_sport:
        continue

    valid_series.append((series, kalshi_sport))

print(f"Found {len(valid_series)} valid Polymarket series\n")

for series, kalshi_sport in valid_series:
    page = 0
    limit = 50

    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "tag_id": 100639,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            slug = e.get("slug")
            if not slug:
                continue

            parsed = parse_polymarket_event_slug(slug)
            if parsed:
                parsed["raw"] = e
                poly_events_by_kalshi_sport[kalshi_sport].append(parsed)

        page += 1

# =========================
# ANALYZE DATA BEFORE MATCHING
# =========================

print("\n" + "="*80)
print("DATA ANALYSIS")
print("="*80 + "\n")

# Analyze Kalshi scopes
print("KALSHI SCOPE DISTRIBUTION:")
kalshi_scope_counts = defaultdict(int)
for sport, events in kalshi_events_by_sport.items():
    for evt in events:
        kalshi_scope_counts[evt.get("scope", "unknown")] += 1

for scope, count in sorted(kalshi_scope_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {scope}: {count} events")

# Analyze Polymarket market types
print("\nPOLYMARKET MARKET TYPE DISTRIBUTION:")
poly_market_counts = defaultdict(int)
for sport, events in poly_events_by_kalshi_sport.items():
    for evt in events:
        poly_market_counts[evt.get("market_type", "unknown")] += 1

for mtype, count in sorted(poly_market_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {mtype}: {count} events")

# Show examples of each Kalshi scope
print("\n" + "="*80)
print("KALSHI SCOPE EXAMPLES")
print("="*80 + "\n")

scope_examples = defaultdict(list)
for sport, events in kalshi_events_by_sport.items():
    for evt in events:
        scope = evt.get("scope", "unknown")
        if len(scope_examples[scope]) < 5:
            scope_examples[scope].append(evt)

for scope in sorted(scope_examples.keys()):
    print(f"\n{scope.upper()}:")
    for evt in scope_examples[scope]:
        print(f"  {evt['event_id']}")
        if evt.get('title'):
            print(f"    Title: {evt['title']}")
        if evt.get('date'):
            print(f"    Date: {evt['date']}")
        if evt.get('home') and evt.get('away'):
            print(f"    Teams: {evt['home']} vs {evt['away']}")

# =========================
# MATCHING
# =========================

print("\n" + "="*80)
print("MATCHING EVENTS")
print("="*80 + "\n")

total_matches = 0
matches_by_strategy = defaultdict(int)

for sport in all_sports:
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])
    
    if not (kalshi_evts and poly_evts):
        continue
    
    matches = []
    
    for ke in kalshi_evts:
        for pe in poly_evts:
            matched, reason = events_match(pe, ke)
            if matched:
                matches.append((pe, ke, reason))
                matches_by_strategy[reason] += 1
                break  # Each Kalshi event matches at most one Polymarket event
    
    if matches:
        print(f"\nSPORT: {sport}")
        print(f"  Kalshi events: {len(kalshi_evts)}")
        print(f"  Polymarket events: {len(poly_evts)}")
        print(f"  Matches: {len(matches)}")
        
        for pe, ke, reason in matches[:3]:
            print(f"\n    [{reason}]")
            print(f"    POLY: {pe['slug']}")
            print(f"    KALSHI: {ke['event_id']}")
            if ke.get('title'):
                print(f"      Title: {ke['title']}")
        
        if len(matches) > 3:
            print(f"\n    ... and {len(matches) - 3} more matches")
        
        print("-" * 80)
        total_matches += len(matches)

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nTotal matches: {total_matches}")
print("\nMatches by strategy:")
for strategy, count in sorted(matches_by_strategy.items(), key=lambda x: x[1], reverse=True):
    print(f"  {strategy}: {count}")
print()

FETCHING KALSHI EVENTS

Fetching Football...
Fetching Basketball...
Fetching Hockey...
Fetching Soccer...
Fetching Tennis...
Fetching Golf...
Fetching MMA...
Fetching Baseball...
Fetching Boxing...
Fetching Chess...
Fetching Cricket...
Fetching Esports...
Fetching Motorsport...
Fetching Olympics...

FETCHING POLYMARKET EVENTS

Found 7 valid Polymarket series


DATA ANALYSIS

KALSHI SCOPE DISTRIBUTION:
  unknown: 763 events
  games: 717 events
  spread: 247 events
  wins: 60 events
  awards: 9 events
  leader: 1 events

POLYMARKET MARKET TYPE DISTRIBUTION:
  game: 186 events
  totals: 17 events

KALSHI SCOPE EXAMPLES


AWARDS:
  KXNFLOROTY-26
    Title: Offensive Rookie of the Year Winner?
  KXNFLDROTY-26
    Title: Defensive Rookie of the Year Winner?
  KXNFLMVP-26
    Title: MVP winner?
  KXNFLSBMVP-26
    Title: Pro Football Championship MVP?
  KXNBAMVP-26
    Title: MVP Winner?

GAMES:
  KXFIBAECUPGAME-26JAN21ZARPEA
    Title: Basket Zaragoza 2002 at Peristeri Athens
    Date: 2026-

In [5]:
import requests
from collections import defaultdict
import re

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# Map Polymarket league codes to Kalshi sport names
POLY_LEAGUE_TO_KALSHI_SPORT = {

    # ===================== SOCCER =====================
    "epl": "Soccer",
    "lal": "Soccer",
    "fl1": "Soccer",
    "bun": "Soccer",
    "sea": "Soccer",
    "ere": "Soccer",
    "por": "Soccer",
    "tur": "Soccer",
    "mls": "Soccer",
    "bra": "Soccer",
    "arg": "Soccer",
    "mex": "Soccer",
    "jap": "Soccer",
    "ja2": "Soccer",
    "kor": "Soccer",
    "spl": "Soccer",
    "chi": "Soccer",
    "aus": "Soccer",
    "ind": "Soccer",
    "nor": "Soccer",
    "den": "Soccer",
    "rus": "Soccer",
    "egy1": "Soccer",
    "cze1": "Soccer",

    # Cups & federations
    "ucl": "Soccer",
    "uel": "Soccer",
    "col": "Soccer",
    "cdr": "Soccer",
    "cde": "Soccer",
    "dfb": "Soccer",
    "efa": "Soccer",
    "efl": "Soccer",
    "ssc": "Soccer",
    "lcs": "Soccer",
    "lib": "Soccer",
    "sud": "Soccer",
    "caf": "Soccer",
    "afc": "Soccer",
    "uef": "Soccer",
    "cof": "Soccer",
    "con": "Soccer",
    "ofc": "Soccer",
    "fif": "Soccer",
    "acn": "Soccer",
    "mar1": "Soccer",

    # ===================== BASKETBALL =====================
    "nba": "Basketball",
    "wnba": "Basketball",
    "ncaab": "Basketball",
    "cbb": "Basketball",
    "cwbb": "Basketball",
    "bkcl": "Basketball",
    "bkligend": "Basketball",
    "bknbl": "Basketball",
    "bkseriea": "Basketball",
    "bkcba": "Basketball",
    "bkfr1": "Basketball",
    "bkarg": "Basketball",
    "bkkbl": "Basketball",

    # ===================== AMERICAN FOOTBALL =====================
    "nfl": "Football",
    "cfb": "Football",

    # ===================== BASEBALL =====================
    "mlb": "Baseball",
    "kbo": "Baseball",

    # ===================== HOCKEY =====================
    "nhl": "Hockey",
    "shl": "Hockey",
    "cehl": "Hockey",
    "dehl": "Hockey",
    "snhl": "Hockey",
    "khl": "Hockey",
    "ahl": "Hockey",

    # ===================== TENNIS =====================
    "atp": "Tennis",
    "wta": "Tennis",

    # ===================== CRICKET =====================
    "ipl": "Cricket",
    "odi": "Cricket",
    "t20": "Cricket",
    "test": "Cricket",
    "abb": "Cricket",
    "csa": "Cricket",
    "she": "Cricket",
    "sasa": "Cricket",
    "lpl": "Cricket",
    "psp": "Cricket",
    "crban": "Cricket",
    "crint": "Cricket",
    "crind": "Cricket",
    "crpak": "Cricket",
    "creng": "Cricket",
    "cruae": "Cricket",
    "crnew": "Cricket",
    "craus": "Cricket",
    "crsou": "Cricket",
    "cru19wc": "Cricket",
    "crwpl20": "Cricket",
    "crwncl": "Cricket",
    "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket",
    "crbtnmlyhkg20": "Cricket",

    # ===================== RUGBY =====================
    "ruprem": "Rugby",
    "rutopft": "Rugby",
    "rusixnat": "Rugby",
    "rueuchamp": "Rugby",
    "ruurc": "Rugby",
    "rusrp": "Rugby",
    "ruchamp": "Rugby",

    # ===================== MMA =====================
    "mma": "MMA",
    "zuffa": "MMA",

    # ===================== ESPORTS =====================
    "lol": "Esports",
    "dota2": "Esports",
    "cs2": "Esports",
    "val": "Esports",
    "sc2": "Esports",
    "sc": "Esports",
    "pubg": "Esports",
    "r6siege": "Esports",
    "rl": "Esports",
    "codmw": "Esports",
    "mlbb": "Esports",
    "ow": "Esports",
    "hok": "Esports",
    "wildrift": "Esports",
    "fifa": "Esports",
}


# =========================
# POLYMARKET PARSER
# =========================

def parse_polymarket_event_slug(slug: str):
    """Parse Polymarket slug and extract all available information"""
    slug_lower = slug.lower()
    original_slug = slug_lower
    
    # Detect market subtype from slug patterns
    market_subtype = "moneyline"  # default
    if slug_lower.endswith("-more-markets"):
        market_subtype = "totals"
        slug_lower = slug_lower.replace("-more-markets", "")
    elif "spread" in slug_lower:
        market_subtype = "spread"
    
    # Detect if this is a game-based market or futures/awards
    is_game_based = True
    if any(kw in slug_lower for kw in ["winner", "champion", "playoff", "title", "trophy"]):
        is_game_based = False
        market_subtype = "futures"
    elif any(kw in slug_lower for kw in ["mvp", "award", "dpoy", "roty", "opoy"]):
        is_game_based = False
        market_subtype = "awards"
    
    parts = slug_lower.split("-")
    
    # Try to extract date (YYYY-MM-DD pattern in parts)
    date = None
    date_idx = None
    for i in range(len(parts) - 2):
        if (parts[i].isdigit() and len(parts[i]) == 4 and  # Year
            parts[i+1].isdigit() and len(parts[i+1]) <= 2 and  # Month
            parts[i+2].isdigit() and len(parts[i+2]) <= 2):  # Day
            date = f"{parts[i]}-{parts[i+1].zfill(2)}-{parts[i+2].zfill(2)}"
            date_idx = i
            break
    
    league = parts[0] if parts else None
    
    # Extract teams if this is a game-type market
    home = None
    away = None
    if is_game_based and date_idx and date_idx >= 3:
        team_parts = parts[1:date_idx]
        if len(team_parts) >= 2:
            mid = len(team_parts) // 2
            home = "-".join(team_parts[:mid])
            away = "-".join(team_parts[mid:])
    
    return {
        "league": league,
        "date": date,
        "home": home,
        "away": away,
        "is_game_based": is_game_based,
        "market_subtype": market_subtype,
        "slug": original_slug,
    }

# =========================
# KALSHI PARSER
# =========================

def parse_kalshi_event_id(event_id: str, event_data: dict = None):
    """Parse Kalshi event ticker and extract all information"""
    if "-" not in event_id:
        return None

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return None

    prefix = prefix[2:]  # remove KX
    
    # Extract league and scope
    league = None
    scope = None
    market_subtype = None
    
    # Game-based scope patterns
    game_scope_patterns = [
        ("GAME", "game", "moneyline"),
        ("TOTALS", "game", "totals"),
        ("SPREAD", "game", "spread"),
        ("ALT", "game", "alt"),
        ("LINES", "game", "lines"),
        ("EVENT", "game", "moneyline"),
    ]
    
    # Non-game scope patterns
    non_game_scope_patterns = [
        ("FUT", "futures", "futures"),
        ("FUTURES", "futures", "futures"),
        ("WINS", "futures", "wins"),
        ("WINTOTALS", "futures", "wins"),
        ("AWARDS", "futures", "awards"),
        ("MVP", "futures", "awards"),
        ("DPOY", "futures", "awards"),
        ("ROTY", "futures", "awards"),
        ("DRAFT", "futures", "draft"),
        ("LEADER", "futures", "leader"),
        ("ADVANCE", "futures", "advance"),
    ]
    
    is_game_based = False
    
    # Check game patterns first
    for pattern, scope_type, subtype in game_scope_patterns:
        if prefix.endswith(pattern):
            scope = scope_type
            market_subtype = subtype
            league = prefix[:-len(pattern)]
            is_game_based = True
            break
    
    # If not game-based, check non-game patterns
    if not is_game_based:
        for pattern, scope_type, subtype in non_game_scope_patterns:
            if prefix.endswith(pattern):
                scope = scope_type
                market_subtype = subtype
                league = prefix[:-len(pattern)]
                break
    
    if not scope:
        scope = "unknown"
        market_subtype = "unknown"
        league = prefix
    
    # Parse date and teams (primarily for game-based events)
    date = None
    home = None
    away = None
    
    if is_game_based and len(payload) >= 7:
        date_block = payload[:7]
        year_part = date_block[:2]
        month_part = date_block[2:5]
        day_part = date_block[5:7]
        
        if month_part in MONTH_MAP and day_part.isdigit():
            year = "20" + year_part
            month = MONTH_MAP[month_part]
            day = day_part
            date = f"{year}-{month}-{day}"
            
            # Extract teams
            team_block = payload[7:]
            if len(team_block) >= 6:
                home = team_block[-6:-3].lower()
                away = team_block[-3:].lower()
    
    # Extract title if available
    title = ""
    if event_data:
        title = event_data.get("title", "")
    
    return {
        "league": league.lower() if league else None,
        "date": date,
        "home": home,
        "away": away,
        "is_game_based": is_game_based,
        "scope": scope,
        "market_subtype": market_subtype,
        "event_id": event_id,
        "title": title,
    }

# =========================
# MATCHING FUNCTIONS
# =========================

def normalize_team(team: str) -> str:
    """Normalize team name for comparison"""
    if not team:
        return ""
    return team.lower().replace("-", "").replace(" ", "").replace(".", "")

def create_game_key(date: str, team1: str, team2: str) -> str:
    """Create a normalized key for a game (date + sorted teams)"""
    teams = sorted([normalize_team(team1), normalize_team(team2)])
    return f"{date}|{'|'.join(teams)}"

def match_game_based_events(poly_events, kalshi_events):
    """
    Match game-based events by date + teams, then group by market subtype.
    Returns: dict[game_key] -> dict[market_subtype] -> list of (poly_evt, kalshi_evt) pairs
    """
    # Index Kalshi events by game key
    kalshi_by_game = defaultdict(lambda: defaultdict(list))
    for ke in kalshi_events:
        if not ke.get("is_game_based"):
            continue
        if not (ke.get("date") and ke.get("home") and ke.get("away")):
            continue
        
        game_key = create_game_key(ke["date"], ke["home"], ke["away"])
        subtype = ke.get("market_subtype", "unknown")
        kalshi_by_game[game_key][subtype].append(ke)
    
    # Index Polymarket events by game key
    poly_by_game = defaultdict(lambda: defaultdict(list))
    for pe in poly_events:
        if not pe.get("is_game_based"):
            continue
        if not (pe.get("date") and pe.get("home") and pe.get("away")):
            continue
        
        game_key = create_game_key(pe["date"], pe["home"], pe["away"])
        subtype = pe.get("market_subtype", "unknown")
        poly_by_game[game_key][subtype].append(pe)
    
    # Create matches grouped by game and market subtype
    matches = defaultdict(lambda: defaultdict(list))
    
    # Find all games that exist in both platforms
    common_games = set(kalshi_by_game.keys()) & set(poly_by_game.keys())
    
    for game_key in common_games:
        # For each market subtype, create matches
        all_subtypes = set(kalshi_by_game[game_key].keys()) | set(poly_by_game[game_key].keys())
        
        for subtype in all_subtypes:
            poly_markets = poly_by_game[game_key].get(subtype, [])
            kalshi_markets = kalshi_by_game[game_key].get(subtype, [])
            
            # Cross product of all matching events
            for pe in poly_markets:
                for ke in kalshi_markets:
                    matches[game_key][subtype].append((pe, ke))
    
    return matches

def match_futures_events(poly_events, kalshi_events):
    """
    Match non-game events (futures, awards, etc.) by title similarity.
    Returns: list of (poly_evt, kalshi_evt, similarity_score) tuples
    """
    matches = []
    
    poly_futures = [pe for pe in poly_events if not pe.get("is_game_based")]
    kalshi_futures = [ke for ke in kalshi_events if not ke.get("is_game_based")]
    
    for pe in poly_futures:
        poly_slug = pe.get("slug", "")
        poly_terms = set(re.findall(r'\w+', poly_slug.lower()))
        
        best_match = None
        best_score = 0
        
        for ke in kalshi_futures:
            kalshi_title = ke.get("title", "").lower()
            kalshi_terms = set(re.findall(r'\w+', kalshi_title))
            
            # Calculate term overlap
            common_terms = poly_terms & kalshi_terms
            # Filter out very short/common terms
            meaningful_common = {t for t in common_terms if len(t) > 2 and t not in {'the', 'and', 'for', 'will'}}
            
            if len(meaningful_common) >= 2:  # At least 2 meaningful common terms
                score = len(meaningful_common) / max(len(poly_terms), len(kalshi_terms))
                
                if score > best_score:
                    best_score = score
                    best_match = ke
        
        if best_match and best_score > 0.3:  # Threshold for matching
            matches.append((pe, best_match, best_score))
    
    return matches

# =========================
# FETCH KALSHI EVENTS
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

print("="*80)
print("FETCHING KALSHI EVENTS")
print("="*80 + "\n")

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    print(f"Fetching {sport}...")
    
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]
    
    count = 0
    for ticker in tickers:
        cursor = None
        
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event_id(
                    e.get("event_ticker", ""),
                    event_data=e
                )
                if parsed:
                    kalshi_events_by_sport[sport].append(parsed)
                    count += 1

            cursor = r.get("cursor")
            if not cursor:
                break
    
    print(f"  → {count} events")

# =========================
# FETCH POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

print("\n" + "="*80)
print("FETCHING POLYMARKET EVENTS")
print("="*80 + "\n")

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []

for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()
    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    
    if not kalshi_sport:
        continue

    valid_series.append((series, kalshi_sport, league_code))

print(f"Found {len(valid_series)} valid Polymarket series\n")

for series, kalshi_sport, league_code in valid_series:
    page = 0
    limit = 50
    count = 0

    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                #"tag_id": 100639,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            slug = e.get("slug")
            if not slug:
                continue

            parsed = parse_polymarket_event_slug(slug)
            if parsed:
                parsed["raw"] = e
                poly_events_by_kalshi_sport[kalshi_sport].append(parsed)
                count += 1

        page += 1
    
    if count > 0:
        print(f"  {league_code} → {kalshi_sport}: {count} events")

# =========================
# MATCHING BY SPORT
# =========================

print("\n" + "="*80)
print("MATCHING EVENTS")
print("="*80 + "\n")

total_game_matches = 0
total_futures_matches = 0

for sport in all_sports:
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])
    
    if not (kalshi_evts and poly_evts):
        continue
    
    print(f"\n{'='*80}")
    print(f"SPORT: {sport}")
    print(f"{'='*80}")
    
    # Count event types
    kalshi_game_count = sum(1 for e in kalshi_evts if e.get("is_game_based"))
    kalshi_futures_count = sum(1 for e in kalshi_evts if not e.get("is_game_based"))
    poly_game_count = sum(1 for e in poly_evts if e.get("is_game_based"))
    poly_futures_count = sum(1 for e in poly_evts if not e.get("is_game_based"))
    
    print(f"\nKalshi: {len(kalshi_evts)} total ({kalshi_game_count} game-based, {kalshi_futures_count} futures)")
    print(f"Polymarket: {len(poly_evts)} total ({poly_game_count} game-based, {poly_futures_count} futures)")
    
    # Match game-based events
    game_matches = match_game_based_events(poly_evts, kalshi_evts)
    
    if game_matches:
        print(f"\n--- GAME-BASED MATCHES ---")
        print(f"Matched {len(game_matches)} unique games:\n")
        
        for game_key, subtypes in sorted(game_matches.items())[:5]:  # Show first 5 games
            date = game_key.split("|")[0]
            teams = game_key.split("|")[1:]
            print(f"  Game: {date} - {' vs '.join(teams)}")
            
            for subtype, matches in subtypes.items():
                print(f"    {subtype}: {len(matches)} market pairs")
                for pe, ke in matches[:2]:  # Show first 2 of each type
                    print(f"      POLY:   {pe['slug'][:60]}")
                    print(f"      KALSHI: {ke['event_id']}")
                if len(matches) > 2:
                    print(f"      ... and {len(matches) - 2} more pairs")
            print()
        
        if len(game_matches) > 5:
            print(f"  ... and {len(game_matches) - 5} more games\n")
        
        # Count total matches
        game_match_count = sum(len(pairs) for subtypes in game_matches.values() for pairs in subtypes.values())
        total_game_matches += game_match_count
    
    # Match futures/non-game events
    futures_matches = match_futures_events(poly_evts, kalshi_evts)
    
    if futures_matches:
        print(f"\n--- FUTURES/AWARDS MATCHES ---")
        print(f"Matched {len(futures_matches)} futures markets:\n")
        
        for pe, ke, score in futures_matches[:5]:  # Show first 5
            print(f"  Match (score: {score:.2f}):")
            print(f"    POLY:   {pe['slug']}")
            print(f"    KALSHI: {ke['event_id']}")
            print(f"            {ke.get('title', '')}")
            print()
        
        if len(futures_matches) > 5:
            print(f"  ... and {len(futures_matches) - 5} more\n")
        
        total_futures_matches += len(futures_matches)
    
    print("-" * 80)

# =========================
# SUMMARY
# =========================

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nTotal game-based matches: {total_game_matches}")
print(f"Total futures/awards matches: {total_futures_matches}")
print(f"TOTAL MATCHES: {total_game_matches + total_futures_matches}")
print()

FETCHING KALSHI EVENTS

Fetching Football...
  → 78 events
Fetching Basketball...
  → 824 events
Fetching Hockey...
  → 123 events
Fetching Soccer...
  → 521 events
Fetching Tennis...
  → 39 events
Fetching Golf...
  → 13 events
Fetching MMA...
  → 15 events
Fetching Baseball...
  → 45 events
Fetching Boxing...
  → 6 events
Fetching Chess...
  → 1 events
Fetching Cricket...
  → 5 events
Fetching Esports...
  → 116 events
Fetching Motorsport...
  → 6 events
Fetching Olympics...
  → 4 events

FETCHING POLYMARKET EVENTS

Found 119 valid Polymarket series

  epl → Soccer: 38 events
  lal → Soccer: 43 events
  bun → Soccer: 40 events
  nfl → Football: 2 events
  fl1 → Soccer: 31 events
  sea → Soccer: 39 events
  ucl → Soccer: 36 events
  ere → Soccer: 57 events
  mex → Soccer: 40 events
  lib → Soccer: 14 events
  tur → Soccer: 53 events
  efl → Soccer: 1 events
  nba → Basketball: 63 events
  nhl → Hockey: 100 events
  uel → Soccer: 36 events
  dota2 → Esports: 23 events
  lol → Esports: 

In [8]:
import requests
from collections import defaultdict
import re

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# Map Polymarket league codes to Kalshi sport names
POLY_LEAGUE_TO_KALSHI_SPORT = {

    # ===================== SOCCER =====================
    "epl": "Soccer",
    "lal": "Soccer",
    "fl1": "Soccer",
    "bun": "Soccer",
    "sea": "Soccer",
    "ere": "Soccer",
    "por": "Soccer",
    "tur": "Soccer",
    "mls": "Soccer",
    "bra": "Soccer",
    "arg": "Soccer",
    "mex": "Soccer",
    "jap": "Soccer",
    "ja2": "Soccer",
    "kor": "Soccer",
    "spl": "Soccer",
    "chi": "Soccer",
    "aus": "Soccer",
    "ind": "Soccer",
    "nor": "Soccer",
    "den": "Soccer",
    "rus": "Soccer",
    "egy1": "Soccer",
    "cze1": "Soccer",

    # Cups & federations
    "ucl": "Soccer",
    "uel": "Soccer",
    "col": "Soccer",
    "cdr": "Soccer",
    "cde": "Soccer",
    "dfb": "Soccer",
    "efa": "Soccer",
    "efl": "Soccer",
    "ssc": "Soccer",
    "lcs": "Soccer",
    "lib": "Soccer",
    "sud": "Soccer",
    "caf": "Soccer",
    "afc": "Soccer",
    "uef": "Soccer",
    "cof": "Soccer",
    "con": "Soccer",
    "ofc": "Soccer",
    "fif": "Soccer",
    "acn": "Soccer",
    "mar1": "Soccer",

    # ===================== BASKETBALL =====================
    "nba": "Basketball",
    "wnba": "Basketball",
    "ncaab": "Basketball",
    "cbb": "Basketball",
    "cwbb": "Basketball",
    "bkcl": "Basketball",
    "bkligend": "Basketball",
    "bknbl": "Basketball",
    "bkseriea": "Basketball",
    "bkcba": "Basketball",
    "bkfr1": "Basketball",
    "bkarg": "Basketball",
    "bkkbl": "Basketball",

    # ===================== AMERICAN FOOTBALL =====================
    "nfl": "Football",
    "cfb": "Football",

    # ===================== BASEBALL =====================
    "mlb": "Baseball",
    "kbo": "Baseball",

    # ===================== HOCKEY =====================
    "nhl": "Hockey",
    "shl": "Hockey",
    "cehl": "Hockey",
    "dehl": "Hockey",
    "snhl": "Hockey",
    "khl": "Hockey",
    "ahl": "Hockey",

    # ===================== TENNIS =====================
    "atp": "Tennis",
    "wta": "Tennis",

    # ===================== CRICKET =====================
    "ipl": "Cricket",
    "odi": "Cricket",
    "t20": "Cricket",
    "test": "Cricket",
    "abb": "Cricket",
    "csa": "Cricket",
    "she": "Cricket",
    "sasa": "Cricket",
    "lpl": "Cricket",
    "psp": "Cricket",
    "crban": "Cricket",
    "crint": "Cricket",
    "crind": "Cricket",
    "crpak": "Cricket",
    "creng": "Cricket",
    "cruae": "Cricket",
    "crnew": "Cricket",
    "craus": "Cricket",
    "crsou": "Cricket",
    "cru19wc": "Cricket",
    "crwpl20": "Cricket",
    "crwncl": "Cricket",
    "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket",
    "crbtnmlyhkg20": "Cricket",

    # ===================== RUGBY =====================
    "ruprem": "Rugby",
    "rutopft": "Rugby",
    "rusixnat": "Rugby",
    "rueuchamp": "Rugby",
    "ruurc": "Rugby",
    "rusrp": "Rugby",
    "ruchamp": "Rugby",

    # ===================== MMA =====================
    "mma": "MMA",
    "zuffa": "MMA",

    # ===================== ESPORTS =====================
    "lol": "Esports",
    "dota2": "Esports",
    "cs2": "Esports",
    "val": "Esports",
    "sc2": "Esports",
    "sc": "Esports",
    "pubg": "Esports",
    "r6siege": "Esports",
    "rl": "Esports",
    "codmw": "Esports",
    "mlbb": "Esports",
    "ow": "Esports",
    "hok": "Esports",
    "wildrift": "Esports",
    "fifa": "Esports",
}

# =========================
# POLYMARKET PARSER
# =========================

def parse_polymarket_event_slug(slug: str):
    """Parse Polymarket slug and extract all available information"""
    slug_lower = slug.lower()
    original_slug = slug_lower
    
    # Detect market subtype from slug patterns
    market_subtype = "moneyline"  # default
    if slug_lower.endswith("-more-markets"):
        market_subtype = "totals"
        slug_lower = slug_lower.replace("-more-markets", "")
    elif "spread" in slug_lower:
        market_subtype = "spread"
    
    # Detect if this is a game-based market or futures/awards
    is_game_based = True
    if any(kw in slug_lower for kw in ["winner", "champion", "playoff", "title", "trophy"]):
        is_game_based = False
        market_subtype = "futures"
    elif any(kw in slug_lower for kw in ["mvp", "award", "dpoy", "roty", "opoy"]):
        is_game_based = False
        market_subtype = "awards"
    
    parts = slug_lower.split("-")
    
    # Try to extract date (YYYY-MM-DD pattern in parts)
    date = None
    date_idx = None
    for i in range(len(parts) - 2):
        if (parts[i].isdigit() and len(parts[i]) == 4 and  # Year
            parts[i+1].isdigit() and len(parts[i+1]) <= 2 and  # Month
            parts[i+2].isdigit() and len(parts[i+2]) <= 2):  # Day
            date = f"{parts[i]}-{parts[i+1].zfill(2)}-{parts[i+2].zfill(2)}"
            date_idx = i
            break
    
    league = parts[0] if parts else None
    
    # Extract teams if this is a game-type market
    home = None
    away = None
    if is_game_based and date_idx and date_idx >= 3:
        team_parts = parts[1:date_idx]
        if len(team_parts) >= 2:
            mid = len(team_parts) // 2
            home = "-".join(team_parts[:mid])
            away = "-".join(team_parts[mid:])
    
    return {
        "league": league,
        "date": date,
        "home": home,
        "away": away,
        "is_game_based": is_game_based,
        "market_subtype": market_subtype,
        "slug": original_slug,
    }

# =========================
# KALSHI PARSER
# =========================

def parse_kalshi_event_id(event_id: str, event_data: dict = None):
    """Parse Kalshi event ticker and extract all information"""
    if "-" not in event_id:
        return None

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return None

    prefix = prefix[2:]  # remove KX
    
    # Extract league and scope
    league = None
    scope = None
    market_subtype = None
    
    # Game-based scope patterns
    game_scope_patterns = [
        ("GAME", "game", "moneyline"),
        ("TOTALS", "game", "totals"),
        ("SPREAD", "game", "spread"),
        ("ALT", "game", "alt"),
        ("LINES", "game", "lines"),
        ("EVENT", "game", "moneyline"),
    ]
    
    # Non-game scope patterns
    non_game_scope_patterns = [
        ("FUT", "futures", "futures"),
        ("FUTURES", "futures", "futures"),
        ("WINS", "futures", "wins"),
        ("WINTOTALS", "futures", "wins"),
        ("AWARDS", "futures", "awards"),
        ("MVP", "futures", "awards"),
        ("DPOY", "futures", "awards"),
        ("ROTY", "futures", "awards"),
        ("DRAFT", "futures", "draft"),
        ("LEADER", "futures", "leader"),
        ("ADVANCE", "futures", "advance"),
    ]
    
    is_game_based = False
    
    # Check game patterns first
    for pattern, scope_type, subtype in game_scope_patterns:
        if prefix.endswith(pattern):
            scope = scope_type
            market_subtype = subtype
            league = prefix[:-len(pattern)]
            is_game_based = True
            break
    
    # If not game-based, check non-game patterns
    if not is_game_based:
        for pattern, scope_type, subtype in non_game_scope_patterns:
            if prefix.endswith(pattern):
                scope = scope_type
                market_subtype = subtype
                league = prefix[:-len(pattern)]
                break
    
    if not scope:
        scope = "unknown"
        market_subtype = "unknown"
        league = prefix
    
    # Parse date and teams (primarily for game-based events)
    date = None
    home = None
    away = None
    
    if is_game_based and len(payload) >= 7:
        date_block = payload[:7]
        year_part = date_block[:2]
        month_part = date_block[2:5]
        day_part = date_block[5:7]
        
        if month_part in MONTH_MAP and day_part.isdigit():
            year = "20" + year_part
            month = MONTH_MAP[month_part]
            day = day_part
            date = f"{year}-{month}-{day}"
            
            # Extract teams
            team_block = payload[7:]
            if len(team_block) >= 6:
                home = team_block[-6:-3].lower()
                away = team_block[-3:].lower()
    
    # Extract title if available
    title = ""
    if event_data:
        title = event_data.get("title", "")
    
    return {
        "league": league.lower() if league else None,
        "date": date,
        "home": home,
        "away": away,
        "is_game_based": is_game_based,
        "scope": scope,
        "market_subtype": market_subtype,
        "event_id": event_id,
        "title": title,
    }

# =========================
# MATCHING FUNCTIONS
# =========================

def normalize_team(team: str) -> str:
    """Normalize team name for comparison"""
    if not team:
        return ""
    return team.lower().replace("-", "").replace(" ", "").replace(".", "")

def create_game_key(date: str, team1: str, team2: str) -> str:
    """Create a normalized key for a game (date + sorted teams)"""
    teams = sorted([normalize_team(team1), normalize_team(team2)])
    return f"{date}|{'|'.join(teams)}"

def match_game_based_events(poly_events, kalshi_events):
    """
    Match game-based events by date + teams, then group by market subtype.
    Returns: dict[game_key] -> dict[market_subtype] -> list of (poly_evt, kalshi_evt) pairs
    """
    # Index Kalshi events by game key
    kalshi_by_game = defaultdict(lambda: defaultdict(list))
    for ke in kalshi_events:
        if not ke.get("is_game_based"):
            continue
        if not (ke.get("date") and ke.get("home") and ke.get("away")):
            continue
        
        game_key = create_game_key(ke["date"], ke["home"], ke["away"])
        subtype = ke.get("market_subtype", "unknown")
        kalshi_by_game[game_key][subtype].append(ke)
    
    # Index Polymarket events by game key
    poly_by_game = defaultdict(lambda: defaultdict(list))
    for pe in poly_events:
        if not pe.get("is_game_based"):
            continue
        if not (pe.get("date") and pe.get("home") and pe.get("away")):
            continue
        
        game_key = create_game_key(pe["date"], pe["home"], pe["away"])
        subtype = pe.get("market_subtype", "unknown")
        poly_by_game[game_key][subtype].append(pe)
    
    # Create matches grouped by game and market subtype
    matches = defaultdict(lambda: defaultdict(list))
    
    # Find all games that exist in both platforms
    common_games = set(kalshi_by_game.keys()) & set(poly_by_game.keys())
    
    for game_key in common_games:
        # For each market subtype, create matches
        all_subtypes = set(kalshi_by_game[game_key].keys()) | set(poly_by_game[game_key].keys())
        
        for subtype in all_subtypes:
            poly_markets = poly_by_game[game_key].get(subtype, [])
            kalshi_markets = kalshi_by_game[game_key].get(subtype, [])
            
            # Cross product of all matching events
            for pe in poly_markets:
                for ke in kalshi_markets:
                    matches[game_key][subtype].append((pe, ke))
    
    return matches

def match_futures_events(poly_events, kalshi_events):
    """
    Match non-game events (futures, awards, etc.) by title similarity.
    Returns: list of (poly_evt, kalshi_evt, similarity_score) tuples
    """
    matches = []
    
    poly_futures = [pe for pe in poly_events if not pe.get("is_game_based")]
    kalshi_futures = [ke for ke in kalshi_events if not ke.get("is_game_based")]
    
    for pe in poly_futures:
        poly_slug = pe.get("slug", "")
        poly_terms = set(re.findall(r'\w+', poly_slug.lower()))
        
        best_match = None
        best_score = 0
        
        for ke in kalshi_futures:
            kalshi_title = ke.get("title", "").lower()
            kalshi_terms = set(re.findall(r'\w+', kalshi_title))
            
            # Calculate term overlap
            common_terms = poly_terms & kalshi_terms
            # Filter out very short/common terms
            meaningful_common = {t for t in common_terms if len(t) > 2 and t not in {'the', 'and', 'for', 'will'}}
            
            if len(meaningful_common) >= 2:  # At least 2 meaningful common terms
                score = len(meaningful_common) / max(len(poly_terms), len(kalshi_terms))
                
                if score > best_score:
                    best_score = score
                    best_match = ke
        
        if best_match and best_score > 0.3:  # Threshold for matching
            matches.append((pe, best_match, best_score))
    
    return matches

# =========================
# FETCH KALSHI EVENTS
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

print("="*80)
print("FETCHING KALSHI EVENTS")
print("="*80 + "\n")

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    print(f"Fetching {sport}...")
    
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]
    
    count = 0
    for ticker in tickers:
        cursor = None
        
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event_id(
                    e.get("event_ticker", ""),
                    event_data=e
                )
                if parsed:
                    kalshi_events_by_sport[sport].append(parsed)
                    count += 1

            cursor = r.get("cursor")
            if not cursor:
                break
    
    print(f"  → {count} events")

# =========================
# FETCH POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

print("\n" + "="*80)
print("FETCHING POLYMARKET EVENTS")
print("="*80 + "\n")

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []

for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()
    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    
    if not kalshi_sport:
        # Log unmapped leagues so you can add them if needed
        print(f"  Unmapped league: {league_code}")
        continue

    valid_series.append((series, kalshi_sport, league_code))

print(f"\nFound {len(valid_series)} valid Polymarket series\n")

for series, kalshi_sport, league_code in valid_series:
    page = 0
    limit = 50
    count = 0

    while True:
        # Try WITHOUT tag_id filter first to get all events
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                # "tag_id": 100639,  # Removed - this was filtering too aggressively
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            slug = e.get("slug")
            if not slug:
                continue

            parsed = parse_polymarket_event_slug(slug)
            if parsed:
                parsed["raw"] = e
                poly_events_by_kalshi_sport[kalshi_sport].append(parsed)
                count += 1

        page += 1
    
    if count > 0:
        print(f"  {league_code} → {kalshi_sport}: {count} events")

# =========================
# MATCHING BY SPORT
# =========================

print("\n" + "="*80)
print("MATCHING EVENTS")
print("="*80 + "\n")

total_game_matches = 0
total_futures_matches = 0

for sport in all_sports:
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])
    
    if not (kalshi_evts and poly_evts):
        continue
    
    print(f"\n{'='*80}")
    print(f"SPORT: {sport}")
    print(f"{'='*80}")
    
    # Count event types
    kalshi_game_count = sum(1 for e in kalshi_evts if e.get("is_game_based"))
    kalshi_futures_count = sum(1 for e in kalshi_evts if not e.get("is_game_based"))
    poly_game_count = sum(1 for e in poly_evts if e.get("is_game_based"))
    poly_futures_count = sum(1 for e in poly_evts if not e.get("is_game_based"))
    
    print(f"\nKalshi: {len(kalshi_evts)} total ({kalshi_game_count} game-based, {kalshi_futures_count} futures)")
    print(f"Polymarket: {len(poly_evts)} total ({poly_game_count} game-based, {poly_futures_count} futures)")
    
    # Match game-based events
    game_matches = match_game_based_events(poly_evts, kalshi_evts)
    
    if game_matches:
        print(f"\n--- GAME-BASED MATCHES ---")
        print(f"Matched {len(game_matches)} unique games:\n")
        
        for game_key, subtypes in sorted(game_matches.items())[:5]:  # Show first 5 games
            date = game_key.split("|")[0]
            teams = game_key.split("|")[1:]
            print(f"  Game: {date} - {' vs '.join(teams)}")
            
            for subtype, matches in subtypes.items():
                print(f"    {subtype}: {len(matches)} market pairs")
                for pe, ke in matches[:2]:  # Show first 2 of each type
                    print(f"      POLY:   {pe['slug'][:60]}")
                    print(f"      KALSHI: {ke['event_id']}")
                if len(matches) > 2:
                    print(f"      ... and {len(matches) - 2} more pairs")
            print()
        
        if len(game_matches) > 5:
            print(f"  ... and {len(game_matches) - 5} more games\n")
        
        # Count total matches
        game_match_count = sum(len(pairs) for subtypes in game_matches.values() for pairs in subtypes.values())
        total_game_matches += game_match_count
    
    # Match futures/non-game events
    futures_matches = match_futures_events(poly_evts, kalshi_evts)
    
    if futures_matches:
        print(f"\n--- FUTURES/AWARDS MATCHES ---")
        print(f"Matched {len(futures_matches)} futures markets:\n")
        
        for pe, ke, score in futures_matches[:5]:  # Show first 5
            print(f"  Match (score: {score:.2f}):")
            print(f"    POLY:   {pe['slug']}")
            print(f"    KALSHI: {ke['event_id']}")
            print(f"            {ke.get('title', '')}")
            print()
        
        if len(futures_matches) > 5:
            print(f"  ... and {len(futures_matches) - 5} more\n")
        
        total_futures_matches += len(futures_matches)
    
    print("-" * 80)

# =========================
# SUMMARY
# =========================

print("\n" + "="*80)
print("SUMMARY")
print("="*80)
print(f"\nTotal game-based matches: {total_game_matches}")
print(f"Total futures/awards matches: {total_futures_matches}")
print(f"TOTAL MATCHES: {total_game_matches + total_futures_matches}")
print()

FETCHING KALSHI EVENTS

Fetching Football...
  → 56 events
Fetching Basketball...
  → 272 events
Fetching Hockey...
  → 76 events
Fetching Soccer...
  → 349 events
Fetching Tennis...
  → 25 events
Fetching Golf...
  → 10 events
Fetching MMA...
  → 3 events
Fetching Baseball...
  → 29 events
Fetching Boxing...
  → 6 events
Fetching Chess...
  → 0 events
Fetching Cricket...
  → 5 events
Fetching Esports...
  → 71 events
Fetching Motorsport...
  → 5 events
Fetching Olympics...
  → 4 events

FETCHING POLYMARKET EVENTS

  Unmapped league: itc
  Unmapped league: bol1
  Unmapped league: rou1
  Unmapped league: bra2
  Unmapped league: per1
  Unmapped league: col1
  Unmapped league: chi1

Found 119 valid Polymarket series

  epl → Soccer: 42 events
  lal → Soccer: 43 events
  bun → Soccer: 40 events
  nfl → Football: 2 events
  fl1 → Soccer: 31 events
  sea → Soccer: 36 events
  ucl → Soccer: 36 events
  ere → Soccer: 53 events
  mex → Soccer: 42 events
  lib → Soccer: 14 events
  tur → Soccer:

# Events matching - not underlying markets

In [ ]:
#pip install fuzzywuzzy python-Levenshtein

event matching without specific Soccer Poly<->kalshi event representation (poly has 2 events for a game, one is moneyline and the other is called "more markets". Kalshi has 4 events for the same game, moneyline, totals, spreads, both teams to score). The code just below doesn't say that for moneyline the matching should be {1 poly<-> 1 kalshi}, and {1 :"more markets" poly<-> 3: totals, spreads, both teams to score} on kalshi

In [5]:
import requests
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd
from datetime import datetime

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# Map Polymarket league codes to Kalshi sport names
POLY_LEAGUE_TO_KALSHI_SPORT = {
    # ===================== SOCCER =====================
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    # Cups & federations
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # ===================== BASKETBALL =====================
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # ===================== AMERICAN FOOTBALL =====================
    "nfl": "Football", "cfb": "Football",
    # ===================== BASEBALL =====================
    "mlb": "Baseball", "kbo": "Baseball",
    # ===================== HOCKEY =====================
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # ===================== TENNIS =====================
    "atp": "Tennis", "wta": "Tennis",
    # ===================== CRICKET =====================
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # ===================== RUGBY =====================
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # ===================== MMA =====================
    "mma": "MMA", "zuffa": "MMA",
    # ===================== ESPORTS =====================
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

# =========================
# POLYMARKET PARSER
# =========================

def extract_date_from_polymarket(event_data: dict) -> str:
    """
    Extract date in YYYY-MM-DD format from Polymarket event
    Uses endDate field with format: 2026-01-22T17:45:00Z
    """
    # Use endDate as the primary field (this is when the event ends/resolves)
    if event_data.get('endDate'):
        end_date = event_data['endDate']
        if isinstance(end_date, str) and 'T' in end_date:
            # Extract just the date part (YYYY-MM-DD)
            return end_date.split('T')[0]
    
    # Fallback to startDate if endDate is not available
    if event_data.get('startDate'):
        start_date = event_data['startDate']
        if isinstance(start_date, str):
            if 'T' in start_date:
                return start_date.split('T')[0]
            if len(start_date) >= 10:
                return start_date[:10]
    
    return None

def parse_polymarket_event(event_data: dict) -> dict:
    """Parse Polymarket event and extract relevant fields"""
    return {
        "platform": "polymarket",
        "slug": event_data.get('slug', ''),
        "title": event_data.get('title', ''),
        "date": extract_date_from_polymarket(event_data),
        "raw": event_data,
    }

# =========================
# KALSHI PARSER
# =========================

def parse_kalshi_event_id(event_id: str) -> dict:
    """
    Parse Kalshi event ticker to extract date
    Format: KXNFL2TD-26JAN25LASEA
    Date is: YYMMMDD (26JAN25)
    """
    if "-" not in event_id:
        return {"date": None}

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return {"date": None}
    
    date = None
    # The date format is YYMMMDD (e.g., 26JAN25)
    if len(payload) >= 7:
        year_part = payload[:2]
        month_part = payload[2:5]
        day_part = payload[5:7]
        
        if day_part.isdigit() and month_part in MONTH_MAP and year_part.isdigit():
            year = "20" + year_part
            month = MONTH_MAP[month_part]
            day = day_part
            date = f"{year}-{month}-{day}"
    
    return {"date": date}

def parse_kalshi_event(event_data: dict) -> dict:
    """Parse Kalshi event and extract relevant fields"""
    event_id = event_data.get('event_ticker', '')
    parsed = parse_kalshi_event_id(event_id)
    
    return {
        "platform": "kalshi",
        "event_id": event_id,
        "title": event_data.get('title', ''),
        "date": parsed["date"],
        "raw": event_data,
    }

# =========================
# MATCHING LOGIC
# =========================

def normalize_title(s: str) -> str:
    """Normalize title for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def match_events(poly_events: list, kalshi_events: list, threshold: int = 80) -> list:
    """
    Match Polymarket events to Kalshi events using fuzzy matching.
    Returns list of matched pairs.
    
    Implements greedy one-to-one matching:
    - Each Polymarket event matches to at most one Kalshi event
    - Each Kalshi event matches to at most one Polymarket event
    - Prioritizes best matches first
    """
    if not poly_events or not kalshi_events:
        return []
    
    # Calculate all similarity scores
    candidates = []
    for poly_idx, poly_evt in enumerate(poly_events):
        poly_title = normalize_title(poly_evt["title"])
        
        for kalshi_idx, kalshi_evt in enumerate(kalshi_events):
            kalshi_title = normalize_title(kalshi_evt["title"])
            
            # Use token_set_ratio for better fuzzy matching
            score = fuzz.token_set_ratio(poly_title, kalshi_title)
            
            if score >= threshold:
                candidates.append({
                    'poly_idx': poly_idx,
                    'kalshi_idx': kalshi_idx,
                    'score': score,
                    'poly_evt': poly_evt,
                    'kalshi_evt': kalshi_evt
                })
    
    # Sort by score (descending) for greedy matching
    candidates.sort(key=lambda x: x['score'], reverse=True)
    
    # Greedy one-to-one matching
    matched_pairs = []
    used_poly = set()
    used_kalshi = set()
    
    for candidate in candidates:
        poly_idx = candidate['poly_idx']
        kalshi_idx = candidate['kalshi_idx']
        
        # Skip if either side already matched
        if poly_idx in used_poly or kalshi_idx in used_kalshi:
            continue
        
        matched_pairs.append({
            'polymarket': candidate['poly_evt'],
            'kalshi': candidate['kalshi_evt'],
            'match_score': candidate['score']
        })
        
        used_poly.add(poly_idx)
        used_kalshi.add(kalshi_idx)
    
    return matched_pairs

# =========================
# FETCH KALSHI EVENTS
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

print("="*80)
print("STEP 1: FETCHING KALSHI EVENTS")
print("="*80 + "\n")

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    print(f"Fetching {sport}...", end=" ")
    
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]
    
    count = 0
    for ticker in tickers:
        cursor = None
        
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event(e)
                kalshi_events_by_sport[sport].append(parsed)
                count += 1

            cursor = r.get("cursor")
            if not cursor:
                break
    
    print(f"{count} events")

# =========================
# FETCH POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

print("\n" + "="*80)
print("STEP 2: FETCHING POLYMARKET EVENTS")
print("="*80 + "\n")

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []
for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()
    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    
    if not kalshi_sport:
        continue

    valid_series.append((series, kalshi_sport, league_code))

print(f"Found {len(valid_series)} valid Polymarket series\n")

for series, kalshi_sport, league_code in valid_series:
    print(f"Fetching {league_code} → {kalshi_sport}...", end=" ")
    page = 0
    limit = 50
    count = 0

    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            parsed = parse_polymarket_event(e)
            poly_events_by_kalshi_sport[kalshi_sport].append(parsed)
            count += 1

        page += 1
    
    print(f"{count} events")

# =========================
# STEP 3: GROUP BY SPORT AND DATE
# =========================

print("\n" + "="*80)
print("STEP 3: GROUPING EVENTS BY SPORT AND DATE")
print("="*80 + "\n")

events_by_sport_date = {}

for sport in all_sports:
    print(f"\n{sport}:")
    print("-" * 80)
    
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])
    
    if not (kalshi_evts or poly_evts):
        print("  No events found")
        continue
    
    print(f"  Total: {len(kalshi_evts)} Kalshi, {len(poly_evts)} Polymarket")
    
    # Group by date
    by_date = defaultdict(lambda: {"kalshi": [], "polymarket": []})
    
    for evt in kalshi_evts:
        if evt["date"]:
            by_date[evt["date"]]["kalshi"].append(evt)
    
    for evt in poly_evts:
        if evt["date"]:
            by_date[evt["date"]]["polymarket"].append(evt)
    
    # Only keep dates that have events from BOTH platforms
    common_dates = {date for date, evts in by_date.items() 
                    if evts["kalshi"] and evts["polymarket"]}
    
    if common_dates:
        print(f"  Dates with events on both platforms: {len(common_dates)}")
        for date in sorted(common_dates)[:5]:
            k_count = len(by_date[date]["kalshi"])
            p_count = len(by_date[date]["polymarket"])
            print(f"    {date}: {k_count} Kalshi, {p_count} Polymarket")
        if len(common_dates) > 5:
            print(f"    ... and {len(common_dates) - 5} more dates")
    else:
        print("  No common dates found")
    
    events_by_sport_date[sport] = {date: by_date[date] for date in common_dates}

# =========================
# STEP 4: FUZZY MATCHING WITHIN EACH DATE
# =========================

print("\n" + "="*80)
print("STEP 4: FUZZY MATCHING WITHIN EACH SPORT/DATE")
print("="*80 + "\n")

SIMILARITY_THRESHOLD = 80
all_matches = []

for sport, dates_dict in events_by_sport_date.items():
    if not dates_dict:
        continue
    
    print(f"\n{sport}:")
    print("-" * 80)
    
    sport_match_count = 0
    
    for date in sorted(dates_dict.keys()):
        events = dates_dict[date]
        poly_events = events["polymarket"]
        kalshi_events = events["kalshi"]
        
        print(f"\n  Date: {date}")
        print(f"    Comparing {len(poly_events)} Poly vs {len(kalshi_events)} Kalshi events")
        
        # Perform matching
        matched_pairs = match_events(poly_events, kalshi_events, SIMILARITY_THRESHOLD)
        
        # Add metadata and collect results
        for match in matched_pairs:
            poly_evt = match['polymarket']
            kalshi_evt = match['kalshi']
            score = match['match_score']
            
            print(f"    ✓ MATCH (score: {score})")
            print(f"      Poly:   {poly_evt['title']}")
            print(f"      Kalshi: {kalshi_evt['title']}")
            
            all_matches.append({
                "sport": sport,
                "date": date,
                "poly_title": poly_evt['title'],
                "poly_slug": poly_evt.get('slug', ''),
                "kalshi_title": kalshi_evt['title'],
                "kalshi_event_id": kalshi_evt.get('event_id', ''),
                "match_score": score,
            })
            
            sport_match_count += 1
        
        if not matched_pairs:
            print(f"    No matches found for this date")
    
    print(f"\n  → Total matches for {sport}: {sport_match_count}")

# =========================
# STEP 5: SAVE RESULTS
# =========================

print("\n" + "="*80)
print("STEP 5: SAVING RESULTS")
print("="*80 + "\n")

if all_matches:
    df = pd.DataFrame(all_matches)
    
    print("\nSUMMARY STATISTICS:")
    print(f"  Total matched pairs: {len(all_matches)}")
    print(f"\nBy sport:")
    print(df['sport'].value_counts().to_string())
    
    print(f"\nMatch score distribution:")
    print(df['match_score'].describe())
    
    print(f"\nSAMPLE MATCHES (first 10):")
    print(df[['sport', 'date', 'poly_title', 'kalshi_title', 'match_score']].head(10).to_string())
    
else:
    print("No matches found!")

print("\n" + "="*80)
print("COMPLETE")
print("="*80)

STEP 1: FETCHING KALSHI EVENTS

Fetching Football... 33 events
Fetching Basketball... 152 events
Fetching Hockey... 57 events
Fetching Soccer... 339 events
Fetching Tennis... 14 events
Fetching Golf... 6 events
Fetching MMA... 2 events
Fetching Baseball... 36 events
Fetching Boxing... 5 events
Fetching Chess... 1 events
Fetching Cricket... 6 events
Fetching Esports... 70 events
Fetching Motorsport... 6 events
Fetching Olympics... 3 events

STEP 2: FETCHING POLYMARKET EVENTS

Found 119 valid Polymarket series

Fetching ncaab → Basketball... 0 events
Fetching epl → Soccer... 38 events
Fetching lal → Soccer... 43 events
Fetching acn → Soccer... 0 events
Fetching ipl → Cricket... 0 events
Fetching wnba → Basketball... 0 events
Fetching bun → Soccer... 40 events
Fetching mlb → Baseball... 0 events
Fetching cfb → Football... 0 events
Fetching nfl → Football... 1 events
Fetching fl1 → Soccer... 21 events
Fetching sea → Soccer... 34 events
Fetching ucl → Soccer... 36 events
Fetching afc → Socc

Matching at event level with same date check - added the soccer poly<->kalshi matching

In [8]:
import requests
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd
from datetime import datetime

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# Map Polymarket league codes to Kalshi sport names
POLY_LEAGUE_TO_KALSHI_SPORT = {
    # ===================== SOCCER =====================
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    # Cups & federations
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # ===================== BASKETBALL =====================
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # ===================== AMERICAN FOOTBALL =====================
    "nfl": "Football", "cfb": "Football",
    # ===================== BASEBALL =====================
    "mlb": "Baseball", "kbo": "Baseball",
    # ===================== HOCKEY =====================
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # ===================== TENNIS =====================
    "atp": "Tennis", "wta": "Tennis",
    # ===================== CRICKET =====================
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # ===================== RUGBY =====================
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # ===================== MMA =====================
    "mma": "MMA", "zuffa": "MMA",
    # ===================== ESPORTS =====================
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

# =========================
# POLYMARKET PARSER
# =========================

def extract_date_from_polymarket(event_data: dict) -> str:
    """
    Extract date in YYYY-MM-DD format from Polymarket event
    Uses endDate field with format: 2026-01-22T17:45:00Z
    """
    # Use endDate as the primary field (this is when the event ends/resolves)
    if event_data.get('endDate'):
        end_date = event_data['endDate']
        if isinstance(end_date, str) and 'T' in end_date:
            # Extract just the date part (YYYY-MM-DD)
            return end_date.split('T')[0]
    
    # Fallback to startDate if endDate is not available
    if event_data.get('startDate'):
        start_date = event_data['startDate']
        if isinstance(start_date, str):
            if 'T' in start_date:
                return start_date.split('T')[0]
            if len(start_date) >= 10:
                return start_date[:10]
    
    return None

def parse_polymarket_event(event_data: dict) -> dict:
    """Parse Polymarket event and extract relevant fields"""
    return {
        "platform": "polymarket",
        "slug": event_data.get('slug', ''),
        "title": event_data.get('title', ''),
        "date": extract_date_from_polymarket(event_data),
        "raw": event_data,
    }

# =========================
# KALSHI PARSER
# =========================

def parse_kalshi_event_id(event_id: str) -> dict:
    """
    Parse Kalshi event ticker to extract date
    Format: KXNFL2TD-26JAN25LASEA
    Date is: YYMMMDD (26JAN25)
    """
    if "-" not in event_id:
        return {"date": None}

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return {"date": None}
    
    date = None
    # The date format is YYMMMDD (e.g., 26JAN25)
    if len(payload) >= 7:
        year_part = payload[:2]
        month_part = payload[2:5]
        day_part = payload[5:7]
        
        if day_part.isdigit() and month_part in MONTH_MAP and year_part.isdigit():
            year = "20" + year_part
            month = MONTH_MAP[month_part]
            day = day_part
            date = f"{year}-{month}-{day}"
    
    return {"date": date}

def parse_kalshi_event(event_data: dict) -> dict:
    """Parse Kalshi event and extract relevant fields"""
    event_id = event_data.get('event_ticker', '')
    parsed = parse_kalshi_event_id(event_id)
    
    return {
        "platform": "kalshi",
        "event_id": event_id,
        "title": event_data.get('title', ''),
        "date": parsed["date"],
        "raw": event_data,
    }

# =========================
# NORMALIZATION
# =========================

def normalize_title(s: str) -> str:
    """Normalize title for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_game_name(title: str, platform: str) -> str:
    """Extract the game name from the event title."""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        else:
            return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        else:
            return title
    return title

# =========================
# MARKET TYPE INFERENCE
# =========================

def infer_kalshi_type(title):
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title):
    return "more_markets" if "More Markets" in title else "moneyline"

# =========================
# MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=80):
    matches = []

    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])

        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])

            # ---- TYPE COMPATIBILITY ----
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue

            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type
                })

    # UPDATED: Greedy matching with support for 1:many for "more_markets"
    # Sort by score descending
    matches.sort(key=lambda x: x["score"], reverse=True)
    used_kalshi = set()
    final_matches = []
    for match in matches:
        kalshi_id = id(match["kalshi"])  # Use object ID to uniquely identify Kalshi events
        if kalshi_id not in used_kalshi:
            final_matches.append({
                "polymarket": match["poly"],
                "kalshi": match["kalshi"],
                "match_score": match["score"]
            })
            # Only mark Kalshi as used; allow "more_markets" Poly to match multiple Kalshi
            used_kalshi.add(kalshi_id)

    return final_matches

# =========================
# FETCH KALSHI EVENTS
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

print("="*80)
print("STEP 1: FETCHING KALSHI EVENTS")
print("="*80 + "\n")

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    print(f"Fetching {sport}...", end=" ")
    
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]
    
    count = 0
    for ticker in tickers:
        cursor = None
        
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event(e)
                kalshi_events_by_sport[sport].append(parsed)
                count += 1

            cursor = r.get("cursor")
            if not cursor:
                break
    
    print(f"{count} events")

# =========================
# FETCH POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

print("\n" + "="*80)
print("STEP 2: FETCHING POLYMARKET EVENTS")
print("="*80 + "\n")

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []
for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()
    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    
    if not kalshi_sport:
        continue

    valid_series.append((series, kalshi_sport, league_code))

print(f"Found {len(valid_series)} valid Polymarket series\n")

for series, kalshi_sport, league_code in valid_series:
    print(f"Fetching {league_code} → {kalshi_sport}...", end=" ")
    page = 0
    limit = 50
    count = 0

    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            parsed = parse_polymarket_event(e)
            poly_events_by_kalshi_sport[kalshi_sport].append(parsed)
            count += 1

        page += 1
    
    print(f"{count} events")

# =========================
# STEP 3: GROUP BY SPORT AND DATE
# =========================

print("\n" + "="*80)
print("STEP 3: GROUPING EVENTS BY SPORT AND DATE")
print("="*80 + "\n")

events_by_sport_date = {}

for sport in all_sports:
    print(f"\n{sport}:")
    print("-" * 80)
    
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])
    
    if not (kalshi_evts or poly_evts):
        print("  No events found")
        continue
    
    print(f"  Total: {len(kalshi_evts)} Kalshi, {len(poly_evts)} Polymarket")
    
    # Group by date
    by_date = defaultdict(lambda: {"kalshi": [], "polymarket": []})
    
    for evt in kalshi_evts:
        if evt["date"]:
            by_date[evt["date"]]["kalshi"].append(evt)
    
    for evt in poly_evts:
        if evt["date"]:
            by_date[evt["date"]]["polymarket"].append(evt)
    
    # Only keep dates that have events from BOTH platforms
    common_dates = {date for date, evts in by_date.items() 
                    if evts["kalshi"] and evts["polymarket"]}
    
    if common_dates:
        print(f"  Dates with events on both platforms: {len(common_dates)}")
        for date in sorted(common_dates)[:5]:
            k_count = len(by_date[date]["kalshi"])
            p_count = len(by_date[date]["polymarket"])
            print(f"    {date}: {k_count} Kalshi, {p_count} Polymarket")
        if len(common_dates) > 5:
            print(f"    ... and {len(common_dates) - 5} more dates")
    else:
        print("  No common dates found")
    
    events_by_sport_date[sport] = {date: by_date[date] for date in common_dates}

# =========================
# STEP 4: FUZZY MATCHING WITHIN EACH DATE
# =========================

print("\n" + "="*80)
print("STEP 4: FUZZY MATCHING WITHIN EACH SPORT/DATE")
print("="*80 + "\n")

SIMILARITY_THRESHOLD = 80
all_matches = []

for sport, dates_dict in events_by_sport_date.items():
    if not dates_dict:
        continue
    
    print(f"\n{sport}:")
    print("-" * 80)
    
    sport_match_count = 0
    
    for date in sorted(dates_dict.keys()):
        events = dates_dict[date]
        poly_events = events["polymarket"]
        kalshi_events = events["kalshi"]
        
        print(f"\n  Date: {date}")
        print(f"    Comparing {len(poly_events)} Poly vs {len(kalshi_events)} Kalshi events")
        
        # Perform matching
        matched_pairs = match_events(poly_events, kalshi_events, SIMILARITY_THRESHOLD)
        
        # Add metadata and collect results
        for match in matched_pairs:
            poly_evt = match['polymarket']
            kalshi_evt = match['kalshi']
            score = match['match_score']
            
            print(f"    ✓ MATCH (score: {score})")
            print(f"      Poly:   {poly_evt['title']}")
            print(f"      Kalshi: {kalshi_evt['title']}")
            
            all_matches.append({
                "sport": sport,
                "date": date,
                "poly_title": poly_evt['title'],
                "poly_slug": poly_evt.get('slug', ''),
                "kalshi_title": kalshi_evt['title'],
                "kalshi_event_id": kalshi_evt.get('event_id', ''),
                "match_score": score,
            })
            
            sport_match_count += 1
        
        if not matched_pairs:
            print(f"    No matches found for this date")
    
    print(f"\n  → Total matches for {sport}: {sport_match_count}")

# =========================
# STEP 5: SAVE RESULTS
# =========================

print("\n" + "="*80)
print("STEP 5: SAVING RESULTS")
print("="*80 + "\n")

if all_matches:
    df = pd.DataFrame(all_matches)
    
    print("\nSUMMARY STATISTICS:")
    print(f"  Total matched pairs: {len(all_matches)}")
    print(f"\nBy sport:")
    print(df['sport'].value_counts().to_string())
    
    print(f"\nMatch score distribution:")
    print(df['match_score'].describe())
    
    print(f"\nSAMPLE MATCHES (first 10):")
    print(df[['sport', 'date', 'poly_title', 'kalshi_title', 'match_score']].head(10).to_string())
    
else:
    print("No matches found!")

print("\n" + "="*80)
print("COMPLETE")
print("="*80)


STEP 1: FETCHING KALSHI EVENTS

Fetching Football... 44 events
Fetching Basketball... 373 events
Fetching Hockey... 75 events
Fetching Soccer... 393 events
Fetching Tennis... 44 events
Fetching Golf... 5 events
Fetching MMA... 2 events
Fetching Baseball... 32 events
Fetching Boxing... 5 events
Fetching Chess... 0 events
Fetching Cricket... 6 events
Fetching Esports... 34 events
Fetching Motorsport... 5 events
Fetching Olympics... 3 events

STEP 2: FETCHING POLYMARKET EVENTS

Found 119 valid Polymarket series

Fetching ncaab → Basketball... 0 events
Fetching epl → Soccer... 38 events
Fetching lal → Soccer... 43 events
Fetching acn → Soccer... 0 events
Fetching ipl → Cricket... 0 events
Fetching wnba → Basketball... 0 events
Fetching bun → Soccer... 40 events
Fetching mlb → Baseball... 0 events
Fetching cfb → Football... 0 events
Fetching nfl → Football... 1 events
Fetching fl1 → Soccer... 21 events
Fetching sea → Soccer... 34 events
Fetching ucl → Soccer... 36 events
Fetching afc → Socc

Matching only events without same date check

In [10]:
import requests
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd
from datetime import datetime

MONTH_MAP = {
    "JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
    "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08",
    "SEP": "09", "OCT": "10", "NOV": "11", "DEC": "12",
}

# Map Polymarket league codes to Kalshi sport names
POLY_LEAGUE_TO_KALSHI_SPORT = {
    # ===================== SOCCER =====================
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    # Cups & federations
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # ===================== BASKETBALL =====================
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # ===================== AMERICAN FOOTBALL =====================
    "nfl": "Football", "cfb": "Football",
    # ===================== BASEBALL =====================
    "mlb": "Baseball", "kbo": "Baseball",
    # ===================== HOCKEY =====================
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # ===================== TENNIS =====================
    "atp": "Tennis", "wta": "Tennis",
    # ===================== CRICKET =====================
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # ===================== RUGBY =====================
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # ===================== MMA =====================
    "mma": "MMA", "zuffa": "MMA",
    # ===================== ESPORTS =====================
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

# =========================
# POLYMARKET PARSER
# =========================

def extract_date_from_polymarket(event_data: dict) -> str:
    """
    Extract date in YYYY-MM-DD format from Polymarket event
    Uses endDate field with format: 2026-01-22T17:45:00Z
    """
    # Use endDate as the primary field (this is when the event ends/resolves)
    if event_data.get('endDate'):
        end_date = event_data['endDate']
        if isinstance(end_date, str) and 'T' in end_date:
            # Extract just the date part (YYYY-MM-DD)
            return end_date.split('T')[0]
    
    # Fallback to startDate if endDate is not available
    if event_data.get('startDate'):
        start_date = event_data['startDate']
        if isinstance(start_date, str):
            if 'T' in start_date:
                return start_date.split('T')[0]
            if len(start_date) >= 10:
                return start_date[:10]
    
    return None

def parse_polymarket_event(event_data: dict) -> dict:
    """Parse Polymarket event and extract relevant fields"""
    return {
        "platform": "polymarket",
        "slug": event_data.get('slug', ''),
        "title": event_data.get('title', ''),
        "date": extract_date_from_polymarket(event_data),
        "raw": event_data,
    }

# =========================
# KALSHI PARSER
# =========================

def parse_kalshi_event_id(event_id: str) -> dict:
    """
    Parse Kalshi event ticker to extract date
    Format: KXNFL2TD-26JAN25LASEA
    Date is: YYMMMDD (26JAN25)
    """
    if "-" not in event_id:
        return {"date": None}

    prefix, payload = event_id.upper().split("-", 1)

    if not prefix.startswith("KX"):
        return {"date": None}
    
    date = None
    # The date format is YYMMMDD (e.g., 26JAN25)
    if len(payload) >= 7:
        year_part = payload[:2]
        month_part = payload[2:5]
        day_part = payload[5:7]
        
        if day_part.isdigit() and month_part in MONTH_MAP and year_part.isdigit():
            year = "20" + year_part
            month = MONTH_MAP[month_part]
            day = day_part
            date = f"{year}-{month}-{day}"
    
    return {"date": date}

def parse_kalshi_event(event_data: dict) -> dict:
    """Parse Kalshi event and extract relevant fields"""
    event_id = event_data.get('event_ticker', '')
    parsed = parse_kalshi_event_id(event_id)
    
    return {
        "platform": "kalshi",
        "event_id": event_id,
        "title": event_data.get('title', ''),
        "date": parsed["date"],
        "raw": event_data,
    }

# =========================
# NORMALIZATION
# =========================

def normalize_title(s: str) -> str:
    """Normalize title for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_game_name(title: str, platform: str) -> str:
    """Extract the game name from the event title."""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        else:
            return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        else:
            return title
    return title

# =========================
# MARKET TYPE INFERENCE
# =========================

def infer_kalshi_type(title):
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title):
    return "more_markets" if "More Markets" in title else "moneyline"

# =========================
# MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=80):
    matches = []

    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])

        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])

            # ---- TYPE COMPATIBILITY ----
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue

            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type
                })

    # UPDATED: Greedy matching with support for 1:many for "more_markets"
    # Sort by score descending
    matches.sort(key=lambda x: x["score"], reverse=True)
    used_kalshi = set()
    final_matches = []
    for match in matches:
        kalshi_id = id(match["kalshi"])  # Use object ID to uniquely identify Kalshi events
        if kalshi_id not in used_kalshi:
            final_matches.append({
                "polymarket": match["poly"],
                "kalshi": match["kalshi"],
                "match_score": match["score"]
            })
            # Only mark Kalshi as used; allow "more_markets" Poly to match multiple Kalshi
            used_kalshi.add(kalshi_id)

    return final_matches

# =========================
# FETCH KALSHI EVENTS
# =========================

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"

print("="*80)
print("STEP 1: FETCHING KALSHI EVENTS")
print("="*80 + "\n")

response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
filters_data = response.json()

all_sports = [
    s for s in filters_data.get("sport_ordering", [])
    if s != "All sports"
]

kalshi_events_by_sport = defaultdict(list)

for sport in all_sports:
    print(f"Fetching {sport}...", end=" ")
    
    response = requests.get(
        f"{KALSHI_API}/series",
        params={"category": "Sports", "tags": sport}
    )
    series_data = response.json().get("series", [])
    tickers = [s["ticker"] for s in series_data if s.get("ticker")]
    
    count = 0
    for ticker in tickers:
        cursor = None
        
        while True:
            params = {
                "status": "open",
                "series_ticker": ticker,
                "limit": 200,
            }
            if cursor:
                params["cursor"] = cursor

            r = requests.get(f"{KALSHI_API}/events", params=params).json()
            events = r.get("events", [])
            if not events:
                break

            for e in events:
                parsed = parse_kalshi_event(e)
                kalshi_events_by_sport[sport].append(parsed)
                count += 1

            cursor = r.get("cursor")
            if not cursor:
                break
    
    print(f"{count} events")

# =========================
# FETCH POLYMARKET EVENTS
# =========================

GAMMA_API = "https://gamma-api.polymarket.com"

print("\n" + "="*80)
print("STEP 2: FETCHING POLYMARKET EVENTS")
print("="*80 + "\n")

response = requests.get(f"{GAMMA_API}/sports")
sports_data_all = response.json()

poly_events_by_kalshi_sport = defaultdict(list)

valid_series = []
for item in sports_data_all:
    series = item.get("series")
    league_code = item.get("sport")

    if not series or not league_code:
        continue

    try:
        series = int(series)
    except (TypeError, ValueError):
        continue

    league_code = league_code.lower()
    kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
    
    if not kalshi_sport:
        continue

    valid_series.append((series, kalshi_sport, league_code))

print(f"Found {len(valid_series)} valid Polymarket series\n")

for series, kalshi_sport, league_code in valid_series:
    print(f"Fetching {league_code} → {kalshi_sport}...", end=" ")
    page = 0
    limit = 50
    count = 0

    while True:
        response = requests.get(
            f"{GAMMA_API}/events",
            params={
                "series_id": series,
                "active": True,
                "closed": False,
                "order": "startTime",
                "ascending": True,
                "limit": limit,
                "offset": page * limit
            }
        )

        events = response.json()
        if not events:
            break

        for e in events:
            parsed = parse_polymarket_event(e)
            poly_events_by_kalshi_sport[kalshi_sport].append(parsed)
            count += 1

        page += 1
    
    print(f"{count} events")

# =========================
# STEP 3: GROUP BY SPORT (NO DATE FILTER)
# =========================

print("\n" + "="*80)
print("STEP 3: GROUPING EVENTS BY SPORT")
print("="*80 + "\n")

events_by_sport = {}

for sport in all_sports:
    print(f"\n{sport}:")
    print("-" * 80)
    
    kalshi_evts = kalshi_events_by_sport.get(sport, [])
    poly_evts = poly_events_by_kalshi_sport.get(sport, [])
    
    if not (kalshi_evts or poly_evts):
        print("  No events found")
        continue
    
    print(f"  Total: {len(kalshi_evts)} Kalshi, {len(poly_evts)} Polymarket")
    
    events_by_sport[sport] = {"kalshi": kalshi_evts, "polymarket": poly_evts}

# =========================
# STEP 4: FUZZY MATCHING WITHIN EACH SPORT
# =========================

print("\n" + "="*80)
print("STEP 4: FUZZY MATCHING WITHIN EACH SPORT")
print("="*80 + "\n")

SIMILARITY_THRESHOLD = 80
all_matches = []

for sport, events in events_by_sport.items():
    if not events:
        continue
    
    print(f"\n{sport}:")
    print("-" * 80)
    
    poly_events = events["polymarket"]
    kalshi_events = events["kalshi"]
    
    print(f"  Comparing {len(poly_events)} Poly vs {len(kalshi_events)} Kalshi events")
    
    # Perform matching
    matched_pairs = match_events(poly_events, kalshi_events, SIMILARITY_THRESHOLD)
    
    # Add metadata and collect results
    for match in matched_pairs:
        poly_evt = match['polymarket']
        kalshi_evt = match['kalshi']
        score = match['match_score']
        
        print(f"    ✓ MATCH (score: {score})")
        print(f"      Poly:   {poly_evt['title']}")
        print(f"      Kalshi: {kalshi_evt['title']}")
        
        all_matches.append({
            "sport": sport,
            "date": poly_evt.get('date', ''),  # Use Poly date if available, else empty
            "poly_title": poly_evt['title'],
            "poly_slug": poly_evt.get('slug', ''),
            "kalshi_title": kalshi_evt['title'],
            "kalshi_event_id": kalshi_evt.get('event_id', ''),
            "match_score": score,
        })
    
    print(f"\n  → Total matches for {sport}: {len(matched_pairs)}")

# =========================
# STEP 5: SAVE RESULTS
# =========================

print("\n" + "="*80)
print("STEP 5: SAVING RESULTS")
print("="*80 + "\n")

if all_matches:
    df = pd.DataFrame(all_matches)
    
    print("\nSUMMARY STATISTICS:")
    print(f"  Total matched pairs: {len(all_matches)}")
    print(f"\nBy sport:")
    print(df['sport'].value_counts().to_string())
    
    print(f"\nMatch score distribution:")
    print(df['match_score'].describe())
    
    print(f"\nSAMPLE MATCHES (first 10):")
    print(df[['sport', 'date', 'poly_title', 'kalshi_title', 'match_score']].head(10).to_string())
    
else:
    print("No matches found!")

print("\n" + "="*80)
print("COMPLETE")
print("="*80)

STEP 1: FETCHING KALSHI EVENTS



Fetching Football... 46 events
Fetching Basketball... 321 events
Fetching Hockey... 96 events
Fetching Soccer... 390 events
Fetching Tennis... 13 events
Fetching Golf... 4 events
Fetching MMA... 2 events
Fetching Baseball... 39 events
Fetching Boxing... 0 events
Fetching Chess... 1 events
Fetching Cricket... 6 events
Fetching Esports... 55 events
Fetching Motorsport... 6 events
Fetching Olympics... 3 events

STEP 2: FETCHING POLYMARKET EVENTS

Found 119 valid Polymarket series

Fetching ncaab → Basketball... 0 events
Fetching epl → Soccer... 38 events
Fetching lal → Soccer... 43 events
Fetching acn → Soccer... 0 events
Fetching ipl → Cricket... 0 events
Fetching wnba → Basketball... 0 events
Fetching bun → Soccer... 40 events
Fetching mlb → Baseball... 0 events
Fetching cfb → Football... 0 events
Fetching nfl → Football... 1 events
Fetching fl1 → Soccer... 21 events
Fetching sea → Soccer... 34 events
Fetching ucl → Soccer... 36 events
Fetching afc → Soccer... 0 events
Fetching ofc → So

As the above is good enough and 371 events, not markets are matched, we try below to include the underlying market matching

## Now we make Markets also match for each event between POLY & KALSHI

In [12]:
import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

POLY_LEAGUE_TO_KALSHI_SPORT = {
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    "nfl": "Football", "cfb": "Football",
    "mlb": "Baseball", "kbo": "Baseball",
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    "atp": "Tennis", "wta": "Tennis",
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    "mma": "MMA", "zuffa": "MMA",
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"

# =========================
# HELPER FUNCTIONS
# =========================
def normalize_kalshi_subtitle(subtitle: str) -> str:
    """
    Normalize Kalshi subtitle for matching
    Examples:
    - "Leeds United" → "leeds united"
    - "Tie" → "tie draw"
    """
    normalized = normalize_title(subtitle)
    
    # Map tie to include draw for better matching
    if normalized == "tie":
        return "draw tie"
    
    return normalized

def normalize_title(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_game_name(title: str, platform: str) -> str:
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def infer_kalshi_type(title):
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title):
    return "more_markets" if "More Markets" in title else "moneyline"

def parse_outcome_prices(raw):
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        else:
            return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        else:
            return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# =========================
# FETCH POLYMARKET DATA
# =========================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    outcomes = market.get("outcomes", [])
                    
                    if not question or not parsed_prices:
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": outcomes
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

# =========================
# FETCH KALSHI DATA
# =========================

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport

# =========================
# EVENT MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=75):
    """Match events between platforms with lower threshold for more matches"""
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            
            # Type compatibility checks
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            
            if poly_type == "more_markets" and kal_type == "moneyline":
                continue
            
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue
            
            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type
                })
    
    matches.sort(key=lambda x: x["score"], reverse=True)
    
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        if kalshi_id in used_kalshi:
            continue
        
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING
# =========================

def extract_outcome_from_poly_question(question: str) -> str:
    """
    Extract the outcome/team name from Polymarket question
    Examples:
    - "Will Everton FC win on 2026-01-26?" → "everton"
    - "Will Arsenal FC vs Leeds United FC end in a draw?" → "draw"
    - "Will Leeds United FC win..." → "leeds united"
    """
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return "draw tie"
    
    # Extract team name from "Will TEAM win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        # Remove common suffixes
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return team
        # Fallback: just normalize the whole question
    return normalize_title(question)

def match_markets_within_event(poly_markets, kalshi_markets, threshold=80):
    """
    Match markets within a matched event pair with improved logic
    Uses outcome extraction rather than full question fuzzy matching
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market["question"]
        poly_outcome = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = normalize_kalshi_subtitle(kalshi_market["title"])
            
            # Use token_set_ratio for flexible matching
            score = fuzz.token_set_ratio(poly_outcome, kalshi_subtitle)
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        # Only add if threshold met
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    # Remove duplicates - if same Kalshi market matched multiple times, keep best
    kalshi_to_best = {}
    for m in matched:
        kalshi_id = id(m["kalshi_market"])
        if kalshi_id not in kalshi_to_best or m["score"] > kalshi_to_best[kalshi_id]["score"]:
            kalshi_to_best[kalshi_id] = m
    
    return list(kalshi_to_best.values())


# =========================
# MAIN EXECUTION
# =========================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

# Collect all matched data
all_matched_data = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=75)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        # Match markets within this event pair
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            threshold=75  # Lowered for more matches
        )
        
        # Collect data for each matched market
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            # Extract prices
            poly_prices = poly_market.get("outcome_prices", [None, None])
            poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
            poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            # Calculate spread
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_market["question"],
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "poly_slug": poly_event.get("slug"),
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

# Create DataFrame
df_matched_markets = pd.DataFrame(all_matched_data)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")
print(f"\nTotal matched markets: {len(df_matched_markets)}")

if len(df_matched_markets) > 0:
    print(f"\nMatched markets by sport:")
    print(df_matched_markets['sport'].value_counts())
    
    print(f"\nSample of matched markets:")
    display_cols = [
        'sport', 'poly_event_title', 'kalshi_event_title',
        'poly_market_question', 'kalshi_market_title',
        'poly_yes_price', 'kalshi_yes_price', 'price_spread'
    ]
    print(df_matched_markets[display_cols].head(10).to_string(index=False))
    
    print(f"\nDataFrame shape: {df_matched_markets.shape}")
    print(f"Columns: {list(df_matched_markets.columns)}")


Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

Total matched markets: 425

Matched markets by sport:
sport
Soccer        254
Basketball    129
Hockey         20
Esports        11
Tennis          8
Cricket         3
Name: count, dtype: int64

Sample of matched markets:
 sport                            poly_event_title       kalshi_event_title                             poly_market_question kalshi_market_title  poly_yes_price  kalshi_yes_price  price_spread
Soccer                   Real Oviedo vs. Girona FC         Oviedo vs Girona              Will Real Oviedo win on 2026-01-31?              Oviedo           0.355              0.34         0.015
Soccer                   Real Oviedo vs. Girona FC         Oviedo vs Girona    Will Real Oviedo vs. Girona FC end in a draw?                 Tie           0.305              0.29         0.015
Soccer                   Real Oviedo vs. Girona FC         Oviedo

In [13]:
df_matched_markets

,sport,poly_event_title,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,poly_slug,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,Real Oviedo vs. Girona FC,Oviedo vs Girona,Will Real Oviedo win on 2026-01-31?,Oviedo,0.355,0.645,0.34,0.63,0.015,100,lal-ovi-gir-2026-01-31,KXLALIGAGAME-26JAN31OVIGIR,1211860,KXLALIGAGAME-26JAN31OVIGIR-OVI
1,Soccer,Real Oviedo vs. Girona FC,Oviedo vs Girona,Will Real Oviedo vs. Girona FC end in a draw?,Tie,0.305,0.695,0.29,0.68,0.015,100,lal-ovi-gir-2026-01-31,KXLALIGAGAME-26JAN31OVIGIR,1211863,KXLALIGAGAME-26JAN31OVIGIR-TIE
2,Soccer,Real Oviedo vs. Girona FC,Oviedo vs Girona,Will Girona FC win on 2026-01-31?,Girona,0.340,0.660,0.32,0.65,0.020,100,lal-ovi-gir-2026-01-31,KXLALIGAGAME-26JAN31OVIGIR,1211866,KXLALIGAGAME-26JAN31OVIGIR-GIR
3,Soccer,CA Osasuna vs. Villarreal CF,Osasuna vs Villarreal,Will CA Osasuna win on 2026-01-31?,Osasuna,0.325,0.675,0.31,0.66,0.015,100,lal-osa-vil-2026-01-31,KXLALIGAGAME-26JAN31OSAVIL,1211874,KXLALIGAGAME-26JAN31OSAVIL-OSA
4,Soccer,CA Osasuna vs. Villarreal CF,Osasuna vs Villarreal,Will CA Osasuna vs. Villarreal CF end in a draw?,Tie,0.285,0.715,0.27,0.71,0.015,100,lal-osa-vil-2026-01-31,KXLALIGAGAME-26JAN31OSAVIL,1211875,KXLALIGAGAME-26JAN31OSAVIL-TIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,Tennis,Australian Open Women's: Jessica Pegula vs Ama...,Pegula vs Anisimova,Australian Open Women's: Jessica Pegula vs Ama...,Jessica Pegula,0.455,0.545,0.45,0.54,0.005,100,wta-pegula-anisimo-2026-01-27,KXWTAMATCH-26JAN26PEGANI,1269672,KXWTAMATCH-26JAN26PEGANI-PEG
421,Tennis,Australian Open Women's: Elena Rybakina vs Iga...,Rybakina vs Swiatek,Australian Open Women's: Elena Rybakina vs Iga...,Iga Swiatek,0.535,0.465,0.48,0.51,0.055,100,wta-rybakin-swiatek-2026-01-28,KXWTAMATCH-26JAN26RYBSWI,1270989,KXWTAMATCH-26JAN26RYBSWI-SWI
422,Cricket,International ODI: Sri Lanka vs England (Game 3),Sri Lanka vs England,International ODI: Sri Lanka vs England (Game 3),Sri Lanka,0.430,0.570,0.41,0.55,0.020,100,crint-lka-gbr-2026-01-27,KXCRICKETODIMATCH-26JAN27ENGSRI,1225148,KXCRICKETODIMATCH-26JAN27ENGSRI-SRI
423,Cricket,International T20 Series: South Africa vs West...,South Africa vs West Indies,International T20 Series: South Africa vs West...,West Indies,0.755,0.245,0.22,0.76,0.535,100,crint-zaf-wst-2026-01-27,KXCRICKETT20IMATCH-26JAN27WISA,1232541,KXCRICKETT20IMATCH-26JAN27WISA-WI


In [14]:
import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

POLY_LEAGUE_TO_KALSHI_SPORT = {
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    "nfl": "Football", "cfb": "Football",
    "mlb": "Baseball", "kbo": "Baseball",
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    "atp": "Tennis", "wta": "Tennis",
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    "mma": "MMA", "zuffa": "MMA",
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"

# =========================
# HELPER FUNCTIONS
# =========================

def normalize_title(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_game_name(title: str, platform: str) -> str:
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def infer_kalshi_type(title):
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title):
    return "more_markets" if "More Markets" in title else "moneyline"

def parse_outcome_prices(raw):
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        else:
            return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        else:
            return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# =========================
# FETCH POLYMARKET DATA
# =========================

def fetch_polymarket_events_with_markets():
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    outcomes = market.get("outcomes", [])
                    
                    if not question or not parsed_prices:
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": outcomes
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

# =========================
# FETCH KALSHI DATA
# =========================

def fetch_kalshi_events_with_markets():
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport

# =========================
# EVENT MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=70):
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            
            if poly_type == "more_markets" and kal_type == "moneyline":
                continue
            
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue
            
            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type
                })
    
    matches.sort(key=lambda x: x["score"], reverse=True)
    
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        if kalshi_id in used_kalshi:
            continue
        
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING
# =========================

def extract_outcome_from_poly_question(question: str) -> str:
    question_lower = question.lower()
    
    if "draw" in question_lower or "tie" in question_lower:
        return "draw tie"
    
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return team
    
    return normalize_title(question)

def normalize_kalshi_subtitle(subtitle: str) -> str:
    normalized = normalize_title(subtitle)
    if normalized == "tie":
        return "draw tie"
    return normalized

def match_markets_within_event(poly_markets, kalshi_markets, threshold=70):
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market["question"]
        poly_outcome = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = normalize_kalshi_subtitle(kalshi_market["title"])
            score = fuzz.token_set_ratio(poly_outcome, kalshi_subtitle)
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    kalshi_to_best = {}
    for m in matched:
        kalshi_id = id(m["kalshi_market"])
        if kalshi_id not in kalshi_to_best or m["score"] > kalshi_to_best[kalshi_id]["score"]:
            kalshi_to_best[kalshi_id] = m
    
    return list(kalshi_to_best.values())

# =========================
# MAIN EXECUTION
# =========================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=70)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        # Track event match
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"]
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            threshold=70
        )
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            poly_prices = poly_market.get("outcome_prices", [None, None])
            poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
            poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_market["question"],
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "poly_slug": poly_event.get("slug"),
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

# Create DataFrames
df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")

print(f"\n🔍 SAMPLE EVENT WITH ALL MARKETS:")
if len(df_matched_markets) > 0:
    sample_event = df_matched_markets.iloc[0]['poly_event_title']
    sample_data = df_matched_markets[df_matched_markets['poly_event_title'] == sample_event]
    display_cols = ['poly_market_question', 'kalshi_market_title', 'poly_yes_price', 'kalshi_yes_price', 'price_spread']
    print(f"\nEvent: {sample_event}")
    print(f"Total markets matched: {len(sample_data)}")
    print(sample_data[display_cols].to_string(index=False))

# The results are stored in:
# - df_matched_markets: All matched markets with prices
# - df_event_summary: Summary of matched events with market counts

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 509

Events by sport:
sport
Soccer        222
Basketball    212
Esports        34
Hockey         27
Tennis         11
Cricket         3
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 542
Average markets per event: 1.06

📈 MARKETS PER EVENT BREAKDOWN:
Events with 1 market: 287
Events with 2 markets: 30
Events with 3+ markets: 61

🔍 SAMPLE EVENT WITH ALL MARKETS:

Event: Leeds United FC vs. Arsenal FC
Total markets matched: 3
                              poly_market_question kalshi_market_title  poly_yes_price  kalshi_yes_price  price_spread
           Will Leeds United FC win on 2026-01-31?        Leeds United           0.145              0.15         0.005
                Will Arsenal FC win on 2026-01-31?             Arsenal           0.625              0.61         0.015
Will Leed

In [15]:
df_event_summary

,sport,poly_event_title,kalshi_event_title,poly_markets_count,kalshi_markets_count,event_match_score
0,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,3,3,100
1,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,3,3,100
2,Soccer,Brighton & Hove Albion FC vs. Everton FC,Brighton vs Everton,3,3,100
3,Soccer,Chelsea FC vs. West Ham United FC,Chelsea vs West Ham,3,3,100
4,Soccer,Liverpool FC vs. Newcastle United FC,Liverpool vs Newcastle,3,3,100
...,...,...,...,...,...,...
504,Tennis,Australian Open Men's: Vit Kopriva vs Jan-Lenn...,Men's Australian Open Winner,12,23,86
505,Tennis,Australian Open Women's: Renee Alame vs Xinran...,Bax vs Sun,9,2,75
506,Cricket,International ODI: Sri Lanka vs England (Game 3),Sri Lanka vs England,1,2,100
507,Cricket,International T20 Series: South Africa vs West...,South Africa vs West Indies,3,2,100


In [16]:
df_matched_markets

,sport,poly_event_title,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,poly_slug,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Leeds United FC win on 2026-01-31?,Leeds United,0.145,0.855,0.15,0.84,0.005,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209694,KXEPLGAME-26JAN31LEEARS-LEE
1,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Arsenal FC win on 2026-01-31?,Arsenal,0.625,0.375,0.61,0.37,0.015,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209696,KXEPLGAME-26JAN31LEEARS-ARS
2,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Leeds United FC vs. Arsenal FC end in a d...,Tie,0.225,0.775,0.22,0.76,0.005,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209695,KXEPLGAME-26JAN31LEEARS-TIE
3,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,Will Wolverhampton Wanderers FC win on 2026-01...,Wolverhampton,0.295,0.705,0.28,0.69,0.015,100,epl-wol-bou-2026-01-31,KXEPLGAME-26JAN31WOLBOU,1209697,KXEPLGAME-26JAN31WOLBOU-WOL
4,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,Will AFC Bournemouth win on 2026-01-31?,Bournemouth,0.435,0.565,0.44,0.55,0.005,100,epl-wol-bou-2026-01-31,KXEPLGAME-26JAN31WOLBOU,1209699,KXEPLGAME-26JAN31WOLBOU-BOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,Tennis,Australian Open Women's: Jessica Pegula vs Ama...,Pegula vs Anisimova,Pegula vs. Anisimova: Set 1 Games O/U 8.5,Amanda Anisimova,0.710,0.290,0.54,0.44,0.170,72,wta-pegula-anisimo-2026-01-27,KXWTAMATCH-26JAN26PEGANI,1269995,KXWTAMATCH-26JAN26PEGANI-ANI
538,Tennis,Australian Open Women's: Elena Rybakina vs Iga...,Rybakina vs Swiatek,Australian Open Women's: Elena Rybakina vs Iga...,Iga Swiatek,0.545,0.455,0.48,0.51,0.065,100,wta-rybakin-swiatek-2026-01-28,KXWTAMATCH-26JAN26RYBSWI,1270989,KXWTAMATCH-26JAN26RYBSWI-SWI
539,Cricket,International ODI: Sri Lanka vs England (Game 3),Sri Lanka vs England,International ODI: Sri Lanka vs England (Game 3),Sri Lanka,0.430,0.570,0.41,0.55,0.020,100,crint-lka-gbr-2026-01-27,KXCRICKETODIMATCH-26JAN27ENGSRI,1225148,KXCRICKETODIMATCH-26JAN27ENGSRI-SRI
540,Cricket,International T20 Series: South Africa vs West...,South Africa vs West Indies,International T20 Series: South Africa vs West...,West Indies,0.755,0.245,0.22,0.76,0.535,100,crint-zaf-wst-2026-01-27,KXCRICKETT20IMATCH-26JAN27WISA,1232541,KXCRICKETT20IMATCH-26JAN27WISA-WI


In [17]:
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

df_matched_markets.to_csv(f'matched_markets_{timestamp}.csv', index=False)
df_event_summary.to_csv(f'event_summary_{timestamp}.csv', index=False)

print(f"✓ Saved matched_markets_{timestamp}.csv")
print(f"✓ Saved event_summary_{timestamp}.csv")

✓ Saved matched_markets_20260126_232306.csv
✓ Saved event_summary_20260126_232306.csv


The matching seems to work well but only 512 events are matched (10k approx on poly) and 509 markets (soccer should have at least 3 for each event)

Next steps: \
1/ Increase the number of matched events -> markets \
2/  Implement an arbitrage check for the actual bid/ask prices on both platforms \
3/ Websocket connection to both platforms especially to catch changes in live games \
4/ Expanding comparison to all markets/events not only sports as that is very competitive

In [6]:
import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

# =========================
# CONFIGURATION
# =========================

POLY_LEAGUE_TO_KALSHI_SPORT = {
    # Soccer
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # Basketball
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # Football
    "nfl": "Football", "cfb": "Football",
    # Baseball
    "mlb": "Baseball", "kbo": "Baseball",
    # Hockey
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # Tennis
    "atp": "Tennis", "wta": "Tennis",
    # Cricket
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # Rugby
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # MMA
    "mma": "MMA", "zuffa": "MMA",
    # Esports
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"

# =========================
# UTILITY FUNCTIONS
# =========================

def normalize_title(s: str) -> str:
    """Normalize text for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def parse_outcome_prices(raw):
    """Parse Polymarket outcome prices"""
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    """Parse Polymarket token IDs"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def parse_outcomes(raw):
    """
    CRITICAL FIX: Parse Polymarket outcomes array
    Outcomes are often JSON strings, not arrays!
    """
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# =========================
# EVENT TYPE DETECTION
# =========================

def extract_game_name(title: str, platform: str) -> str:
    """Extract game name from event title"""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def infer_kalshi_type(title: str) -> str:
    """Infer Kalshi event type from title"""
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title: str) -> str:
    """Infer Polymarket event type from title"""
    return "more_markets" if "More Markets" in title else "moneyline"

def is_soccer_sport(sport: str) -> bool:
    """Check if sport uses soccer-style matching"""
    return sport.lower() == "soccer"

# =========================
# POLYMARKET DATA FETCHING
# =========================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets, grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    raw_outcomes = market.get("outcomes", [])
                    parsed_outcomes = parse_outcomes(raw_outcomes)  # CRITICAL FIX: Parse outcomes!
                    
                    if not question or not parsed_prices:
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": parsed_outcomes  # Now properly parsed!
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

# =========================
# KALSHI DATA FETCHING
# =========================

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets, grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport

# =========================
# EVENT MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=70):
    """Match events between Polymarket and Kalshi with strict type checking"""
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            
            # STRICT TYPE COMPATIBILITY
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            if poly_type == "more_markets" and kal_type == "moneyline":
                continue
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue
            
            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type
                })
    
    matches.sort(key=lambda x: x["score"], reverse=True)
    
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        if kalshi_id in used_kalshi:
            continue
        
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING - SOCCER
# =========================

def extract_outcome_from_poly_question(question: str) -> str:
    """
    UPDATED: Extract outcome from Polymarket question (Soccer only)
    Now handles Over/Under for More Markets
    """
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return "draw tie"
    
    # CHANGE 1: Check for over/under (totals) - NEW!
    if "over" in question_lower or "o/u" in question_lower:
        return "over under totals"
    
    # Extract team name from "Will [TEAM] win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return team
    
    return normalize_title(question)

def normalize_kalshi_subtitle(subtitle: str) -> str:
    """
    UPDATED: Normalize Kalshi subtitle for matching
    Now maps totals/goals to common format
    """
    normalized = normalize_title(subtitle)
    
    # Map tie variations
    if normalized == "tie":
        return "draw tie"
    
    # CHANGE 2: Map over/under/goals variations - NEW!
    if "over" in normalized or "under" in normalized or "goals" in normalized:
        return "over under totals"
    
    return normalized

def match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60):
    """
    UPDATED: Match markets for Soccer using question-based matching
    CHANGE 3: Lower threshold (60 instead of 70) for better totals matching
    ALWAYS uses questions, even for More Markets (totals, spreads)
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market["question"]
        poly_outcome = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = normalize_kalshi_subtitle(kalshi_market["title"])
            score = fuzz.token_set_ratio(poly_outcome, kalshi_subtitle)
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score,
                "poly_outcome": poly_outcome
            })
    
    kalshi_to_best = {}
    for m in matched:
        kalshi_id = id(m["kalshi_market"])
        if kalshi_id not in kalshi_to_best or m["score"] > kalshi_to_best[kalshi_id]["score"]:
            kalshi_to_best[kalshi_id] = m
    
    return list(kalshi_to_best.values())

# =========================
# MARKET MATCHING - OTHER SPORTS
# =========================

def match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70):
    """
    UPDATED: Match markets for Basketball/NFL/Hockey/Tennis/Cricket using outcome-based matching
    CRITICAL FIX: Lower threshold (70 instead of 80) to handle name variations
    Example: "Maryland Eastern Shore Hawks" vs "Maryland-Eastern Shore"
    """
    matched = []
    
    for poly_market in poly_markets:
        outcomes = poly_market.get("outcomes", [])
        outcome_prices = poly_market.get("outcome_prices", [])
        
        # Skip if no outcomes
        if not outcomes:
            continue
        
        for i, outcome in enumerate(outcomes):
            outcome_normalized = normalize_title(outcome)
            outcome_price = outcome_prices[i] if i < len(outcome_prices) else None
            
            best_match = None
            best_score = 0
            
            for kalshi_market in kalshi_markets:
                kalshi_subtitle = normalize_title(kalshi_market["title"])
                score = fuzz.token_set_ratio(outcome_normalized, kalshi_subtitle)
                
                if score > best_score:
                    best_score = score
                    best_match = kalshi_market
            
            # Lower threshold to catch name variations
            if best_match and best_score >= threshold:
                matched.append({
                    "poly_market": poly_market,
                    "kalshi_market": best_match,
                    "score": best_score,
                    "poly_outcome": outcome,
                    "poly_outcome_price": outcome_price,
                    "poly_outcome_index": i
                })
    
    # DON'T deduplicate - allow multiple outcomes to match different Kalshi markets
    return matched

# =========================
# MARKET MATCHING ROUTER
# =========================

def match_markets_within_event(poly_markets, kalshi_markets, sport, threshold=70):
    """
    UPDATED: Route to appropriate matching function based on sport
    CRITICAL FIX: Lower threshold for outcomes (70 instead of 80) to handle name variations
    """
    if is_soccer_sport(sport):
        return match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60)
    else:
        return match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70)

# =========================
# MAIN EXECUTION
# =========================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []
debug_no_markets = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=70)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"]
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            sport,
            threshold=70
        )
        
        if not market_matches:
            debug_no_markets.append({
                'sport': sport,
                'poly_event': poly_event["title"],
                'kalshi_event': kalshi_event["title"],
                'poly_markets': len(poly_event["markets"]),
                'kalshi_markets': len(kalshi_event["markets"]),
                'sample_poly_question': poly_event["markets"][0]["question"] if poly_event["markets"] else "N/A",
                'sample_poly_outcomes': str(poly_event["markets"][0].get("outcomes", [])) if poly_event["markets"] else "N/A",
                'sample_kalshi_title': kalshi_event["markets"][0]["title"] if kalshi_event["markets"] else "N/A"
            })
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            if not is_soccer_sport(sport) and "poly_outcome_price" in market_match:
                poly_yes_price = market_match["poly_outcome_price"]
                poly_no_price = 1 - poly_yes_price if poly_yes_price else None
                poly_display_question = f"{poly_market['question']} - {market_match['poly_outcome']}"
            else:
                poly_prices = poly_market.get("outcome_prices", [None, None])
                poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
                poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
                poly_display_question = poly_market["question"]
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_display_question,
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "poly_slug": poly_event.get("slug"),
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)
df_debug_no_markets = pd.DataFrame(debug_no_markets)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
if len(df_event_summary) > 0:
    print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:")
print(f"Total events with 0 markets matched: {len(df_debug_no_markets)}")
if len(df_debug_no_markets) > 0:
    print(f"\nDistribution by sport:")
    print(df_debug_no_markets['sport'].value_counts())
    print(f"\nFirst 5 problematic events:")
    debug_cols = ['sport', 'poly_event', 'kalshi_event', 'sample_poly_question', 'sample_poly_outcomes', 'sample_kalshi_title']
    print(df_debug_no_markets[debug_cols].head(5).to_string(index=False, max_colwidth=40))

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
if len(df_matched_markets) > 0:
    markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
    print(f"Events with 0 markets: {len(df_event_summary) - len(markets_per_event)}")
    print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
    print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
    print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")

# FINAL OUTPUT: 3 DataFrames available
# - df_matched_markets: All matched markets with prices
# - df_event_summary: Summary of matched events  
# - df_debug_no_markets: Events that failed market matching

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 606

Events by sport:
sport
Soccer        271
Basketball    270
Esports        39
Hockey         14
Tennis          9
Cricket         3
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 1241
Average markets per event: 2.05

⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:
Total events with 0 markets matched: 32

Distribution by sport:
sport
Esports       25
Hockey         4
Tennis         2
Basketball     1
Name: count, dtype: int64

First 5 problematic events:
     sport                               poly_event                             kalshi_event                     sample_poly_question                     sample_poly_outcomes sample_kalshi_title
Basketball California Golden Bears vs. North Car...                     Golden State at Utah California Golden Bears vs. North Car... ['Califor

In [8]:
df_matched_markets

,sport,poly_event_title,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,poly_slug,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Leeds United FC win on 2026-01-31?,Leeds United,0.150,0.850,0.14,0.84,0.010,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209694,KXEPLGAME-26JAN31LEEARS-LEE
1,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Arsenal FC win on 2026-01-31?,Arsenal,0.625,0.375,0.63,0.36,0.005,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209696,KXEPLGAME-26JAN31LEEARS-ARS
2,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Leeds United FC vs. Arsenal FC end in a d...,Tie,0.225,0.775,0.22,0.77,0.005,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209695,KXEPLGAME-26JAN31LEEARS-TIE
3,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,Will Wolverhampton Wanderers FC win on 2026-01...,Wolverhampton,0.295,0.705,0.28,0.69,0.015,100,epl-wol-bou-2026-01-31,KXEPLGAME-26JAN31WOLBOU,1209697,KXEPLGAME-26JAN31WOLBOU-WOL
4,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,Will AFC Bournemouth win on 2026-01-31?,Bournemouth,0.435,0.565,0.45,0.54,0.015,100,epl-wol-bou-2026-01-31,KXEPLGAME-26JAN31WOLBOU,1209699,KXEPLGAME-26JAN31WOLBOU-BOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,Cricket,International T20 Series: South Africa vs West...,South Africa vs West Indies,International T20 Series: South Africa vs West...,West Indies,0.505,0.495,0.24,0.70,0.265,100,crint-zaf-wst-2026-01-27,KXCRICKETT20IMATCH-26JAN27WISA,1232546,KXCRICKETT20IMATCH-26JAN27WISA-WI
1237,Cricket,International T20 Series: India vs New Zealand...,India vs New Zealand,International T20 Series: India vs New Zealand...,India,0.755,0.245,0.75,0.22,0.005,100,crint-ind-nzl-2026-01-28,KXCRICKETT20IMATCH-26JAN28NZIND,1234797,KXCRICKETT20IMATCH-26JAN28NZIND-IND
1238,Cricket,International T20 Series: India vs New Zealand...,India vs New Zealand,International T20 Series: India vs New Zealand...,New Zealand,0.245,0.755,0.24,0.73,0.005,100,crint-ind-nzl-2026-01-28,KXCRICKETT20IMATCH-26JAN28NZIND,1234797,KXCRICKETT20IMATCH-26JAN28NZIND-NZ
1239,Cricket,International T20 Series: India vs New Zealand...,India vs New Zealand,International T20 Series: India vs New Zealand...,India,0.505,0.495,0.75,0.22,0.245,100,crint-ind-nzl-2026-01-28,KXCRICKETT20IMATCH-26JAN28NZIND,1234798,KXCRICKETT20IMATCH-26JAN28NZIND-IND


In [9]:
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

df_matched_markets.to_csv(f'matched_markets_{timestamp}.csv', index=False)
df_event_summary.to_csv(f'event_summary_{timestamp}.csv', index=False)

print(f"✓ Saved matched_markets_{timestamp}.csv")
print(f"✓ Saved event_summary_{timestamp}.csv")

✓ Saved matched_markets_20260127_094447.csv
✓ Saved event_summary_20260127_094447.csv


3rd trial

In [20]:
import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

# =========================
# CONFIGURATION
# =========================

POLY_LEAGUE_TO_KALSHI_SPORT = {
    # Soccer
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # Basketball
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # Football
    "nfl": "Football", "cfb": "Football",
    # Baseball
    "mlb": "Baseball", "kbo": "Baseball",
    # Hockey
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # Tennis
    "atp": "Tennis", "wta": "Tennis",
    # Cricket
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # Rugby
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # MMA
    "mma": "MMA", "zuffa": "MMA",
    # Esports
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"

# =========================
# UTILITY FUNCTIONS
# =========================

def normalize_title(s: str) -> str:
    """Normalize text for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalize_kalshi_subtitle(subtitle: str) -> str:
    """
    More precise normalization for Kalshi soccer markets
    Distinguishes between totals and spreads
    """
    normalized = normalize_title(subtitle)
    
    # Map tie variations
    if normalized == "tie":
        return "draw tie"
    
    # CRITICAL: Distinguish totals from spreads
    # Totals: "Over 3.5 goals scored", "Under 2.5 goals"
    if ("over" in normalized or "under" in normalized) and "goal" in normalized:
        return "total goals"
    
    # Spreads: "Team wins by over 2.5", "Team cover", "Team -1.5"
    if "wins by" in normalized or "cover" in normalized or "-" in subtitle or "+" in subtitle:
        return "spread handicap"
    
    return normalized

def parse_outcome_prices(raw):
    """Parse Polymarket outcome prices"""
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    """Parse Polymarket token IDs"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def parse_outcomes(raw):
    """
    CRITICAL FIX: Parse Polymarket outcomes array
    Outcomes are often JSON strings, not arrays!
    """
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# =========================
# EVENT TYPE DETECTION
# =========================

def extract_game_name(title: str, platform: str) -> str:
    """Extract game name from event title"""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def infer_kalshi_type(title: str) -> str:
    """Infer Kalshi event type from title"""
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title: str) -> str:
    """Infer Polymarket event type from title"""
    return "more_markets" if "More Markets" in title else "moneyline"

def is_soccer_sport(sport: str) -> bool:
    """Check if sport uses soccer-style matching"""
    return sport.lower() == "soccer"

def extract_outcome_from_poly_question(question: str) -> str:
    """Extract the key outcome identifier from a Polymarket question"""
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return "draw tie"
    
    # CRITICAL: Distinguish between totals and spreads
    # Totals have "o/u", "over", "under" with goals/points
    if ("o/u" in question_lower or "over" in question_lower or "under" in question_lower):
        # Check if it mentions goals (totals) vs spread
        if "goal" in question_lower or "score" in question_lower:
            return "total goals"
        # If no "goal" mentioned but has over/under, still likely totals
        return "total goals"
    
    # Spread markets mention "handicap" or specific +/- numbers
    if "handicap" in question_lower or "spread" in question_lower:
        return "spread handicap"
    
    # Extract team name from "Will [TEAM] win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return team
    
    return normalize_title(question)

# =========================
# POLYMARKET DATA FETCHING
# =========================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets, grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    raw_outcomes = market.get("outcomes", [])
                    parsed_outcomes = parse_outcomes(raw_outcomes)
                    
                    if not question or not parsed_prices:
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": parsed_outcomes
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

# =========================
# KALSHI DATA FETCHING
# =========================

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets, grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport

# =========================
# EVENT MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=70):
    """Match events between Polymarket and Kalshi with strict type checking"""
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            
            # STRICT TYPE COMPATIBILITY
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            if poly_type == "more_markets" and kal_type == "moneyline":
                continue
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue
            
            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type
                })
    
    matches.sort(key=lambda x: x["score"], reverse=True)
    
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        if kalshi_id in used_kalshi:
            continue
        
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING - SOCCER
# =========================

def match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60):
    """
    Match markets within a matched soccer event using outcome-based matching.
    Soccer typically has: moneyline (team1/draw/team2), totals (over/under goals), 
    spreads (handicap), and other prop markets.
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market.get("question", "")
        poly_outcome = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = kalshi_market.get("title", "")
            kalshi_outcome = normalize_kalshi_subtitle(kalshi_subtitle)
            
            # Compare normalized outcomes
            score = fuzz.token_set_ratio(poly_outcome, kalshi_outcome)
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    # Deduplicate: ensure each Kalshi market is only matched once
    used_kalshi = set()
    final_matches = []
    
    for match in sorted(matched, key=lambda x: x["score"], reverse=True):
        kalshi_id = match["kalshi_market"].get("market_id")
        if kalshi_id not in used_kalshi:
            final_matches.append(match)
            used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING - OTHER SPORTS
# =========================

def match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70):
    """
    UPDATED: Match markets for Basketball/NFL/Hockey/Tennis/Cricket using outcome-based matching
    CRITICAL FIX: Lower threshold (70 instead of 80) to handle name variations
    Example: "Maryland Eastern Shore Hawks" vs "Maryland-Eastern Shore"
    """
    matched = []
    
    for poly_market in poly_markets:
        outcomes = poly_market.get("outcomes", [])
        outcome_prices = poly_market.get("outcome_prices", [])
        
        # Skip if no outcomes
        if not outcomes:
            continue
        
        for i, outcome in enumerate(outcomes):
            outcome_normalized = normalize_title(outcome)
            outcome_price = outcome_prices[i] if i < len(outcome_prices) else None
            
            best_match = None
            best_score = 0
            
            for kalshi_market in kalshi_markets:
                kalshi_subtitle = normalize_title(kalshi_market["title"])
                score = fuzz.token_set_ratio(outcome_normalized, kalshi_subtitle)
                
                if score > best_score:
                    best_score = score
                    best_match = kalshi_market
            
            # Lower threshold to catch name variations
            if best_match and best_score >= threshold:
                matched.append({
                    "poly_market": poly_market,
                    "kalshi_market": best_match,
                    "score": best_score,
                    "poly_outcome": outcome,
                    "poly_outcome_price": outcome_price,
                    "poly_outcome_index": i
                })
    
    # DON'T deduplicate - allow multiple outcomes to match different Kalshi markets
    return matched

# =========================
# MARKET MATCHING ROUTER
# =========================

def match_markets_within_event(poly_markets, kalshi_markets, sport, threshold=70):
    """
    UPDATED: Route to appropriate matching function based on sport
    CRITICAL FIX: Lower threshold for outcomes (70 instead of 80) to handle name variations
    """
    if is_soccer_sport(sport):
        return match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60)
    else:
        return match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70)

# =========================
# MAIN EXECUTION
# =========================


print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []
debug_no_markets = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=70)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"]
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            sport,
            threshold=70
        )
        
        if not market_matches:
            debug_no_markets.append({
                'sport': sport,
                'poly_event': poly_event["title"],
                'kalshi_event': kalshi_event["title"],
                'poly_markets': len(poly_event["markets"]),
                'kalshi_markets': len(kalshi_event["markets"]),
                'sample_poly_question': poly_event["markets"][0]["question"] if poly_event["markets"] else "N/A",
                'sample_poly_outcomes': str(poly_event["markets"][0].get("outcomes", [])) if poly_event["markets"] else "N/A",
                'sample_kalshi_title': kalshi_event["markets"][0]["title"] if kalshi_event["markets"] else "N/A"
            })
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            if not is_soccer_sport(sport) and "poly_outcome_price" in market_match:
                poly_yes_price = market_match["poly_outcome_price"]
                poly_no_price = 1 - poly_yes_price if poly_yes_price else None
                poly_display_question = f"{poly_market['question']} - {market_match['poly_outcome']}"
            else:
                poly_prices = poly_market.get("outcome_prices", [None, None])
                poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
                poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
                poly_display_question = poly_market["question"]
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_display_question,
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "poly_slug": poly_event.get("slug"),
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)
df_debug_no_markets = pd.DataFrame(debug_no_markets)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
if len(df_event_summary) > 0:
    print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:")
print(f"Total events with 0 markets matched: {len(df_debug_no_markets)}")
if len(df_debug_no_markets) > 0:
    print(f"\nDistribution by sport:")
    print(df_debug_no_markets['sport'].value_counts())
    print(f"\nFirst 5 problematic events:")
    debug_cols = ['sport', 'poly_event', 'kalshi_event', 'sample_poly_question', 'sample_poly_outcomes', 'sample_kalshi_title']
    print(df_debug_no_markets[debug_cols].head(5).to_string(index=False, max_colwidth=40))

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
if len(df_matched_markets) > 0:
    markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
    print(f"Events with 0 markets: {len(df_event_summary) - len(markets_per_event)}")
    print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
    print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
    print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 608

Events by sport:
sport
Basketball    277
Soccer        257
Esports        37
Hockey         27
Tennis          7
Cricket         3
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 1205
Average markets per event: 1.98

⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:
Total events with 0 markets matched: 35

Distribution by sport:
sport
Esports    24
Soccer      5
Hockey      4
Tennis      1
Cricket     1
Name: count, dtype: int64

First 5 problematic events:
 sport                               poly_event                             kalshi_event                   sample_poly_question             sample_poly_outcomes sample_kalshi_title
Soccer Sunderland AFC vs. Burnley FC - More ...                    Sunderland vs Burnley          Spread: Sunderland AFC (-1.5) ['Sunderland AFC', 'Burnley

In [ ]:
# Filter for specific sport
df_matched_markets[df_matched_markets['sport'] == 'Soccer'].head()

# Find arbitrage opportunities (large price spreads)
#df_matched_markets[df_matched_markets['price_spread'] > 0.05].sort_values('price_spread', ascending=False)

,sport,poly_event_title,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,poly_slug,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Leeds United FC win on 2026-01-31?,Leeds United,0.150,0.850,0.14,0.84,0.010,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209694,KXEPLGAME-26JAN31LEEARS-LEE
1,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Arsenal FC win on 2026-01-31?,Arsenal,0.625,0.375,0.63,0.36,0.005,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209696,KXEPLGAME-26JAN31LEEARS-ARS
2,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,Will Leeds United FC vs. Arsenal FC end in a d...,Tie,0.230,0.770,0.21,0.77,0.020,100,epl-lee-ars-2026-01-31,KXEPLGAME-26JAN31LEEARS,1209695,KXEPLGAME-26JAN31LEEARS-TIE
3,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,Will Wolverhampton Wanderers FC win on 2026-01...,Wolverhampton,0.305,0.695,0.29,0.69,0.015,100,epl-wol-bou-2026-01-31,KXEPLGAME-26JAN31WOLBOU,1209697,KXEPLGAME-26JAN31WOLBOU-WOL
4,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,Will AFC Bournemouth win on 2026-01-31?,Bournemouth,0.435,0.565,0.41,0.56,0.025,100,epl-wol-bou-2026-01-31,KXEPLGAME-26JAN31WOLBOU,1209699,KXEPLGAME-26JAN31WOLBOU-BOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,Soccer,Liverpool FC vs. Qarabağ Ağdam FK - More Markets,Qarabag at Liverpool: Both Teams to Score,Liverpool FC vs. Qarabağ Ağdam FK: Both Teams ...,Both Teams To Score,0.420,0.580,0.40,0.56,0.020,100,ucl-liv1-qar1-2026-01-28-more-markets,KXUCLBTTS-26JAN28LFCQAR,1193715,KXUCLBTTS-26JAN28LFCQAR
405,Soccer,Liverpool FC vs. Qarabağ Ağdam FK - More Markets,Qarabag at Liverpool: Totals,Liverpool FC vs. Qarabağ Ağdam FK: O/U 1.5,Over 3.5 goals scored,0.875,0.125,0.50,0.49,0.375,100,ucl-liv1-qar1-2026-01-28-more-markets,KXUCLTOTAL-26JAN28LFCQAR,1193711,KXUCLTOTAL-26JAN28LFCQAR-3
406,Soccer,Carabao Cup Winner,2026 Men's World Cup winner?,Will Wolves win the 2025–26 Carabao Cup?,Wales,0.000,1.000,NaN,0.99,NaN,73,carabao-cup-winner,KXMENWORLDCUP-26,581574,KXMENWORLDCUP-26-WA
407,Soccer,Carabao Cup Winner,2026 Men's World Cup winner?,Will Arsenal win the 2025–26 Carabao Cup?,Senegal,0.460,0.540,NaN,0.99,NaN,71,carabao-cup-winner,KXMENWORLDCUP-26,581560,KXMENWORLDCUP-26-SN


The code above is too permissive and matches wrong events

In [ ]:
# ============================================================================
# CELL 1: IMPORTS AND CONFIGURATION
# ============================================================================

import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

POLY_LEAGUE_TO_KALSHI_SPORT = {
    # Soccer
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # Basketball
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # Football
    "nfl": "Football", "cfb": "Football",
    # Baseball
    "mlb": "Baseball", "kbo": "Baseball",
    # Hockey
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # Tennis
    "atp": "Tennis", "wta": "Tennis",
    # Cricket
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # Rugby
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # MMA
    "mma": "MMA", "zuffa": "MMA",
    # Esports
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"


# ============================================================================
# CELL 2: UTILITY FUNCTIONS
# ============================================================================

def normalize_title(s: str) -> str:
    """Normalize text for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_number_from_text(text: str) -> float:
    """Extract a decimal number from text (e.g., '1.5', '3.5', '2', etc.)"""
    match = re.search(r'\b(\d+\.?\d*)\b', text)
    if match:
        return float(match.group(1))
    return None

def extract_totals_info(text: str) -> dict:
    """
    Extract totals information from market text.
    Returns: {'type': 'totals', 'number': 3.5, 'direction': 'over'/'under'}
    """
    text_lower = text.lower()
    
    # Check if this is a totals market
    if not any(keyword in text_lower for keyword in ['o/u', 'over', 'under', 'total', 'goal']):
        return None
    
    # Extract the number
    number = extract_number_from_text(text_lower)
    if number is None:
        return None
    
    # Determine direction (if specified)
    direction = None
    if 'over' in text_lower and 'under' not in text_lower:
        direction = 'over'
    elif 'under' in text_lower and 'over' not in text_lower:
        direction = 'under'
    
    return {
        'type': 'totals',
        'number': number,
        'direction': direction
    }

def normalize_kalshi_subtitle(subtitle: str) -> str:
    """More precise normalization for Kalshi soccer markets"""
    normalized = normalize_title(subtitle)
    
    if normalized == "tie":
        return "draw tie"
    
    # Check if it's a totals market
    totals_info = extract_totals_info(subtitle)
    if totals_info:
        return f"total goals {totals_info['number']}"
    
    # Spreads
    if "wins by" in normalized or "cover" in normalized or "-" in subtitle or "+" in subtitle:
        return "spread handicap"
    
    # BTTS
    if "both teams" in normalized and "score" in normalized:
        return "both teams to score"
    
    return normalized

def extract_outcome_from_poly_question(question: str) -> dict:
    """
    Extract outcome information from Polymarket question.
    Returns dict with 'type' and relevant info for matching.
    """
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return {'type': 'draw', 'normalized': 'draw tie'}
    
    # Check for totals
    totals_info = extract_totals_info(question)
    if totals_info:
        return {
            'type': 'totals',
            'number': totals_info['number'],
            'direction': totals_info['direction'],
            'normalized': f"total goals {totals_info['number']}"
        }
    
    # Check for BTTS
    if "both teams" in question_lower and "score" in question_lower:
        return {'type': 'btts', 'normalized': 'both teams to score'}
    
    # Check for spread/handicap
    if "handicap" in question_lower or "spread" in question_lower:
        return {'type': 'spread', 'normalized': 'spread handicap'}
    
    # Extract team name from "Will [TEAM] win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return {'type': 'team', 'team': team, 'normalized': normalize_title(team)}
    
    return {'type': 'other', 'normalized': normalize_title(question)}

def parse_outcome_prices(raw):
    """Parse Polymarket outcome prices"""
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    """Parse Polymarket token IDs"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def parse_outcomes(raw):
    """Parse Polymarket outcomes array"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def extract_game_name(title: str, platform: str) -> str:
    """Extract game name from event title"""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def extract_team_names(title: str) -> set:
    """
    Extract team names from event title for better matching.
    Returns a set of normalized team name tokens.
    """
    # Remove common suffixes
    title = re.sub(r'\s+(FC|United|City|AFC|CF|vs\.?|at)\s+', ' ', title, flags=re.IGNORECASE)
    title = normalize_title(title)
    
    # Split into tokens and remove very short words
    tokens = [t for t in title.split() if len(t) > 2]
    return set(tokens)

def infer_kalshi_type(title: str) -> str:
    """Infer Kalshi event type from title"""
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or "over" in t or "under" in t:
        return "totals"
    if "spread" in t or "cover" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title: str) -> str:
    """Infer Polymarket event type from title"""
    return "more_markets" if "More Markets" in title else "moneyline"

def is_soccer_sport(sport: str) -> bool:
    """Check if sport uses soccer-style matching"""
    return sport.lower() == "soccer"


# ============================================================================
# CELL 3: DATA FETCHING FUNCTIONS
# ============================================================================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets, grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    raw_outcomes = market.get("outcomes", [])
                    parsed_outcomes = parse_outcomes(raw_outcomes)
                    
                    if not question or not parsed_prices:
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": parsed_outcomes
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets, grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport


# ============================================================================
# CELL 4: MATCHING FUNCTIONS
# ============================================================================

def match_events(poly_events, kalshi_events, threshold=75):
    """
    Match events between Polymarket and Kalshi with strict type checking.
    UPDATED: Increased threshold to 75 and added team name overlap check.
    """
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        poly_teams = extract_team_names(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            kal_teams = extract_team_names(k["title"])
            
            # STRICT TYPE COMPATIBILITY
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            if poly_type == "more_markets" and kal_type == "moneyline":
                continue
            if poly_type == "more_markets" and kal_type not in ["totals", "spreads", "btts"]:
                continue
            
            # Calculate fuzzy match score
            score = fuzz.token_set_ratio(poly_game, kal_game)
            
            # ADDITIONAL CHECK: Require at least 2 matching team name tokens
            team_overlap = len(poly_teams & kal_teams)
            if team_overlap < 2:
                continue  # Skip matches without sufficient team overlap
            
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type,
                    "team_overlap": team_overlap
                })
    
    matches.sort(key=lambda x: (x["team_overlap"], x["score"]), reverse=True)
    
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        if kalshi_id in used_kalshi:
            continue
        
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

def match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60):
    """
    Match markets within a matched soccer event.
    UPDATED: Strict totals matching - numbers must match exactly!
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market.get("question", "")
        poly_outcome_info = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = kalshi_market.get("title", "")
            kalshi_normalized = normalize_kalshi_subtitle(kalshi_subtitle)
            
            # CRITICAL: For totals, extract the number from Kalshi market
            kalshi_totals_info = extract_totals_info(kalshi_subtitle)
            
            # Type matching
            poly_normalized = poly_outcome_info.get('normalized', '')
            
            # STRICT TOTALS MATCHING: Numbers must match!
            if poly_outcome_info.get('type') == 'totals':
                if kalshi_totals_info is None:
                    continue  # Kalshi market is not a totals market
                
                poly_number = poly_outcome_info.get('number')
                kalshi_number = kalshi_totals_info.get('number')
                
                # Numbers must match exactly
                if poly_number != kalshi_number:
                    continue
                
                # If numbers match, give high score
                score = 100
            else:
                # For non-totals, use fuzzy matching
                score = fuzz.token_set_ratio(poly_normalized, kalshi_normalized)
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    # Deduplicate
    used_kalshi = set()
    final_matches = []
    
    for match in sorted(matched, key=lambda x: x["score"], reverse=True):
        kalshi_id = match["kalshi_market"].get("market_id")
        if kalshi_id not in used_kalshi:
            final_matches.append(match)
            used_kalshi.add(kalshi_id)
    
    return final_matches

def match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70):
    """Match markets for non-soccer sports using outcome-based matching"""
    matched = []
    
    for poly_market in poly_markets:
        outcomes = poly_market.get("outcomes", [])
        outcome_prices = poly_market.get("outcome_prices", [])
        
        if not outcomes:
            continue
        
        for i, outcome in enumerate(outcomes):
            outcome_normalized = normalize_title(outcome)
            outcome_price = outcome_prices[i] if i < len(outcome_prices) else None
            
            best_match = None
            best_score = 0
            
            for kalshi_market in kalshi_markets:
                kalshi_subtitle = normalize_title(kalshi_market["title"])
                score = fuzz.token_set_ratio(outcome_normalized, kalshi_subtitle)
                
                if score > best_score:
                    best_score = score
                    best_match = kalshi_market
            
            if best_match and best_score >= threshold:
                matched.append({
                    "poly_market": poly_market,
                    "kalshi_market": best_match,
                    "score": best_score,
                    "poly_outcome": outcome,
                    "poly_outcome_price": outcome_price,
                    "poly_outcome_index": i
                })
    
    return matched

def match_markets_within_event(poly_markets, kalshi_markets, sport, threshold=70):
    """Route to appropriate matching function based on sport"""
    if is_soccer_sport(sport):
        return match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60)
    else:
        return match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70)


# ============================================================================
# CELL 5: MAIN EXECUTION
# ============================================================================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []
debug_no_markets = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=75)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"],
            'team_overlap': event_match.get("team_overlap", 0)
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            sport,
            threshold=70
        )
        
        if not market_matches:
            debug_no_markets.append({
                'sport': sport,
                'poly_event': poly_event["title"],
                'kalshi_event': kalshi_event["title"],
                'poly_markets': len(poly_event["markets"]),
                'kalshi_markets': len(kalshi_event["markets"]),
                'sample_poly_question': poly_event["markets"][0]["question"] if poly_event["markets"] else "N/A",
                'sample_poly_outcomes': str(poly_event["markets"][0].get("outcomes", [])) if poly_event["markets"] else "N/A",
                'sample_kalshi_title': kalshi_event["markets"][0]["title"] if kalshi_event["markets"] else "N/A"
            })
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            if not is_soccer_sport(sport) and "poly_outcome_price" in market_match:
                poly_yes_price = market_match["poly_outcome_price"]
                poly_no_price = 1 - poly_yes_price if poly_yes_price else None
                poly_display_question = f"{poly_market['question']} - {market_match['poly_outcome']}"
            else:
                poly_prices = poly_market.get("outcome_prices", [None, None])
                poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
                poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
                poly_display_question = poly_market["question"]
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_display_question,
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "poly_slug": poly_event.get("slug"),
                "poly_market_slug": poly_market.get("slug"),
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)
df_debug_no_markets = pd.DataFrame(debug_no_markets)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
if len(df_event_summary) > 0:
    print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
if len(df_event_summary) > 0:
    print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:")
print(f"Total events with 0 markets matched: {len(df_debug_no_markets)}")
if len(df_debug_no_markets) > 0:
    print(f"\nDistribution by sport:")
    print(df_debug_no_markets['sport'].value_counts())
    print(f"\nFirst 5 problematic events:")
    debug_cols = ['sport', 'poly_event', 'kalshi_event', 'sample_poly_question', 'sample_poly_outcomes', 'sample_kalshi_title']
    print(df_debug_no_markets[debug_cols].head(5).to_string(index=False, max_colwidth=40))

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
if len(df_matched_markets) > 0:
    markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
    print(f"Events with 0 markets: {len(df_event_summary) - len(markets_per_event)}")
    print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
    print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
    print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")


# ============================================================================
# CELL 6 (OPTIONAL): VIEW AND VALIDATE RESULTS
# ============================================================================

# Display first 20 matches
print("\n📋 SAMPLE OF MATCHED MARKETS:")
if len(df_matched_markets) > 0:
    display_cols = ['sport', 'poly_event_title', 'poly_market_question', 'kalshi_market_title', 
                    'poly_yes_price', 'kalshi_yes_price', 'price_spread', 'market_match_score']
    print(df_matched_markets[display_cols].head(20).to_string(index=False, max_colwidth=50))

# Check for potential arbitrage opportunities
print("\n💰 POTENTIAL ARBITRAGE OPPORTUNITIES (price_spread > 0.05):")
if len(df_matched_markets) > 0:
    arb_opps = df_matched_markets[df_matched_markets['price_spread'] > 0.05].sort_values('price_spread', ascending=False)
    if len(arb_opps) > 0:
        print(arb_opps[display_cols].head(10).to_string(index=False, max_colwidth=50))
    else:
        print("No significant arbitrage opportunities found.")

# Verify totals matching for soccer
print("\n⚽ SOCCER TOTALS VERIFICATION:")
if len(df_matched_markets) > 0:
    soccer_totals = df_matched_markets[
        (df_matched_markets['sport'] == 'Soccer') & 
        (df_matched_markets['poly_market_question'].str.contains('O/U|over|under', case=False, na=False))
    ]
    if len(soccer_totals) > 0:
        print(soccer_totals[['poly_market_question', 'kalshi_market_title', 'market_match_score']].head(10).to_string(index=False, max_colwidth=60))
    else:
        print("No soccer totals markets found in matches.")

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 497

Events by sport:
sport
Basketball    278
Soccer        174
Hockey         22
Esports        11
Tennis          8
Cricket         4
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 1241
Average markets per event: 2.50

⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:
Total events with 0 markets matched: 18

Distribution by sport:
sport
Soccer        13
Tennis         2
Basketball     1
Hockey         1
Cricket        1
Name: count, dtype: int64

First 5 problematic events:
 sport                               poly_event                        kalshi_event                     sample_poly_question             sample_poly_outcomes                  sample_kalshi_title
Soccer TSG 1899 Hoffenheim vs. 1. FC Union B...  Union Berlin at Hoffenheim: Totals TSG 1899 Hoffenheim vs. 1. FC Union B...  

issue here above is still having some false matches for totals<->spreads

In [35]:
df_matched_markets

,sport,poly_event_title,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,poly_slug,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,Chelsea FC vs. West Ham United FC,Chelsea vs West Ham,Will Chelsea FC vs. West Ham United FC end in ...,Tie,0.2050,0.7950,0.20,0.78,0.0050,100,epl-che-wes-2026-01-31,KXEPLGAME-26JAN31CFCWHU,1209704,KXEPLGAME-26JAN31CFCWHU-TIE
1,Soccer,Chelsea FC vs. West Ham United FC,Chelsea vs West Ham,Will Chelsea FC win on 2026-01-31?,Chelsea,0.6400,0.3600,0.63,0.34,0.0100,100,epl-che-wes-2026-01-31,KXEPLGAME-26JAN31CFCWHU,1209703,KXEPLGAME-26JAN31CFCWHU-CFC
2,Soccer,Chelsea FC vs. West Ham United FC,Chelsea vs West Ham,Will West Ham United FC win on 2026-01-31?,West Ham,0.1500,0.8500,0.14,0.85,0.0100,100,epl-che-wes-2026-01-31,KXEPLGAME-26JAN31CFCWHU,1209705,KXEPLGAME-26JAN31CFCWHU-WHU
3,Soccer,Nottingham Forest FC vs. Crystal Palace FC,Nottingham vs Crystal Palace,Will Nottingham Forest FC win on 2026-02-01?,Nottingham,0.4950,0.5050,0.48,0.50,0.0150,100,epl-not-cry-2026-02-01,KXEPLGAME-26FEB01NFOCRY,1215416,KXEPLGAME-26FEB01NFOCRY-NFO
4,Soccer,Nottingham Forest FC vs. Crystal Palace FC,Nottingham vs Crystal Palace,Will Crystal Palace FC win on 2026-02-01?,Crystal Palace,0.2400,0.7600,0.23,0.75,0.0100,100,epl-not-cry-2026-02-01,KXEPLGAME-26FEB01NFOCRY,1215418,KXEPLGAME-26FEB01NFOCRY-CRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,Cricket,International ODI: Sri Lanka vs England (Game 3),Sri Lanka vs England,International ODI: Sri Lanka vs England (Game ...,England,0.9995,0.0005,0.99,NaN,0.0095,100,crint-lka-gbr-2026-01-27,KXCRICKETODIMATCH-26JAN27ENGSRI,1225148,KXCRICKETODIMATCH-26JAN27ENGSRI-ENG
1237,Cricket,International T20 Series: India vs New Zealand...,India vs New Zealand,International T20 Series: India vs New Zealand...,India,0.7700,0.2300,0.75,0.22,0.0200,100,crint-ind-nzl-2026-01-28,KXCRICKETT20IMATCH-26JAN28NZIND,1234797,KXCRICKETT20IMATCH-26JAN28NZIND-IND
1238,Cricket,International T20 Series: India vs New Zealand...,India vs New Zealand,International T20 Series: India vs New Zealand...,New Zealand,0.2300,0.7700,0.24,0.75,0.0100,100,crint-ind-nzl-2026-01-28,KXCRICKETT20IMATCH-26JAN28NZIND,1234797,KXCRICKETT20IMATCH-26JAN28NZIND-NZ
1239,Cricket,International T20 Series: India vs New Zealand...,India vs New Zealand,International T20 Series: India vs New Zealand...,India,0.5000,0.5000,0.75,0.22,0.2500,100,crint-ind-nzl-2026-01-28,KXCRICKETT20IMATCH-26JAN28NZIND,1234798,KXCRICKETT20IMATCH-26JAN28NZIND-IND


In [42]:
# ============================================================================
# CELL 1: IMPORTS AND CONFIGURATION
# ============================================================================

import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

POLY_LEAGUE_TO_KALSHI_SPORT = {
    # Soccer
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # Basketball
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # Football
    "nfl": "Football", "cfb": "Football",
    # Baseball
    "mlb": "Baseball", "kbo": "Baseball",
    # Hockey
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # Tennis
    "atp": "Tennis", "wta": "Tennis",
    # Cricket
    "ipl": "Cricket", "odi": "Cricket", "t20": "Cricket", "test": "Cricket",
    "abb": "Cricket", "csa": "Cricket", "she": "Cricket", "sasa": "Cricket",
    "lpl": "Cricket", "psp": "Cricket", "crban": "Cricket", "crint": "Cricket",
    "crind": "Cricket", "crpak": "Cricket", "creng": "Cricket", "cruae": "Cricket",
    "crnew": "Cricket", "craus": "Cricket", "crsou": "Cricket", "cru19wc": "Cricket",
    "crwpl20": "Cricket", "crwncl": "Cricket", "crwt20wcgq": "Cricket",
    "crafgwi20": "Cricket", "crbtnmlyhkg20": "Cricket",
    # Rugby
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # MMA
    "mma": "MMA", "zuffa": "MMA",
    # Esports
    "lol": "Esports", "dota2": "Esports", "cs2": "Esports", "val": "Esports",
    "sc2": "Esports", "sc": "Esports", "pubg": "Esports", "r6siege": "Esports",
    "rl": "Esports", "codmw": "Esports", "mlbb": "Esports", "ow": "Esports",
    "hok": "Esports", "wildrift": "Esports", "fifa": "Esports",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"


# ============================================================================
# CELL 2: UTILITY FUNCTIONS
# ============================================================================

def normalize_title(s: str) -> str:
    """Normalize text for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_number_from_text(text: str) -> float:
    """Extract a decimal number from text (e.g., '1.5', '3.5', '2', etc.)"""
    match = re.search(r'\b(\d+\.?\d*)\b', text)
    if match:
        return float(match.group(1))
    return None

def is_spread_market(text: str) -> bool:
    """Detect if this is a SPREAD market (team wins by X goals)."""
    text_lower = text.lower()
    if "wins by" in text_lower:
        return True
    if "cover" in text_lower:
        return True
    if "spread" in text_lower and "price spread" not in text_lower:
        return True
    return False

def is_totals_market(text: str) -> bool:
    """Detect if this is a TOTALS market (total goals by both teams)."""
    text_lower = text.lower()
    if not any(keyword in text_lower for keyword in ['o/u', 'over', 'under', 'total']):
        return False
    if is_spread_market(text):
        return False
    if "goal" in text_lower or "score" in text_lower or "point" in text_lower:
        return True
    return False

def extract_totals_info(text: str) -> dict:
    """Extract totals information from market text."""
    if not is_totals_market(text):
        return None
    
    text_lower = text.lower()
    number = extract_number_from_text(text_lower)
    if number is None:
        return None
    
    direction = None
    if 'over' in text_lower and 'under' not in text_lower:
        direction = 'over'
    elif 'under' in text_lower and 'over' not in text_lower:
        direction = 'under'
    
    return {
        'type': 'totals',
        'number': number,
        'direction': direction
    }

def normalize_kalshi_subtitle(subtitle: str) -> str:
    """More precise normalization for Kalshi soccer markets"""
    normalized = normalize_title(subtitle)
    
    if normalized == "tie":
        return "draw tie"
    
    # Check if it's a totals market
    totals_info = extract_totals_info(subtitle)
    if totals_info:
        return f"total goals {totals_info['number']}"
    
    # BTTS
    if "both teams" in normalized and "score" in normalized:
        return "both teams to score"
    
    return normalized

def extract_outcome_from_poly_question(question: str) -> dict:
    """Extract outcome information from Polymarket question."""
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return {'type': 'draw', 'normalized': 'draw tie'}
    
    # Check for totals (BEFORE checking for spreads)
    totals_info = extract_totals_info(question)
    if totals_info:
        return {
            'type': 'totals',
            'number': totals_info['number'],
            'direction': totals_info['direction'],
            'normalized': f"total goals {totals_info['number']}"
        }
    
    # Check for BTTS
    if "both teams" in question_lower and "score" in question_lower:
        return {'type': 'btts', 'normalized': 'both teams to score'}
    
    # Check for spread/handicap (AFTER totals check)
    if is_spread_market(question):
        return {'type': 'spread', 'normalized': 'spread handicap'}
    
    # Extract team name from "Will [TEAM] win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return {'type': 'team', 'team': team, 'normalized': normalize_title(team)}
    
    return {'type': 'other', 'normalized': normalize_title(question)}

def parse_outcome_prices(raw):
    """Parse Polymarket outcome prices"""
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    """Parse Polymarket token IDs"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def parse_outcomes(raw):
    """Parse Polymarket outcomes array"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def extract_game_name(title: str, platform: str) -> str:
    """Extract game name from event title"""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def extract_team_names(title: str) -> set:
    """Extract team names from event title for better matching."""
    title = re.sub(r'\s+(FC|United|City|AFC|CF|vs\.?|at)\s+', ' ', title, flags=re.IGNORECASE)
    title = normalize_title(title)
    tokens = [t for t in title.split() if len(t) > 2]
    return set(tokens)

def infer_kalshi_type(title: str) -> str:
    """Infer Kalshi event type from title"""
    t = title.lower()
    if "both teams to score" in t:
        return "btts"
    if "total" in t or ("over" in t and "goal" in t and "wins by" not in t):
        return "totals"
    if "spread" in t or "cover" in t or "wins by" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title: str) -> str:
    """Infer Polymarket event type from title"""
    return "more_markets" if "More Markets" in title else "moneyline"

def is_soccer_sport(sport: str) -> bool:
    """Check if sport uses soccer-style matching"""
    return sport.lower() == "soccer"


# ============================================================================
# CELL 3: DATA FETCHING FUNCTIONS
# ============================================================================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets, grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    raw_outcomes = market.get("outcomes", [])
                    parsed_outcomes = parse_outcomes(raw_outcomes)
                    
                    if not question or not parsed_prices:
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": parsed_outcomes
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets, grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport


# ============================================================================
# CELL 4: MATCHING FUNCTIONS
# ============================================================================

def match_events(poly_events, kalshi_events, threshold=75):
    """
    Match events between Polymarket and Kalshi.
    CRITICAL: One Poly "More Markets" can match MULTIPLE Kalshi events (Totals, BTTS, etc.)
    """
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        poly_teams = extract_team_names(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            kal_teams = extract_team_names(k["title"])
            
            # TYPE COMPATIBILITY
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            # For "more_markets", allow matching with totals, btts (EXCLUDE spreads)
            if poly_type == "more_markets" and kal_type not in ["totals", "btts"]:
                continue
            
            # Calculate fuzzy match score
            score = fuzz.token_set_ratio(poly_game, kal_game)
            
            # Require at least 2 matching team name tokens
            team_overlap = len(poly_teams & kal_teams)
            if team_overlap < 2:
                continue
            
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type,
                    "team_overlap": team_overlap
                })
    
    matches.sort(key=lambda x: (x["team_overlap"], x["score"]), reverse=True)
    
    # CRITICAL: For moneylines, use 1-to-1 matching
    # For "more_markets", use 1-to-MANY matching
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        # Kalshi events can only be matched once
        if kalshi_id in used_kalshi:
            continue
        
        # Moneyline poly events can only match once
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        # "More markets" poly events CAN match multiple Kalshi events
        # (e.g., one for Totals, one for BTTS)
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

def match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60):
    """
    Match markets within a matched soccer event.
    UPDATED: Strict totals matching - numbers must match exactly!
    EXCLUDES spreads entirely.
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market.get("question", "")
        poly_outcome_info = extract_outcome_from_poly_question(poly_question)
        poly_type = poly_outcome_info.get('type')
        
        # EXCLUDE SPREADS
        if poly_type == 'spread':
            continue
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = kalshi_market.get("title", "")
            
            # EXCLUDE SPREADS
            if is_spread_market(kalshi_subtitle):
                continue
            
            kalshi_normalized = normalize_kalshi_subtitle(kalshi_subtitle)
            kalshi_totals_info = extract_totals_info(kalshi_subtitle)
            
            poly_normalized = poly_outcome_info.get('normalized', '')
            
            # STRICT TOTALS MATCHING: Numbers must match!
            if poly_type == 'totals':
                if kalshi_totals_info is None:
                    continue
                
                poly_number = poly_outcome_info.get('number')
                kalshi_number = kalshi_totals_info.get('number')
                
                if poly_number != kalshi_number:
                    continue
                
                score = 100
            else:
                # For non-totals, use fuzzy matching
                score = fuzz.token_set_ratio(poly_normalized, kalshi_normalized)
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    # Deduplicate
    used_kalshi = set()
    final_matches = []
    
    for match in sorted(matched, key=lambda x: x["score"], reverse=True):
        kalshi_id = match["kalshi_market"].get("market_id")
        if kalshi_id not in used_kalshi:
            final_matches.append(match)
            used_kalshi.add(kalshi_id)
    
    return final_matches

def match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=80):
    """Match markets for non-soccer sports using outcome-based matching"""
    matched = []
    
    for poly_market in poly_markets:
        outcomes = poly_market.get("outcomes", [])
        outcome_prices = poly_market.get("outcome_prices", [])
        
        if not outcomes:
            continue
        
        for i, outcome in enumerate(outcomes):
            outcome_normalized = normalize_title(outcome)
            outcome_price = outcome_prices[i] if i < len(outcome_prices) else None
            
            best_match = None
            best_score = 0
            
            for kalshi_market in kalshi_markets:
                kalshi_subtitle = normalize_title(kalshi_market["title"])
                score = fuzz.token_set_ratio(outcome_normalized, kalshi_subtitle)
                
                if score > best_score:
                    best_score = score
                    best_match = kalshi_market
            
            if best_match and best_score >= threshold:
                matched.append({
                    "poly_market": poly_market,
                    "kalshi_market": best_match,
                    "score": best_score,
                    "poly_outcome": outcome,
                    "poly_outcome_price": outcome_price,
                    "poly_outcome_index": i
                })
    
    return matched

def match_markets_within_event(poly_markets, kalshi_markets, sport, threshold=70):
    """Route to appropriate matching function based on sport"""
    if is_soccer_sport(sport):
        return match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60)
    else:
        return match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=80)


# ============================================================================
# CELL 5: MAIN EXECUTION
# ============================================================================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []
debug_no_markets = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=75)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'poly_slug': poly_event.get("slug", ""),
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"],
            'team_overlap': event_match.get("team_overlap", 0)
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            sport,
            threshold=70
        )
        
        if not market_matches:
            debug_no_markets.append({
                'sport': sport,
                'poly_event': poly_event["title"],
                'poly_slug': poly_event.get("slug", ""),
                'kalshi_event': kalshi_event["title"],
                'poly_markets': len(poly_event["markets"]),
                'kalshi_markets': len(kalshi_event["markets"]),
                'sample_poly_question': poly_event["markets"][0]["question"] if poly_event["markets"] else "N/A",
                'sample_poly_outcomes': str(poly_event["markets"][0].get("outcomes", [])) if poly_event["markets"] else "N/A",
                'sample_kalshi_title': kalshi_event["markets"][0]["title"] if kalshi_event["markets"] else "N/A"
            })
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            if not is_soccer_sport(sport) and "poly_outcome_price" in market_match:
                poly_yes_price = market_match["poly_outcome_price"]
                poly_no_price = 1 - poly_yes_price if poly_yes_price else None
                poly_display_question = f"{poly_market['question']} - {market_match['poly_outcome']}"
            else:
                poly_prices = poly_market.get("outcome_prices", [None, None])
                poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
                poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
                poly_display_question = poly_market["question"]
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "poly_slug": poly_event.get("slug", ""),
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_display_question,
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)
df_debug_no_markets = pd.DataFrame(debug_no_markets)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
if len(df_event_summary) > 0:
    print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
if len(df_event_summary) > 0:
    print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:")
print(f"Total events with 0 markets matched: {len(df_debug_no_markets)}")
if len(df_debug_no_markets) > 0:
    print(f"\nDistribution by sport:")
    print(df_debug_no_markets['sport'].value_counts())
    print(f"\nFirst 10 problematic events:")
    debug_cols = ['sport', 'poly_event', 'poly_slug', 'kalshi_event', 'sample_poly_question', 'sample_kalshi_title']
    print(df_debug_no_markets[debug_cols].head(10).to_string(index=False, max_colwidth=50))

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
if len(df_matched_markets) > 0:
    markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
    print(f"Events with 0 markets: {len(df_event_summary) - len(markets_per_event)}")
    print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
    print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
    print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")


# ============================================================================
# CELL 6 (OPTIONAL): VIEW AND VALIDATE RESULTS
# ============================================================================

print("\n📋 SAMPLE OF MATCHED MARKETS:")
if len(df_matched_markets) > 0:
    display_cols = ['sport', 'poly_slug', 'poly_market_question', 'kalshi_market_title', 
                    'poly_yes_price', 'kalshi_yes_price', 'price_spread', 'market_match_score']
    print(df_matched_markets[display_cols].head(20).to_string(index=False, max_colwidth=50))

print("\n💰 POTENTIAL ARBITRAGE OPPORTUNITIES (price_spread > 0.05):")
if len(df_matched_markets) > 0:
    arb_opps = df_matched_markets[df_matched_markets['price_spread'] > 0.05].sort_values('price_spread', ascending=False)
    if len(arb_opps) > 0:
        print(arb_opps[display_cols].head(10).to_string(index=False, max_colwidth=50))
    else:
        print("No significant arbitrage opportunities found.")

print("\n⚽ SOCCER TOTALS VERIFICATION:")
if len(df_matched_markets) > 0:
    soccer_totals = df_matched_markets[
        (df_matched_markets['sport'] == 'Soccer') & 
        (df_matched_markets['poly_market_question'].str.contains('O/U', case=False, na=False))
    ]
    if len(soccer_totals) > 0:
        print(f"Total soccer totals markets matched: {len(soccer_totals)}")
        print(soccer_totals[['poly_slug', 'poly_market_question', 'kalshi_market_title', 'market_match_score']].head(15).to_string(index=False, max_colwidth=60))
        
        # Check for spread contamination
        bad_matches = soccer_totals[soccer_totals['kalshi_market_title'].str.contains('wins by', case=False, na=False)]
        if len(bad_matches) > 0:
            print(f"\n⚠️ WARNING: Found {len(bad_matches)} incorrect totals-to-spread matches!")
        else:
            print("\n✅ All totals correctly matched!")
        
        # Distribution
        soccer_totals_copy = soccer_totals.copy()
        soccer_totals_copy['totals_number'] = soccer_totals_copy['poly_market_question'].str.extract(r'O/U (\d+\.?\d*)', expand=False)
        print("\n📊 Distribution:")
        print(soccer_totals_copy['totals_number'].value_counts().sort_index())
    else:
        print("No soccer totals markets found.")

print("\n🚫 MATCHING RULES:")
print("✓ Spreads excluded entirely")
print("✓ Totals: exact number matching (1.5=1.5, 2.5=2.5, 3.5=3.5)")  
print("✓ One Poly 'More Markets' can match MULTIPLE Kalshi events (Totals, BTTS)")
print("✓ Moneylines: 1-to-1 matching")
print("✓ Soccer threshold=60, Non-soccer threshold=80")

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 464

Events by sport:
sport
Basketball    288
Soccer        130
Esports        23
Hockey         18
Tennis          5
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 1109
Average markets per event: 2.39

⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:
Total events with 0 markets matched: 20

Distribution by sport:
sport
Soccer        15
Basketball     3
Hockey         1
Tennis         1
Name: count, dtype: int64

First 10 problematic events:
 sport                                         poly_event                           poly_slug                       kalshi_event                               sample_poly_question   sample_kalshi_title
Soccer TSG 1899 Hoffenheim vs. 1. FC Union Berlin - Mo... bun-hof-uni-2026-01-31-more-markets Union Berlin at Hoffenheim: Totals TSG 1899 Hoffenheim vs. 

In [43]:
df_matched_markets

,sport,poly_event_title,poly_slug,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,FK Crvena zvezda vs. RC Celta de Vigo,uel-crv1-cel4-2026-01-29,Crvena Zvezda vs Celta Vigo,Will FK Crvena zvezda win on 2026-01-29?,Crvena Zvezda,0.425,0.575,0.41,0.56,0.015,100,KXUELGAME-26JAN29CZVRCC,1200076,KXUELGAME-26JAN29CZVRCC-CZV
1,Soccer,FK Crvena zvezda vs. RC Celta de Vigo,uel-crv1-cel4-2026-01-29,Crvena Zvezda vs Celta Vigo,Will FK Crvena zvezda vs. RC Celta de Vigo end...,Tie,0.270,0.730,0.26,0.72,0.010,100,KXUELGAME-26JAN29CZVRCC,1200077,KXUELGAME-26JAN29CZVRCC-TIE
2,Soccer,FK Crvena zvezda vs. RC Celta de Vigo,uel-crv1-cel4-2026-01-29,Crvena Zvezda vs Celta Vigo,Will RC Celta de Vigo win on 2026-01-29?,Celta Vigo,0.310,0.690,0.31,0.68,0.000,100,KXUELGAME-26JAN29CZVRCC,1200078,KXUELGAME-26JAN29CZVRCC-RCC
3,Soccer,MH Maccabi Tel Aviv vs. Bologna FC 1909,uel-mta1-bol1-2026-01-29,Maccabi Tel Aviv vs Bologna,Will MH Maccabi Tel Aviv win on 2026-01-29?,Maccabi Tel Aviv,0.145,0.855,0.13,0.85,0.015,100,KXUELGAME-26JAN29MTABFC,1200085,KXUELGAME-26JAN29MTABFC-MTA
4,Soccer,MH Maccabi Tel Aviv vs. Bologna FC 1909,uel-mta1-bol1-2026-01-29,Maccabi Tel Aviv vs Bologna,Will MH Maccabi Tel Aviv vs. Bologna FC 1909 e...,Tie,0.215,0.785,0.20,0.78,0.015,100,KXUELGAME-26JAN29MTABFC,1200086,KXUELGAME-26JAN29MTABFC-TIE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,Tennis,Australian Open Women's: Jessica Pegula vs Ele...,wta-pegula-rybakin-2026-01-29,Pegula vs Rybakina,Australian Open Women's: Jessica Pegula vs Ele...,Elena Rybakina,0.665,0.335,0.66,0.33,0.005,100,KXWTAMATCH-26JAN29PEGRYB,1281215,KXWTAMATCH-26JAN29PEGRYB-RYB
1105,Tennis,Australian Open Women's: Jessica Pegula vs Ele...,wta-pegula-rybakin-2026-01-29,Pegula vs Rybakina,Set 1 Winner: Pegula vs Rybakina - Pegula,Jessica Pegula,0.620,0.380,0.33,0.66,0.290,100,KXWTAMATCH-26JAN29PEGRYB,1281552,KXWTAMATCH-26JAN29PEGRYB-PEG
1106,Tennis,Australian Open Women's: Jessica Pegula vs Ele...,wta-pegula-rybakin-2026-01-29,Pegula vs Rybakina,Set 1 Winner: Pegula vs Rybakina - Rybakina,Elena Rybakina,0.380,0.620,0.66,0.33,0.280,100,KXWTAMATCH-26JAN29PEGRYB,1281552,KXWTAMATCH-26JAN29PEGRYB-RYB
1107,Tennis,Australian Open Women's: Jessica Pegula vs Ele...,wta-pegula-rybakin-2026-01-29,Pegula vs Rybakina,Set Handicap: Rybakina (-1.5) vs Pegula (+1.5)...,Elena Rybakina,0.460,0.540,0.66,0.33,0.200,100,KXWTAMATCH-26JAN29PEGRYB,1281554,KXWTAMATCH-26JAN29PEGRYB-RYB


In [44]:
from datetime import datetime

df_matched_markets.to_csv(f'matched_markets.csv', index=False)

print(f"✓ Saved matched_markets_{timestamp}.csv")

✓ Saved matched_markets_20260127_102732.csv


In [45]:
import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

# =========================
# CONFIGURATION
# =========================

POLY_LEAGUE_TO_KALSHI_SPORT = {
    # Soccer ONLY - exclude Tennis, Cricket, Esports
    "epl": "Soccer", "lal": "Soccer", "fl1": "Soccer", "bun": "Soccer",
    "sea": "Soccer", "ere": "Soccer", "por": "Soccer", "tur": "Soccer",
    "mls": "Soccer", "bra": "Soccer", "arg": "Soccer", "mex": "Soccer",
    "jap": "Soccer", "ja2": "Soccer", "kor": "Soccer", "spl": "Soccer",
    "chi": "Soccer", "aus": "Soccer", "ind": "Soccer", "nor": "Soccer",
    "den": "Soccer", "rus": "Soccer", "egy1": "Soccer", "cze1": "Soccer",
    "ucl": "Soccer", "uel": "Soccer", "col": "Soccer", "cdr": "Soccer",
    "cde": "Soccer", "dfb": "Soccer", "efa": "Soccer", "efl": "Soccer",
    "ssc": "Soccer", "lcs": "Soccer", "lib": "Soccer", "sud": "Soccer",
    "caf": "Soccer", "afc": "Soccer", "uef": "Soccer", "cof": "Soccer",
    "con": "Soccer", "ofc": "Soccer", "fif": "Soccer", "acn": "Soccer",
    "mar1": "Soccer",
    # Basketball
    "nba": "Basketball", "wnba": "Basketball", "ncaab": "Basketball",
    "cbb": "Basketball", "cwbb": "Basketball", "bkcl": "Basketball",
    "bkligend": "Basketball", "bknbl": "Basketball", "bkseriea": "Basketball",
    "bkcba": "Basketball", "bkfr1": "Basketball", "bkarg": "Basketball",
    "bkkbl": "Basketball",
    # Football
    "nfl": "Football", "cfb": "Football",
    # Baseball
    "mlb": "Baseball", "kbo": "Baseball",
    # Hockey
    "nhl": "Hockey", "shl": "Hockey", "cehl": "Hockey", "dehl": "Hockey",
    "snhl": "Hockey", "khl": "Hockey", "ahl": "Hockey",
    # Rugby
    "ruprem": "Rugby", "rutopft": "Rugby", "rusixnat": "Rugby",
    "rueuchamp": "Rugby", "ruurc": "Rugby", "rusrp": "Rugby", "ruchamp": "Rugby",
    # MMA
    "mma": "MMA", "zuffa": "MMA",
}

KALSHI_API = "https://api.elections.kalshi.com/trade-api/v2"
GAMMA_API = "https://gamma-api.polymarket.com"

# =========================
# UTILITY FUNCTIONS
# =========================

def normalize_title(s: str) -> str:
    """Normalize text for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_number_from_text(text: str) -> float:
    """Extract a decimal number from text (e.g., '1.5', '3.5', '2', etc.)"""
    match = re.search(r'\b(\d+\.?\d*)\b', text)
    if match:
        return float(match.group(1))
    return None

def parse_outcome_prices(raw):
    """Parse Polymarket outcome prices"""
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    """Parse Polymarket token IDs"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def parse_outcomes(raw):
    """Parse Polymarket outcomes array"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# =========================
# EVENT TYPE DETECTION
# =========================

def extract_game_name(title: str, platform: str) -> str:
    """Extract game name from event title"""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def infer_kalshi_type(title: str) -> str:
    """Infer Kalshi event type from title"""
    t = title.lower()
    if "both teams to score" in t or "btts" in t:
        return "btts"
    if "total" in t or ("over" in t and "goal" in t):
        return "totals"
    if "spread" in t or "cover" in t or "wins by" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title: str) -> str:
    """Infer Polymarket event type from title"""
    return "more_markets" if "More Markets" in title else "moneyline"

def is_soccer_sport(sport: str) -> bool:
    """Check if sport uses soccer-style matching"""
    return sport.lower() == "soccer"

# =========================
# POLYMARKET DATA FETCHING
# =========================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets, grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    raw_outcomes = market.get("outcomes", [])
                    parsed_outcomes = parse_outcomes(raw_outcomes)
                    
                    if not question or not parsed_prices:
                        continue
                    
                    # EXCLUDE SPREADS from Polymarket
                    if "spread" in question.lower() and "price spread" not in question.lower():
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": parsed_outcomes
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

# =========================
# KALSHI DATA FETCHING
# =========================

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets, grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        # EXCLUDE SPREAD EVENTS entirely
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            event_type = infer_kalshi_type(event_title)
            
            # Skip spread events
            if event_type == "spreads":
                continue
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "last_price_yes": market.get("last_price", 0) / 100.0 if market.get("last_price") else None,
                    "yes_bid": market.get("yes_bid", 0) / 100.0 if market.get("yes_bid") else None,
                    "no_bid": market.get("no_bid", 0) / 100.0 if market.get("no_bid") else None,
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport

# =========================
# EVENT MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=70):
    """
    Match events between Polymarket and Kalshi.
    CRITICAL: One Poly "More Markets" can match MULTIPLE Kalshi events (Totals, BTTS)
    """
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            
            # TYPE COMPATIBILITY
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            # For "more_markets", allow matching with totals and btts (spreads already excluded)
            if poly_type == "more_markets" and kal_type not in ["totals", "btts"]:
                continue
            
            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type
                })
    
    matches.sort(key=lambda x: x["score"], reverse=True)
    
    # CRITICAL: For moneylines, use 1-to-1 matching
    # For "more_markets", use 1-to-MANY matching
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        # Kalshi events can only be matched once
        if kalshi_id in used_kalshi:
            continue
        
        # Moneyline poly events can only match once
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        # "More markets" poly events CAN match multiple Kalshi events
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING - SOCCER
# =========================

def extract_outcome_from_poly_question(question: str) -> dict:
    """
    Extract outcome from Polymarket question with NUMBER for totals.
    CRITICAL FIX: Now extracts the actual number (1.5, 2.5, 3.5, etc.)
    """
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return {'type': 'draw', 'normalized': 'draw tie'}
    
    # CRITICAL: Check for O/U with NUMBER
    if "o/u" in question_lower or ("over" in question_lower or "under" in question_lower):
        number = extract_number_from_text(question)
        if number is not None:
            return {'type': 'totals', 'number': number, 'normalized': f'total {number}'}
    
    # Check for BTTS
    if "both teams" in question_lower and "score" in question_lower:
        return {'type': 'btts', 'normalized': 'both teams to score'}
    
    # Extract team name from "Will [TEAM] win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return {'type': 'team', 'team': team, 'normalized': normalize_title(team)}
    
    return {'type': 'other', 'normalized': normalize_title(question)}

def normalize_kalshi_subtitle(subtitle: str) -> dict:
    """
    Normalize Kalshi subtitle with NUMBER for totals.
    CRITICAL FIX: Now extracts the actual number
    """
    normalized = normalize_title(subtitle)
    
    if normalized == "tie":
        return {'type': 'tie', 'normalized': 'draw tie'}
    
    # CRITICAL: Extract number for totals
    if "over" in normalized or "under" in normalized or "goals" in normalized or "total" in normalized:
        number = extract_number_from_text(subtitle)
        if number is not None:
            return {'type': 'totals', 'number': number, 'normalized': f'total {number}'}
    
    # BTTS
    if "both teams" in normalized and "score" in normalized:
        return {'type': 'btts', 'normalized': 'both teams to score'}
    
    return {'type': 'other', 'normalized': normalized}

def match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60):
    """
    Match markets for Soccer.
    CRITICAL FIX: Now matches totals by EXACT number (1.5 matches 1.5, 2.5 matches 2.5, etc.)
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market["question"]
        poly_outcome_info = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = kalshi_market["title"]
            kalshi_info = normalize_kalshi_subtitle(kalshi_subtitle)
            
            # CRITICAL: For totals, numbers must match exactly
            if poly_outcome_info.get('type') == 'totals' and kalshi_info.get('type') == 'totals':
                if poly_outcome_info.get('number') == kalshi_info.get('number'):
                    score = 100  # Perfect match
                else:
                    continue  # Numbers don't match, skip
            else:
                # For non-totals, use fuzzy matching
                score = fuzz.token_set_ratio(
                    poly_outcome_info.get('normalized', ''),
                    kalshi_info.get('normalized', '')
                )
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    # Deduplicate: each Kalshi market matches at most one Poly market
    kalshi_to_best = {}
    for m in matched:
        kalshi_id = m["kalshi_market"]["market_id"]
        if kalshi_id not in kalshi_to_best or m["score"] > kalshi_to_best[kalshi_id]["score"]:
            kalshi_to_best[kalshi_id] = m
    
    return list(kalshi_to_best.values())

# =========================
# MARKET MATCHING - OTHER SPORTS
# =========================

def match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70):
    """Match markets for Basketball/NFL/Hockey using outcome-based matching"""
    matched = []
    
    for poly_market in poly_markets:
        outcomes = poly_market.get("outcomes", [])
        outcome_prices = poly_market.get("outcome_prices", [])
        
        if not outcomes:
            continue
        
        for i, outcome in enumerate(outcomes):
            outcome_normalized = normalize_title(outcome)
            outcome_price = outcome_prices[i] if i < len(outcome_prices) else None
            
            best_match = None
            best_score = 0
            
            for kalshi_market in kalshi_markets:
                kalshi_subtitle = normalize_title(kalshi_market["title"])
                score = fuzz.token_set_ratio(outcome_normalized, kalshi_subtitle)
                
                if score > best_score:
                    best_score = score
                    best_match = kalshi_market
            
            if best_match and best_score >= threshold:
                matched.append({
                    "poly_market": poly_market,
                    "kalshi_market": best_match,
                    "score": best_score,
                    "poly_outcome": outcome,
                    "poly_outcome_price": outcome_price,
                    "poly_outcome_index": i
                })
    
    return matched

# =========================
# MARKET MATCHING ROUTER
# =========================

def match_markets_within_event(poly_markets, kalshi_markets, sport, threshold=70):
    """Route to appropriate matching function based on sport"""
    if is_soccer_sport(sport):
        return match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60)
    else:
        return match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70)

# =========================
# MAIN EXECUTION
# =========================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []
debug_no_markets = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    event_matches = match_events(poly_events, kalshi_events, threshold=70)
    
    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'poly_slug': poly_event.get("slug", ""),
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"]
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            sport,
            threshold=70
        )
        
        if not market_matches:
            debug_no_markets.append({
                'sport': sport,
                'poly_event': poly_event["title"],
                'poly_slug': poly_event.get("slug", ""),
                'kalshi_event': kalshi_event["title"],
                'poly_markets': len(poly_event["markets"]),
                'kalshi_markets': len(kalshi_event["markets"]),
                'sample_poly_question': poly_event["markets"][0]["question"] if poly_event["markets"] else "N/A",
                'sample_poly_outcomes': str(poly_event["markets"][0].get("outcomes", [])) if poly_event["markets"] else "N/A",
                'sample_kalshi_title': kalshi_event["markets"][0]["title"] if kalshi_event["markets"] else "N/A"
            })
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            if not is_soccer_sport(sport) and "poly_outcome_price" in market_match:
                poly_yes_price = market_match["poly_outcome_price"]
                poly_no_price = 1 - poly_yes_price if poly_yes_price else None
                poly_display_question = f"{poly_market['question']} - {market_match['poly_outcome']}"
            else:
                poly_prices = poly_market.get("outcome_prices", [None, None])
                poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
                poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
                poly_display_question = poly_market["question"]
            
            kalshi_yes_price = kalshi_market.get("yes_bid")
            kalshi_no_price = kalshi_market.get("no_bid")
            
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_price is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_price)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "poly_slug": poly_event.get("slug", ""),
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_question": poly_display_question,
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_price": kalshi_yes_price,
                "kalshi_no_price": kalshi_no_price,
                "price_spread": price_spread,
                "market_match_score": market_match["score"],
                "kalshi_event_id": kalshi_event.get("event_id"),
                "poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)
df_debug_no_markets = pd.DataFrame(debug_no_markets)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
if len(df_event_summary) > 0:
    print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
if len(df_event_summary) > 0:
    print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:")
print(f"Total events with 0 markets matched: {len(df_debug_no_markets)}")
if len(df_debug_no_markets) > 0:
    print(f"\nDistribution by sport:")
    print(df_debug_no_markets['sport'].value_counts())
    print(f"\nFirst 10 problematic events:")
    debug_cols = ['sport', 'poly_event', 'poly_slug', 'kalshi_event', 'sample_poly_question', 'sample_kalshi_title']
    print(df_debug_no_markets[debug_cols].head(10).to_string(index=False, max_colwidth=50))

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
if len(df_matched_markets) > 0:
    markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
    print(f"Events with 0 markets: {len(df_event_summary) - len(markets_per_event)}")
    print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
    print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
    print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")

print("\n✅ FINAL DATASETS READY:")
print("  - df_matched_markets: All matched markets with prices")
print("  - df_event_summary: Summary of matched events")
print("  - df_debug_no_markets: Events that failed market matching")

print("\n🚫 EXCLUDED:")
print("  - Cricket, Tennis, Esports (too complex)")
print("  - ALL Spread events from both platforms")

print("\n✅ FEATURES:")
print("  - One Poly 'More Markets' → Multiple Kalshi events (Totals, BTTS)")
print("  - Exact number matching for totals (1.5→1.5, 2.5→2.5, 3.5→3.5, 4.5→4.5)")
print("  - Moneyline: 1-to-1 matching")

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 393

Events by sport:
sport
Soccer        218
Basketball    130
Hockey         45
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 854
Average markets per event: 2.17

⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:
Total events with 0 markets matched: 14

Distribution by sport:
sport
Soccer        8
Basketball    4
Hockey        2
Name: count, dtype: int64

First 10 problematic events:
     sport                                         poly_event                             poly_slug                         kalshi_event                               sample_poly_question   sample_kalshi_title
    Soccer TSG 1899 Hoffenheim vs. 1. FC Union Berlin - Mo...   bun-hof-uni-2026-01-31-more-markets   Union Berlin at Hoffenheim: Totals TSG 1899 Hoffenheim vs. 1. FC Union Berlin: O/U... Over 3.5 goals

In [46]:
df_matched_markets

,sport,poly_event_title,poly_slug,kalshi_event_title,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_price,kalshi_no_price,price_spread,market_match_score,kalshi_event_id,poly_market_id,kalshi_market_id
0,Soccer,Leeds United FC vs. Arsenal FC,epl-lee-ars-2026-01-31,Leeds United vs Arsenal,Will Leeds United FC win on 2026-01-31?,Leeds United,0.145,0.855,0.14,0.84,0.005,100,KXEPLGAME-26JAN31LEEARS,1209694,KXEPLGAME-26JAN31LEEARS-LEE
1,Soccer,Leeds United FC vs. Arsenal FC,epl-lee-ars-2026-01-31,Leeds United vs Arsenal,Will Arsenal FC win on 2026-01-31?,Arsenal,0.630,0.370,0.62,0.36,0.010,100,KXEPLGAME-26JAN31LEEARS,1209696,KXEPLGAME-26JAN31LEEARS-ARS
2,Soccer,Leeds United FC vs. Arsenal FC,epl-lee-ars-2026-01-31,Leeds United vs Arsenal,Will Leeds United FC vs. Arsenal FC end in a d...,Tie,0.225,0.775,0.21,0.77,0.015,100,KXEPLGAME-26JAN31LEEARS,1209695,KXEPLGAME-26JAN31LEEARS-TIE
3,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,epl-wol-bou-2026-01-31,Wolverhampton vs Bournemouth,Will Wolverhampton Wanderers FC win on 2026-01...,Wolverhampton,0.300,0.700,0.29,0.69,0.010,100,KXEPLGAME-26JAN31WOLBOU,1209697,KXEPLGAME-26JAN31WOLBOU-WOL
4,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,epl-wol-bou-2026-01-31,Wolverhampton vs Bournemouth,Will AFC Bournemouth win on 2026-01-31?,Bournemouth,0.425,0.575,0.42,0.56,0.005,100,KXEPLGAME-26JAN31WOLBOU,1209699,KXEPLGAME-26JAN31WOLBOU-BOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,Hockey,KHL: Sibir Novosibirsk vs. Traktor,khl-sib-tra-2026-01-31,Traktor Chelyabinsk at Sibir Novosibirsk,KHL: Sibir Novosibirsk vs. Traktor - Traktor,Traktor Chelyabinsk,0.705,0.295,0.38,0.01,0.325,100,KXKHLGAME-26JAN31TRASIB,1266320,KXKHLGAME-26JAN31TRASIB-TRA
850,Hockey,KHL: Amur Khabarovsk vs. Barys Astana,khl-amu-bar-2026-01-31,HC Barys at Amur Khabarovsk,KHL: Amur Khabarovsk vs. Barys Astana - Amur K...,Amur Khabarovsk,0.350,0.650,0.01,0.26,0.340,100,KXKHLGAME-26JAN31HCBAMU,1266330,KXKHLGAME-26JAN31HCBAMU-AMU
851,Hockey,KHL: Amur Khabarovsk vs. Barys Astana,khl-amu-bar-2026-01-31,HC Barys at Amur Khabarovsk,KHL: Amur Khabarovsk vs. Barys Astana - Barys ...,HC Barys,0.650,0.350,0.26,0.01,0.390,77,KXKHLGAME-26JAN31HCBAMU,1266330,KXKHLGAME-26JAN31HCBAMU-HCB
852,Hockey,KHL: Avangard Omsk vs. Traktor,khl-ava-tra-2026-01-29,Traktor Chelyabinsk at HK Avangard Omsk,KHL: Avangard Omsk vs. Traktor - Avangard Omsk,HK Avangard Omsk,0.630,0.370,0.57,0.24,0.060,100,KXKHLGAME-26JAN29TRAHKA,1249990,KXKHLGAME-26JAN29TRAHKA-HKA


In [47]:
df_matched_markets.to_csv(f'matched_markets.csv', index=False)

print(f"✓ Saved matched_markets.csv")

✓ Saved matched_markets.csv


### FINAL CODE WORKING WELL -> NO WRONG MATCHES (SOCCER, BASKETBALL, HOCKEY ONLY)

In [ ]:
import requests
import json
from collections import defaultdict
import re
from fuzzywuzzy import fuzz
import pandas as pd

# =========================
# UTILITY FUNCTIONS
# =========================

def normalize_title(s: str) -> str:
    """Normalize text for comparison"""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def extract_number_from_text(text: str) -> float:
    """Extract a decimal number from text (e.g., '1.5', '3.5', '2', etc.)"""
    match = re.search(r'\b(\d+\.?\d*)\b', text)
    if match:
        return float(match.group(1))
    return None

def parse_outcome_prices(raw):
    """Parse Polymarket outcome prices"""
    try:
        if isinstance(raw, list):
            return [float(raw[0]), float(raw[1])]
        elif isinstance(raw, str):
            parsed = json.loads(raw)
            return [float(parsed[0]), float(parsed[1])]
        return None
    except (ValueError, IndexError, TypeError, json.JSONDecodeError):
        return None

def parse_token_ids(raw):
    """Parse Polymarket token IDs"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

def parse_outcomes(raw):
    """Parse Polymarket outcomes array"""
    try:
        if isinstance(raw, list):
            return raw
        elif isinstance(raw, str):
            return json.loads(raw)
        return None
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# =========================
# EVENT TYPE DETECTION
# =========================

def extract_game_name(title: str, platform: str) -> str:
    """Extract game name from event title"""
    if platform == "polymarket":
        if " - More Markets" in title:
            return title.replace(" - More Markets", "").strip()
        return title
    elif platform == "kalshi":
        if ":" in title:
            return title.split(":")[0].strip()
        return title
    return title

def infer_kalshi_type(title: str) -> str:
    """Infer Kalshi event type from title"""
    t = title.lower()
    if "both teams to score" in t or "btts" in t:
        return "btts"
    if "total" in t or ("over" in t and "goal" in t):
        return "totals"
    if "spread" in t or "cover" in t or "wins by" in t:
        return "spreads"
    return "moneyline"

def infer_poly_type(title: str) -> str:
    """Infer Polymarket event type from title"""
    return "more_markets" if "More Markets" in title else "moneyline"

def is_soccer_sport(sport: str) -> bool:
    """Check if sport uses soccer-style matching"""
    return sport.lower() == "soccer"

# =========================
# POLYMARKET DATA FETCHING
# =========================

def fetch_polymarket_events_with_markets():
    """Fetch all Polymarket events with their markets, grouped by sport"""
    sports = requests.get(f"{GAMMA_API}/sports").json()
    events_by_sport = defaultdict(list)
    
    max_limit = 200
    limit = 50
    
    for item in sports:
        series = item.get("series")
        league_code = item.get("sport")
        
        if not series or not league_code:
            continue
        
        try:
            series = int(series)
        except (TypeError, ValueError):
            continue
        
        league_code = league_code.lower()
        kalshi_sport = POLY_LEAGUE_TO_KALSHI_SPORT.get(league_code)
        
        if not kalshi_sport:
            continue
        
        page_id = 0
        while page_id < max_limit:
            resp = requests.get(
                f"{GAMMA_API}/events",
                params={
                    "series_id": series,
                    "active": True,
                    "closed": False,
                    "limit": limit,
                    "offset": page_id * limit
                }
            )
            events = resp.json()
            if not events:
                break
            
            for event in events:
                event_title = event.get("title", "")
                event_slug = event.get("slug", "")
                
                markets = []
                for market in event.get("markets", []):
                    question = market.get("question")
                    raw_prices = market.get("outcomePrices")
                    parsed_prices = parse_outcome_prices(raw_prices)
                    raw_tokenIDs = market.get("clobTokenIds")
                    parsed_tokenIDs = parse_token_ids(raw_tokenIDs)
                    raw_outcomes = market.get("outcomes", [])
                    parsed_outcomes = parse_outcomes(raw_outcomes)
                    volume = market.get("volumeNum", 0)

                    if not question or not parsed_prices:
                        continue
                    
                    # EXCLUDE SPREADS from Polymarket
                    if "spread" in question.lower() and "price spread" not in question.lower():
                        continue
                    
                    markets.append({
                        "question": question,
                        "market_id": market.get("id"),
                        "market_slug": market.get("slug", ""),  # Added: poly market slug
                        "condition_id": market.get("conditionId"),
                        "outcome_prices": parsed_prices,
                        "token_ids": parsed_tokenIDs,
                        "outcomes": parsed_outcomes,
                        "volume_p": volume
                    })
                
                if markets:
                    events_by_sport[kalshi_sport].append({
                        "title": event_title,
                        "slug": event_slug,
                        "markets": markets
                    })
            
            page_id += 1
    
    return events_by_sport

# =========================
# KALSHI DATA FETCHING
# =========================

def fetch_kalshi_events_with_markets():
    """Fetch all Kalshi events with their markets, grouped by sport"""
    response = requests.get(f"{KALSHI_API}/search/filters_by_sport")
    filters_data = response.json()
    all_sports = [s for s in filters_data.get("sport_ordering", []) if s != "All sports"]
    
    events_by_sport = defaultdict(list)
    
    for sport in all_sports:
        response = requests.get(f"{KALSHI_API}/series", params={"category": "Sports", "tags": sport})
        series_data = response.json().get('series', [])
        sport_tickers = [s.get("ticker") for s in series_data if s.get("ticker")]
        
        all_events = []
        for ticker in sport_tickers:
            cursor = None
            limit = 200
            while True:
                params = {
                    "status": "open",
                    "series_ticker": ticker,
                    "with_nested_markets": True,
                    "limit": limit
                }
                if cursor:
                    params["cursor"] = cursor
                
                response = requests.get(f"{KALSHI_API}/events", params=params)
                data_response = response.json()
                events = data_response.get('events', [])
                
                if not events:
                    break
                
                all_events.extend(events)
                cursor = data_response.get('cursor')
                if not cursor:
                    break
        
        # EXCLUDE SPREAD EVENTS entirely
        for event in all_events:
            event_id = event.get("event_ticker", "")
            event_title = event.get("title", "")
            event_type = infer_kalshi_type(event_title)
            
            # Skip spread events
            if event_type == "spreads":
                continue
            
            markets = []
            for market in event.get("markets", []):
                market_id = market.get("ticker", "")
                market_title = market.get("yes_sub_title", "")
                
                # UPDATED: Extract ask/bid prices directly in dollars (no division needed)
                markets.append({
                    "market_id": market_id,
                    "title": market_title,
                    "yes_ask": float(market.get("yes_ask_dollars")),
                    "yes_bid": float(market.get("yes_bid_dollars")),
                    "no_ask": float(market.get("no_ask_dollars")),
                    "no_bid": float(market.get("no_bid_dollars")),
                    "volume": market.get("volume_24h", 0)
                })
            
            if markets:
                events_by_sport[sport].append({
                    "event_id": event_id,
                    "title": event_title,
                    "markets": markets
                })
    
    return events_by_sport

# =========================
# EVENT MATCHING
# =========================

def match_events(poly_events, kalshi_events, threshold=70):
    """
    Match events between Polymarket and Kalshi.
    CRITICAL: One Poly "More Markets" can match MULTIPLE Kalshi events (Totals, BTTS)
    """
    matches = []
    
    for p in poly_events:
        poly_game = normalize_title(extract_game_name(p["title"], "polymarket"))
        poly_type = infer_poly_type(p["title"])
        
        for k in kalshi_events:
            kal_game = normalize_title(extract_game_name(k["title"], "kalshi"))
            kal_type = infer_kalshi_type(k["title"])
            
            # TYPE COMPATIBILITY
            if poly_type == "moneyline" and kal_type != "moneyline":
                continue
            # For "more_markets", allow matching with totals and btts (spreads already excluded)
            if poly_type == "more_markets" and kal_type not in ["totals", "btts"]:
                continue
            
            score = fuzz.token_set_ratio(poly_game, kal_game)
            if score >= threshold:
                matches.append({
                    "poly": p,
                    "kalshi": k,
                    "score": score,
                    "poly_type": poly_type,
                    "kalshi_type": kal_type
                })
    
    matches.sort(key=lambda x: x["score"], reverse=True)
    
    # CRITICAL: For moneylines, use 1-to-1 matching
    # For "more_markets", use 1-to-MANY matching
    used_kalshi = set()
    used_poly_moneyline = set()
    final_matches = []
    
    for match in matches:
        poly_id = id(match["poly"])
        kalshi_id = id(match["kalshi"])
        poly_type = match["poly_type"]
        
        # Kalshi events can only be matched once
        if kalshi_id in used_kalshi:
            continue
        
        # Moneyline poly events can only match once
        if poly_type == "moneyline":
            if poly_id in used_poly_moneyline:
                continue
            used_poly_moneyline.add(poly_id)
        
        # "More markets" poly events CAN match multiple Kalshi events
        
        final_matches.append(match)
        used_kalshi.add(kalshi_id)
    
    return final_matches

# =========================
# MARKET MATCHING - SOCCER
# =========================

def extract_outcome_from_poly_question(question: str) -> dict:
    """
    Extract outcome from Polymarket question with NUMBER for totals.
    CRITICAL FIX: Now extracts the actual number (1.5, 2.5, 3.5, etc.)
    """
    question_lower = question.lower()
    
    # Check for draw/tie
    if "draw" in question_lower or "tie" in question_lower:
        return {'type': 'draw', 'normalized': 'draw tie'}
    
    # CRITICAL: Check for O/U with NUMBER
    if "o/u" in question_lower or ("over" in question_lower or "under" in question_lower):
        number = extract_number_from_text(question)
        if number is not None:
            return {'type': 'totals', 'number': number, 'normalized': f'total {number}'}
    
    # Check for BTTS
    if "both teams" in question_lower and "score" in question_lower:
        return {'type': 'btts', 'normalized': 'both teams to score'}
    
    # Extract team name from "Will [TEAM] win" pattern
    win_pattern = r"will\s+([\w\s]+?)\s+(?:win|fc win|united win)"
    match = re.search(win_pattern, question_lower)
    if match:
        team = match.group(1).strip()
        team = re.sub(r'\s+(fc|united|city|afc|cf)$', '', team).strip()
        return {'type': 'team', 'team': team, 'normalized': normalize_title(team)}
    
    return {'type': 'other', 'normalized': normalize_title(question)}

def normalize_kalshi_subtitle(subtitle: str) -> dict:
    """
    Normalize Kalshi subtitle with NUMBER for totals.
    CRITICAL FIX: Now extracts the actual number
    """
    normalized = normalize_title(subtitle)
    
    if normalized == "tie":
        return {'type': 'tie', 'normalized': 'draw tie'}
    
    # CRITICAL: Extract number for totals
    if "over" in normalized or "under" in normalized or "goals" in normalized or "total" in normalized:
        number = extract_number_from_text(subtitle)
        if number is not None:
            return {'type': 'totals', 'number': number, 'normalized': f'total {number}'}
    
    # BTTS
    if "both teams" in normalized and "score" in normalized:
        return {'type': 'btts', 'normalized': 'both teams to score'}
    
    return {'type': 'other', 'normalized': normalized}

def match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60):
    """
    Match markets for Soccer.
    CRITICAL FIX: Now matches totals by EXACT number (1.5 matches 1.5, 2.5 matches 2.5, etc.)
    """
    matched = []
    
    for poly_market in poly_markets:
        poly_question = poly_market["question"]
        poly_outcome_info = extract_outcome_from_poly_question(poly_question)
        
        best_match = None
        best_score = 0
        
        for kalshi_market in kalshi_markets:
            kalshi_subtitle = kalshi_market["title"]
            kalshi_info = normalize_kalshi_subtitle(kalshi_subtitle)
            
            # CRITICAL: For totals, numbers must match exactly
            if poly_outcome_info.get('type') == 'totals' and kalshi_info.get('type') == 'totals':
                if poly_outcome_info.get('number') == kalshi_info.get('number'):
                    score = 100  # Perfect match
                else:
                    continue  # Numbers don't match, skip
            else:
                # For non-totals, use fuzzy matching
                score = fuzz.token_set_ratio(
                    poly_outcome_info.get('normalized', ''),
                    kalshi_info.get('normalized', '')
                )
            
            if score > best_score:
                best_score = score
                best_match = kalshi_market
        
        if best_match and best_score >= threshold:
            matched.append({
                "poly_market": poly_market,
                "kalshi_market": best_match,
                "score": best_score
            })
    
    # Deduplicate: each Kalshi market matches at most one Poly market
    kalshi_to_best = {}
    for m in matched:
        kalshi_id = m["kalshi_market"]["market_id"]
        if kalshi_id not in kalshi_to_best or m["score"] > kalshi_to_best[kalshi_id]["score"]:
            kalshi_to_best[kalshi_id] = m
    
    return list(kalshi_to_best.values())

# =========================
# MARKET MATCHING - OTHER SPORTS
# =========================

def match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70):
    """Match markets for Basketball/NFL/Hockey using outcome-based matching"""
    matched = []
    
    for poly_market in poly_markets:
        outcomes = poly_market.get("outcomes", [])
        outcome_prices = poly_market.get("outcome_prices", [])
        
        if not outcomes:
            continue
        
        for i, outcome in enumerate(outcomes):
            outcome_normalized = normalize_title(outcome)
            outcome_price = outcome_prices[i] if i < len(outcome_prices) else None
            
            best_match = None
            best_score = 0
            
            for kalshi_market in kalshi_markets:
                kalshi_subtitle = normalize_title(kalshi_market["title"])
                score = fuzz.token_set_ratio(outcome_normalized, kalshi_subtitle)
                
                if score > best_score:
                    best_score = score
                    best_match = kalshi_market
            
            if best_match and best_score >= threshold:
                matched.append({
                    "poly_market": poly_market,
                    "kalshi_market": best_match,
                    "score": best_score,
                    "poly_outcome": outcome,
                    "poly_outcome_price": outcome_price,
                    "poly_outcome_index": i
                })
    
    return matched

# =========================
# MARKET MATCHING ROUTER
# =========================

def match_markets_within_event(poly_markets, kalshi_markets, sport, threshold=70):
    """Route to appropriate matching function based on sport"""
    if is_soccer_sport(sport):
        return match_markets_within_event_soccer(poly_markets, kalshi_markets, threshold=60)
    else:
        return match_markets_within_event_outcomes(poly_markets, kalshi_markets, threshold=70)

# =========================
# MAIN EXECUTION
# =========================

print("Fetching Polymarket events with markets...")
poly_events_by_sport = fetch_polymarket_events_with_markets()

print("Fetching Kalshi events with markets...")
kalshi_events_by_sport = fetch_kalshi_events_with_markets()

print("\nMatching events and markets...")

all_matched_data = []
event_match_summary = []
debug_no_markets = []

for sport in poly_events_by_sport.keys():
    if sport not in kalshi_events_by_sport:
        continue
    
    poly_events = poly_events_by_sport[sport]
    kalshi_events = kalshi_events_by_sport[sport]
    
    #event_matches = match_events(poly_events, kalshi_events, threshold=70)
    event_threshold = 90 if sport == "Soccer" else 70
    event_matches = match_events(poly_events, kalshi_events, threshold=event_threshold)

    for event_match in event_matches:
        poly_event = event_match["poly"]
        kalshi_event = event_match["kalshi"]
        
        event_match_summary.append({
            'sport': sport,
            'poly_event_title': poly_event["title"],
            'poly_event_slug': poly_event.get("slug", ""),
            'kalshi_event_title': kalshi_event["title"],
            'poly_markets_count': len(poly_event["markets"]),
            'kalshi_markets_count': len(kalshi_event["markets"]),
            'event_match_score': event_match["score"]
        })
        
        market_matches = match_markets_within_event(
            poly_event["markets"],
            kalshi_event["markets"],
            sport,
            threshold=70
        )
        
        if not market_matches:
            debug_no_markets.append({
                'sport': sport,
                'poly_event': poly_event["title"],
                'poly_event_slug': poly_event.get("slug", ""),
                'kalshi_event': kalshi_event["title"],
                'poly_markets': len(poly_event["markets"]),
                'kalshi_markets': len(kalshi_event["markets"]),
                'sample_poly_question': poly_event["markets"][0]["question"] if poly_event["markets"] else "N/A",
                'sample_poly_outcomes': str(poly_event["markets"][0].get("outcomes", [])) if poly_event["markets"] else "N/A",
                'sample_kalshi_title': kalshi_event["markets"][0]["title"] if kalshi_event["markets"] else "N/A"
            })
        
        for market_match in market_matches:
            poly_market = market_match["poly_market"]
            kalshi_market = market_match["kalshi_market"]
            
            if not is_soccer_sport(sport) and "poly_outcome_price" in market_match:
                poly_yes_price = market_match["poly_outcome_price"]
                poly_no_price = 1 - poly_yes_price if poly_yes_price else None
                poly_display_question = f"{poly_market['question']} - {market_match['poly_outcome']}"
            else:
                poly_prices = poly_market.get("outcome_prices", [None, None])
                poly_yes_price = poly_prices[0] if len(poly_prices) > 0 else None
                poly_no_price = poly_prices[1] if len(poly_prices) > 1 else None
                poly_display_question = poly_market["question"]
                poly_volume = poly_market["volume_p"] 
            
            # UPDATED: Use direct Kalshi dollar prices (no calculation needed)
            kalshi_yes_ask = kalshi_market.get("yes_ask")
            kalshi_yes_bid = kalshi_market.get("yes_bid")
            kalshi_no_ask = kalshi_market.get("no_ask")
            kalshi_no_bid = kalshi_market.get("no_bid")
            kalshi_volume = kalshi_market.get("volume")
            
            # Calculate price spread using yes_ask prices
            price_spread = None
            if poly_yes_price is not None and kalshi_yes_bid is not None:
                price_spread = abs(poly_yes_price - kalshi_yes_ask)
            
            all_matched_data.append({
                "sport": sport,
                "poly_event_title": poly_event["title"],
                "kalshi_event_title": kalshi_event["title"],
                "poly_market_slug": poly_market.get("market_slug", ""),
                "poly_market_question": poly_display_question,
                "kalshi_market_title": kalshi_market["title"],
                "poly_yes_price": poly_yes_price,
                "poly_no_price": poly_no_price,
                "kalshi_yes_ask": kalshi_yes_ask,  # UPDATED
                "kalshi_yes_bid": kalshi_yes_bid,  # UPDATED
                "kalshi_no_ask": kalshi_no_ask,    # UPDATED
                "kalshi_no_bid": kalshi_no_bid,    # UPDATED
                "price_spread": price_spread,
                "poly_volume24h": poly_volume,          
                "kalshi_volume24h": kalshi_volume,
                #"market_match_score": market_match["score"],
                #"kalshi_event_id": kalshi_event.get("event_id"),
                #"poly_market_id": poly_market.get("market_id"),
                "kalshi_market_id": kalshi_market.get("market_id"),
            })

df_matched_markets = pd.DataFrame(all_matched_data)
df_event_summary = pd.DataFrame(event_match_summary)
df_debug_no_markets = pd.DataFrame(debug_no_markets)

print(f"\n{'='*80}")
print(f"MATCHING COMPLETE")
print(f"{'='*80}")

print(f"\n📊 EVENT-LEVEL SUMMARY:")
print(f"Total matched events: {len(df_event_summary)}")
print(f"\nEvents by sport:")
if len(df_event_summary) > 0:
    print(df_event_summary['sport'].value_counts())

print(f"\n📊 MARKET-LEVEL SUMMARY:")
print(f"Total matched markets: {len(df_matched_markets)}")
if len(df_event_summary) > 0:
    print(f"Average markets per event: {len(df_matched_markets) / len(df_event_summary):.2f}")

print(f"\n⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:")
print(f"Total events with 0 markets matched: {len(df_debug_no_markets)}")
if len(df_debug_no_markets) > 0:
    print(f"\nDistribution by sport:")
    print(df_debug_no_markets['sport'].value_counts())
    print(f"\nFirst 10 problematic events:")
    debug_cols = ['sport', 'poly_event', 'poly_event_slug', 'kalshi_event', 'sample_poly_question', 'sample_kalshi_title']
    print(df_debug_no_markets[debug_cols].head(10).to_string(index=False, max_colwidth=50))

print(f"\n📈 MARKETS PER EVENT BREAKDOWN:")
if len(df_matched_markets) > 0:
    markets_per_event = df_matched_markets.groupby(['poly_event_title', 'kalshi_event_title']).size()
    print(f"Events with 0 markets: {len(df_event_summary) - len(markets_per_event)}")
    print(f"Events with 1 market: {(markets_per_event == 1).sum()}")
    print(f"Events with 2 markets: {(markets_per_event == 2).sum()}")
    print(f"Events with 3+ markets: {(markets_per_event >= 3).sum()}")

print("\n✅ FINAL DATASETS READY:")
print("  - df_matched_markets: All matched markets with prices")
print("  - df_event_summary: Summary of matched events")
print("  - df_debug_no_markets: Events that failed market matching")

Fetching Polymarket events with markets...
Fetching Kalshi events with markets...

Matching events and markets...

MATCHING COMPLETE

📊 EVENT-LEVEL SUMMARY:
Total matched events: 545

Events by sport:
sport
Basketball    317
Soccer        182
Hockey         45
Football        1
Name: count, dtype: int64

📊 MARKET-LEVEL SUMMARY:
Total matched markets: 1169
Average markets per event: 2.14

⚠️ DEBUG - EVENTS WITH NO MARKET MATCHES:
Total events with 0 markets matched: 5

Distribution by sport:
sport
Soccer    5
Name: count, dtype: int64

First 10 problematic events:
 sport                                         poly_event                     poly_event_slug                       kalshi_event                               sample_poly_question   sample_kalshi_title
Soccer TSG 1899 Hoffenheim vs. 1. FC Union Berlin - Mo... bun-hof-uni-2026-01-31-more-markets Union Berlin at Hoffenheim: Totals TSG 1899 Hoffenheim vs. 1. FC Union Berlin: O/U... Over 3.5 goals scored
Soccer Eintracht Frankfurt

In [2]:
df_matched_markets

,sport,poly_event_title,kalshi_event_title,poly_market_slug,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_ask,kalshi_yes_bid,kalshi_no_ask,kalshi_no_bid,price_spread,poly_volume24h,kalshi_volume24h,kalshi_market_id
0,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,epl-lee-ars-2026-01-31-lee,Will Leeds United FC win on 2026-01-31?,Leeds United,0.145,0.855,0.16,0.15,0.85,0.84,0.015,6974.360761,7924,KXEPLGAME-26JAN31LEEARS-LEE
1,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,epl-lee-ars-2026-01-31-ars,Will Arsenal FC win on 2026-01-31?,Arsenal,0.635,0.365,0.64,0.62,0.38,0.36,0.005,30344.899439,75519,KXEPLGAME-26JAN31LEEARS-ARS
2,Soccer,Leeds United FC vs. Arsenal FC,Leeds United vs Arsenal,epl-lee-ars-2026-01-31-draw,Will Leeds United FC vs. Arsenal FC end in a d...,Tie,0.225,0.775,0.23,0.21,0.79,0.77,0.005,771.905778,1429,KXEPLGAME-26JAN31LEEARS-TIE
3,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,epl-wol-bou-2026-01-31-wol,Will Wolverhampton Wanderers FC win on 2026-01...,Wolverhampton,0.305,0.695,0.32,0.31,0.69,0.68,0.015,2191.085575,712,KXEPLGAME-26JAN31WOLBOU-WOL
4,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,epl-wol-bou-2026-01-31-bou,Will AFC Bournemouth win on 2026-01-31?,Bournemouth,0.425,0.575,0.44,0.43,0.57,0.56,0.015,23728.591353,6262,KXEPLGAME-26JAN31WOLBOU-BOU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164,Hockey,KHL: Sibir Novosibirsk vs. Traktor,Traktor Chelyabinsk at Sibir Novosibirsk,khl-sib-tra-2026-01-31,KHL: Sibir Novosibirsk vs. Traktor - Traktor,Traktor Chelyabinsk,0.690,0.310,0.90,0.40,0.60,0.10,0.210,0.000000,0,KXKHLGAME-26JAN31TRASIB-TRA
1165,Hockey,KHL: Amur Khabarovsk vs. Barys Astana,HC Barys at Amur Khabarovsk,khl-amu-bar-2026-01-31,KHL: Amur Khabarovsk vs. Barys Astana - Amur K...,Amur Khabarovsk,0.590,0.410,0.68,0.53,0.47,0.32,0.090,0.000000,0,KXKHLGAME-26JAN31HCBAMU-AMU
1166,Hockey,KHL: Amur Khabarovsk vs. Barys Astana,HC Barys at Amur Khabarovsk,khl-amu-bar-2026-01-31,KHL: Amur Khabarovsk vs. Barys Astana - Barys ...,HC Barys,0.410,0.590,0.50,0.37,0.63,0.50,0.090,0.000000,0,KXKHLGAME-26JAN31HCBAMU-HCB
1167,Hockey,SHL: Skellefteaa vs. Timraa,Timra IK at Skellefteå,shl-ske-tim-2026-01-29,SHL: Skellefteaa vs. Timraa - Skellefteaa,Skellefteå,0.725,0.275,0.76,0.67,0.33,0.24,0.035,0.000000,0,KXSHLGAME-26JAN29TIMSKE-SKE


In [74]:
df_matched_markets.to_csv(f'matched_markets.csv', index=False)

print(f"✓ Saved matched_markets.csv")

✓ Saved matched_markets.csv


In [3]:
# =========================
# ARBITRAGE
# =========================

def detect_true_arb(row):
    # YES Poly + NO Kalshi
    if (
        pd.notna(row["poly_yes_price"]) and
        pd.notna(row["kalshi_no_ask"]) and
        row["poly_yes_price"] + row["kalshi_no_ask"] < 1
    ):
        return "YES_POLY__NO_KALSHI"

    # YES Kalshi + NO Poly
    if (
        pd.notna(row["kalshi_yes_ask"]) and
        pd.notna(row["poly_no_price"]) and
        row["kalshi_yes_ask"] + row["poly_no_price"] < 1
    ):
        return "YES_KALSHI__NO_POLY"

    return None

def true_arb_edge(row):
    if row["true_arb_direction"] == "YES_POLY__NO_KALSHI":
        total_cost = row["poly_yes_price"] + row["kalshi_no_ask"]
    elif row["true_arb_direction"] == "YES_KALSHI__NO_POLY":
        total_cost = row["kalshi_yes_ask"] + row["poly_no_price"]
    else:
        return None

    return 1 - total_cost

def true_arb_executable(row):
    if row["true_arb_direction"] == "YES_POLY__NO_KALSHI":
        return pd.notna(row["poly_yes_price"]) and pd.notna(row["kalshi_no_ask"])
    if row["true_arb_direction"] == "YES_KALSHI__NO_POLY":
        return pd.notna(row["kalshi_yes_ask"]) and pd.notna(row["poly_no_price"])
    return False

# =========================
# ARBITRAGE EXTRACTION (>1%)
# =========================
"""
arbitrage = df_matched_markets[
    df_matched_markets["price_spread"].notna() &
    (df_matched_markets["price_spread"] > 0.01)
].copy()
"""

VOLUME_THRESHOLD = 100  

# Keep only markets where BOTH Poly and Kalshi volume are above threshold
arbitrage = df_matched_markets[
    (df_matched_markets["poly_volume24h"] >= VOLUME_THRESHOLD) &
    (df_matched_markets["kalshi_volume24h"] >= VOLUME_THRESHOLD)
].copy()


arbitrage["true_arb_direction"] = arbitrage.apply(detect_true_arb, axis=1)
arbitrage["true_arb_edge"] = arbitrage.apply(true_arb_edge, axis=1)
arbitrage["true_arb_executable"] = arbitrage.apply(true_arb_executable, axis=1)

true_arbitrage = arbitrage[
    arbitrage["true_arb_direction"].notna() &
    (arbitrage["true_arb_edge"] > 0)
].sort_values("true_arb_edge", ascending=False)

print("\n🔥 TRUE ARBITRAGE OPPORTUNITIES")
print(f"Count: {len(true_arbitrage)}")

cols = [
    "sport",
    "poly_market_question",
    "true_arb_direction",
    "poly_yes_price",
    "poly_no_price",
    "kalshi_yes_ask",
    "kalshi_no_ask",
    "true_arb_edge",
    "true_arb_executable"
]

print(true_arbitrage[cols].head(10).to_string(index=False))



🔥 TRUE ARBITRAGE OPPORTUNITIES
Count: 31
 sport                                        poly_market_question  true_arb_direction  poly_yes_price  poly_no_price  kalshi_yes_ask  kalshi_no_ask  true_arb_edge  true_arb_executable
Soccer                      Will FC Midtjylland win on 2026-01-29? YES_POLY__NO_KALSHI           0.515          0.485            0.54           0.47          0.015                 True
Soccer                         Will Real Oviedo win on 2026-01-31? YES_KALSHI__NO_POLY           0.355          0.645            0.34           0.67          0.015                 True
Soccer Will Aston Villa FC vs. FC Red Bull Salzburg end in a draw? YES_KALSHI__NO_POLY           0.180          0.820            0.17           0.84          0.010                 True
Soccer                 Will Al Nassr Saudi Club win on 2026-01-30? YES_POLY__NO_KALSHI           0.780          0.220            0.80           0.21          0.010                 True
Soccer                          W

In [4]:
soccer_true_arb = true_arbitrage[true_arbitrage["sport"] == "Soccer"]
soccer_true_arb

,sport,poly_event_title,kalshi_event_title,poly_market_slug,poly_market_question,kalshi_market_title,poly_yes_price,poly_no_price,kalshi_yes_ask,kalshi_yes_bid,kalshi_no_ask,kalshi_no_bid,price_spread,poly_volume24h,kalshi_volume24h,kalshi_market_id,true_arb_direction,true_arb_edge,true_arb_executable
213,Soccer,FC Midtjylland vs. GNK Dinamo Zagreb,Midtjylland vs Dinamo Zagreb,uel-mid1-din6-2026-01-29-mid1,Will FC Midtjylland win on 2026-01-29?,Midtjylland,0.515,0.485,0.54,0.53,0.47,0.46,0.025,17917.196252,9424,KXUELGAME-26JAN29FCMDIN-FCM,YES_POLY__NO_KALSHI,0.015,True
84,Soccer,Real Oviedo vs. Girona FC,Oviedo vs Girona,lal-ovi-gir-2026-01-31-ovi,Will Real Oviedo win on 2026-01-31?,Oviedo,0.355,0.645,0.34,0.33,0.67,0.66,0.015,5855.001523,20464,KXLALIGAGAME-26JAN31OVIGIR-OVI,YES_KALSHI__NO_POLY,0.015,True
211,Soccer,Aston Villa FC vs. FC Red Bull Salzburg,Aston Villa vs Salzburg,uel-ast4-rbs1-2026-01-29-draw,Will Aston Villa FC vs. FC Red Bull Salzburg e...,Tie,0.180,0.820,0.17,0.16,0.84,0.83,0.010,2763.809667,2357,KXUELGAME-26JAN29AVLRBS-TIE,YES_KALSHI__NO_POLY,0.010,True
260,Soccer,Al Kholood Saudi Club vs. Al Nassr Saudi Club,Al-Kholood vs Al Nassr,spl-kho-nsr-2026-01-30-nsr,Will Al Nassr Saudi Club win on 2026-01-30?,Al Nassr,0.780,0.220,0.80,0.79,0.21,0.20,0.020,445.446227,606,KXSAUDIPLGAME-26JAN30AKHALN-ALN,YES_POLY__NO_KALSHI,0.010,True
148,Soccer,SSC Napoli vs. ACF Fiorentina,Napoli vs Fiorentina,sea-nap-fio-2026-01-31-nap,Will SSC Napoli win on 2026-01-31?,Napoli,0.550,0.450,0.57,0.56,0.44,0.43,0.020,3872.713627,1069,KXSERIEAGAME-26JAN31NAPFIO-NAP,YES_POLY__NO_KALSHI,0.010,True
113,Soccer,Eintracht Frankfurt vs. Bayer 04 Leverkusen,Frankfurt vs Leverkusen,bun-ein-b04-2026-01-31-b04,Will Bayer 04 Leverkusen win on 2026-01-31?,Leverkusen,0.435,0.565,0.43,0.42,0.58,0.57,0.005,3162.631020,576,KXBUNDESLIGAGAME-26JAN31SGELEV-LEV,YES_KALSHI__NO_POLY,0.005,True
3,Soccer,Wolverhampton Wanderers FC vs. AFC Bournemouth,Wolverhampton vs Bournemouth,epl-wol-bou-2026-01-31-wol,Will Wolverhampton Wanderers FC win on 2026-01...,Wolverhampton,0.305,0.695,0.32,0.31,0.69,0.68,0.015,2191.085575,712,KXEPLGAME-26JAN31WOLBOU-WOL,YES_POLY__NO_KALSHI,0.005,True
210,Soccer,Aston Villa FC vs. FC Red Bull Salzburg,Aston Villa vs Salzburg,uel-ast4-rbs1-2026-01-29-ast4,Will Aston Villa FC win on 2026-01-29?,Aston Villa,0.705,0.295,0.72,0.71,0.29,0.28,0.015,111873.116871,39539,KXUELGAME-26JAN29AVLRBS-AVL,YES_POLY__NO_KALSHI,0.005,True
433,Soccer,Go Ahead Eagles vs. SC Braga,GA Eagles vs Braga,uel-goa-scb-2026-01-29-scb,Will SC Braga win on 2026-01-29?,Braga,0.585,0.415,0.60,0.59,0.41,0.40,0.015,75363.804074,5759,KXUELGAME-26JAN29GAEBRA-BRA,YES_POLY__NO_KALSHI,0.005,True
23,Soccer,Manchester United FC vs. Fulham FC,Manchester United vs Fulham,epl-mun-ful-2026-02-01-ful,Will Fulham FC win on 2026-02-01?,Fulham,0.175,0.825,0.19,0.18,0.82,0.81,0.015,7708.556553,2799,KXEPLGAME-26FEB01MUNFUL-FUL,YES_POLY__NO_KALSHI,0.005,True
